In [ ]:
# Cell 1: Setup, Data Loading, and Phase 1.1 Analysis (Hit Quality Review - Evalue/Pident)
# (This cell remains the same - ensure it's run first)
# NOTE: This cell should load the database that HAS the Euk Hit info but LACKS coverage, 
# which is 'proteome_database_combined_v2.0.csv' based on user feedback.

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time
from collections import Counter
import glob # For finding files in a folder
import re # For parsing organism names
import sys # For exit
import logging # For detailed logging
from pathlib import Path # For handling paths
# Ensure Biopython is installed: pip install biopython
try:
    from Bio import SeqIO 
except ImportError:
    print("ERROR: Biopython is required for this script. Please install it: pip install biopython")
    sys.exit(1)

# --- Plotly Imports ---
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

# --- Setup Logging ---
log_formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger()
# Prevent adding handlers multiple times if script is re-run in interactive session
if not logger.hasHandlers():
    logger.setLevel(logging.INFO) 
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(log_formatter)
    logger.addHandler(console_handler)
else:
    # Ensure level is set if handlers already exist (e.g., in notebook re-run)
    logger.setLevel(logging.INFO) 

print("--- Notebook Setup: Asgard Eukaryotic Hit Validation - Phase 1 ---")

# --- Configuration ---
# INPUT file path: Use the database version that already has Euk annotations
# but is missing coverage columns, as indicated by the user.
db_with_annots_path = 'proteome_database_combined_v2.0.csv' 

# INPUT: Eukaryotic proteome FASTA file (needed for helper function definition in this cell)
euk_fasta_path = Path('euk63_proteomes_final.fasta') 

# Directory to save plots and summary data for this analysis phase
output_plot_dir_phase1 = 'output_plots_hit_validation_phase1'
output_summary_dir_phase1 = 'output_summary_data_hit_validation_phase1'

# Create output directories if they don't exist
# Use Path objects here for consistency
Path(output_plot_dir_phase1).mkdir(parents=True, exist_ok=True)
print(f"Ensured plot output directory exists: {output_plot_dir_phase1}")
Path(output_summary_dir_phase1).mkdir(parents=True, exist_ok=True)
print(f"Ensured summary data directory exists: {output_summary_dir_phase1}")


# Path to InterPro entry list file (ensure this file is in the correct location)
interpro_entry_path = 'interpro_entry.txt' 

# DIAMOND parsing parameters (for re-parsing in Cell 2)
diamond_results_folder_for_coverage = 'euk_diamond_search_results'
e_value_threshold_for_coverage = 1e-10 # Should match initial filtering used to generate hits in DB
diamond_col_names_for_coverage = [
    'qseqid', 'sseqid_full', 'pident', 'length', 'mismatch', 'gapopen',
    'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'qlen', 'slen'
]


# --- Define Key Column Names (Based on user provided list for v2.0) ---
protein_id_col = 'ProteinID'
# sequence_col = 'Sequence' # Not needed for this analysis
# length_col = 'Length' # Use Original_Seq_Length
source_dataset_col = 'Source_Dataset'
source_genome_accession_col = 'Source_Genome_Assembly_Accession'
source_protein_annotation_col = 'Source_Protein_Annotation'
ncbi_taxid_col = 'NCBI_TaxID'
asgard_phylum_col = 'Asgard_Phylum' 
virus_family_col = 'Virus_Family'
virus_name_col = 'Virus_Name'
orthogroup_col = 'Orthogroup'
ipr_col = 'IPR_Signatures' 
ipr_go_terms_col = 'IPR_GO_Terms'
uniprotkb_ac_col = 'UniProtKB_AC'
num_domains_col = 'Num_Domains'
domain_arch_col = 'Domain_Architecture' 
type_col = 'Type' # As in 'Annotated', 'Uncharacterized'
is_hypothetical_col = 'Is_Hypothetical'
has_known_structure_col = 'Has_Known_Structure' # Not used directly, Is_Structurally_Dark is derived
percent_disorder_col = 'Percent_Disorder'
specific_func_cat_col = 'Specific_Functional_Category'
broad_func_cat_col = 'Broad_Functional_Category' 
category_trigger_col = 'Category_Trigger'
signal_peptide_col = 'Signal_Peptide_USPNet'
sp_cleavage_site_col = 'SP_Cleavage_Site_USPNet'
original_seq_length_col = 'Original_Seq_Length' # Use this as the definitive query length
group_col = 'Group'
seqsearch_pdb_hit_col = 'SeqSearch_PDB_Hit'
seqsearch_afdb_hit_col = 'SeqSearch_AFDB_Hit'
has_reference_structure_col = 'Has_Reference_Structure'
localization_col = 'Predicted_Subcellular_Localization' 
mature_protein_sequence_col = 'Mature_Protein_Sequence'
mature_seq_length_col = 'Mature_Seq_Length'
seqsearch_mgnify_hit_col = 'SeqSearch_MGnify_Hit'
seqsearch_esma_hit_col = 'SeqSearch_ESMA_Hit'
structurally_dark_col = 'Is_Structurally_Dark'
esp_col = 'Is_ESP'

# DIAMOND Hit Columns (expected to be ALREADY in db_with_annots_path)
hit_flag_col = 'Has_Euk_DIAMOND_Hit'
euk_hit_sseqid_col = 'Euk_Hit_SSEQID'
euk_hit_organism_col = 'Euk_Hit_Organism' 
euk_hit_pident_col = 'Euk_Hit_PIDENT'
euk_hit_evalue_col = 'Euk_Hit_EVALUE'
euk_hit_protein_name_col = 'Euk_Hit_Protein_Name' 

# New DIAMOND Hit Detail Columns (to be ADDED in Cell 2)
euk_hit_qstart_col = 'Euk_Hit_Qstart'
euk_hit_qend_col = 'Euk_Hit_Qend'
euk_hit_sstart_col = 'Euk_Hit_Sstart'
euk_hit_send_col = 'Euk_Hit_Send'
# euk_hit_qlen_diamond_col = 'Euk_Hit_Qlen_Diamond' # We'll use Original_Seq_Length instead
euk_hit_slen_diamond_col = 'Euk_Hit_Slen_Diamond' # Length of Euk hit from DIAMOND
query_coverage_col = 'Query_Coverage' # (qend-qstart+1)/Original_Seq_Length
subject_coverage_col = 'Subject_Coverage' # (send-sstart+1)/slen

# Filtering criteria for high-quality hits (defined here for use in Cell 2)
min_query_coverage_filter = 0.70 
min_subject_coverage_filter = 0.50
min_pident_filter = 30.0 

# Output file for Cell 2 (DB with coverage added) - Use Path object
output_db_with_coverage_path = Path(output_summary_dir_phase1) / 'proteome_database_combined_v2.1_coverage.csv'

# --- Define Arcadia Color Palettes ---
print("\n--- Defining Manual Arcadia Color Palettes ---")
# (Color definitions remain the same as previous version)
arcadia_colors_manual = {
    "aegean": "#5088C5", "amber": "#F28360", "seaweed": "#3B9886", "canary": "#F7B846",
    "aster": "#7A77AB", "rose": "#F898AE", "vital": "#73B5E3", "tangerine": "#FFB984",
    "oat": "#F5E4BE", "wish": "#BABEE0", "lime": "#97CD78", "dragon": "#C85152",
    "sky": "#C6E7F4", "dress": "#F8C5C1", "taupe": "#DBD1C3", "denim": "#B6C8D4",
    "sage": "#B5BEA4", "mars": "#DA9085", "marine": "#8A99AD", "shell": "#EDE0D6",
    "white": "#FFFFFF", "gray": "#EBEDE8", "chateau": "#BAB0A8", "bark": "#8F8885",
    "slate": "#43413F", "charcoal": "#484B50", "crow": "#292928", "black": "#09090A",
    "forest": "#596F74", "parchment": "#FEF7F1", "zephyr": "#F4FBFF",
    "lichen": "#F7FBEF", "dawn": "#F8F4F1"
}
arcadia_primary_manual = [ arcadia_colors_manual[c] for c in ["aegean", "amber", "seaweed", "canary", "aster", "rose", "vital", "tangerine", "oat", "wish", "lime", "dragon"] ]
arcadia_secondary_manual = [ arcadia_colors_manual[c] for c in ["sky", "dress", "taupe", "denim", "sage", "mars", "marine", "shell"] ]
arcadia_neutrals_manual = [ arcadia_colors_manual[c] for c in ["gray", "chateau", "bark", "slate", "charcoal", "forest", "crow"] ]
print("Manual Arcadia palettes created.")

# --- Configure Plotly Defaults ---
print("\n--- Configuring Plotly Defaults ---")
pio.templates.default = "plotly_white"
print("Plotly default template set to 'plotly_white'.")

# --- Load InterPro Entry Data ---
print(f"\n--- Loading InterPro Entry Data from '{interpro_entry_path}' ---")
ipr_lookup = {} 
start_time_ipr = time.time()
# (IPR loading logic remains the same as previous version)
try:
    ipr_info_df = pd.read_csv( interpro_entry_path, sep='\t', usecols=[0, 1, 2], names=['IPR_ID', 'Type', 'Name'], header=0, comment='#', on_bad_lines='warn' )
    if 'ENTRY_AC' in ipr_info_df.columns: ipr_info_df.rename(columns={'ENTRY_AC': 'IPR_ID'}, inplace=True)
    if 'ENTRY_TYPE' in ipr_info_df.columns: ipr_info_df.rename(columns={'ENTRY_TYPE': 'Type'}, inplace=True)
    if 'ENTRY_NAME' in ipr_info_df.columns: ipr_info_df.rename(columns={'ENTRY_NAME': 'Name'}, inplace=True)
    if 'IPR_ID' in ipr_info_df.columns and 'Name' in ipr_info_df.columns and 'Type' in ipr_info_df.columns:
        ipr_info_df['IPR_ID'] = ipr_info_df['IPR_ID'].astype(str).str.strip()
        ipr_lookup = ipr_info_df.set_index('IPR_ID')[['Type', 'Name']].to_dict('index')
        print(f"Loaded InterPro entry data for {len(ipr_lookup)} entries in {time.time() - start_time_ipr:.2f} seconds.")
    else: print(f"Warning: Expected columns for IPR lookup not all found in '{interpro_entry_path}'.")
except FileNotFoundError: print(f"Warning: InterPro entry file not found at '{interpro_entry_path}'.")
except Exception as e: print(f"Warning: An error occurred loading or processing '{interpro_entry_path}': {e}")
if not ipr_lookup: print("Warning: ipr_lookup is empty. Domain name translations will not be available.")

# --- Define Helper Functions ---
print("\n--- Defining Helper Functions ---")
MAX_RAW_ARCH_LEN_CSV = 500 
print(f"Raw architecture strings in CSVs will be truncated to {MAX_RAW_ARCH_LEN_CSV} characters.")
# (Helper functions: translate_architecture, truncate_string, get_ipr_counts, clean_protein_name remain the same)
def translate_architecture(arch_string, lookup_dict=ipr_lookup): 
    if not isinstance(arch_string, str) or not arch_string or not lookup_dict: return arch_string
    processed_arch_string = str(arch_string).replace('|', ';'); ipr_ids = processed_arch_string.split(';')
    translated_parts = []
    for ipr_id in ipr_ids:
        ipr_id_clean = ipr_id.strip()
        if ipr_id_clean in lookup_dict: name = lookup_dict[ipr_id_clean].get('Name', ipr_id_clean); name = name[:40] + '...' if len(name) > 43 else name; translated_parts.append(f"{name} ({ipr_id_clean})")
        elif ipr_id_clean: translated_parts.append(ipr_id_clean)
    full_translation = "; ".join(translated_parts); max_len = 150 
    if len(full_translation) > max_len: full_translation = full_translation[:max_len-3] + "..."
    return full_translation
def truncate_string(text, max_len):
    if isinstance(text, str) and len(text) > max_len: return text[:max_len-3] + "..."
    return text
def get_ipr_counts(df, ipr_column_name):
    if ipr_column_name not in df.columns: print(f"Warning: IPR column '{ipr_column_name}' not found. Returning empty Counter."); return Counter()
    all_iprs_list = []
    for ipr_string_raw in df[ipr_column_name].dropna():
        processed_ipr_string = str(ipr_string_raw).replace('|', ';'); individual_iprs = processed_ipr_string.split(';')
        for ipr_item in individual_iprs:
            stripped_ipr = ipr_item.strip()
            if stripped_ipr and stripped_ipr.startswith("IPR"): all_iprs_list.append(stripped_ipr)
    return Counter(all_iprs_list)
def clean_protein_name(name):
    if pd.isna(name): return "Unknown/Not Found"; name = str(name).strip()
    name = re.sub(r'\s*\|\s*.*','', name); name = re.sub(r'\bisoform\s+[\w-]+\b', '', name, flags=re.IGNORECASE).strip()
    name = re.sub(r'\bpartial\b', '', name, flags=re.IGNORECASE).strip(); name = re.sub(r'\bputative\b', '', name, flags=re.IGNORECASE).strip()
    name = re.sub(r'\bpredicted protein\b', '', name, flags=re.IGNORECASE).strip(); name = re.sub(r'\btype\s+\w+\b', '', name, flags=re.IGNORECASE).strip()
    name = re.sub(r'\bprotein\b', '', name, flags=re.IGNORECASE).strip(); name = re.sub(r'\buncharacterized\b', 'Uncharacterized', name, flags=re.IGNORECASE).strip() 
    name = re.sub(r'[;,]$', '', name).strip(); name = re.sub(r'\s*\(Fragment\)$', '', name, flags=re.IGNORECASE).strip()
    name = re.sub(r'\s+', ' ', name).strip(); name = name[0].upper() + name[1:] if len(name) > 0 else name 
    return name if name else "Unknown/Not Found"
# Helper function for parsing FASTA headers (from add_euk_fasta_annotations_script)
def parse_euk_fasta_headers(fasta_file: Path) -> dict:
    """
    Parses a FASTA file to extract information from headers.
    Extracts ID before the first pipe '|' or space ' '.
    Extracts Genus species (first two words) for the organism name.
    """
    logging.info(f"Parsing FASTA headers from: {fasta_file}...")
    header_info = {}
    count = 0; skipped_count = 0
    if not fasta_file.is_file(): logging.error(f"FASTA file not found: {fasta_file}"); return header_info
    try:
        for record in SeqIO.parse(fasta_file, "fasta"):
            count += 1; description = record.description 
            first_pipe=description.find('|'); first_space=description.find(' '); end_of_id=-1
            if first_pipe!=-1 and first_space!=-1: end_of_id=min(first_pipe, first_space)
            elif first_pipe!=-1: end_of_id=first_pipe
            elif first_space!=-1: end_of_id=first_space
            fasta_id_key = description[:end_of_id].strip() if end_of_id!=-1 else description.strip()
            if not fasta_id_key: skipped_count+=1; logging.warning(f"Could not extract valid ID Key from header: {description[:100]}..."); continue
            genus_species_raw="Unknown Species"; protein_name="Unknown Protein"; parts=description.split('|')
            if len(parts)>=3: genus_species_raw=parts[1].strip(); protein_name=" | ".join(p.strip() for p in parts[2:]) 
            elif len(parts)==2: 
                second_part=parts[1].strip()
                if ' ' in second_part and not second_part.isupper(): genus_species_raw=second_part; protein_name="Unknown Protein (Genus/Species only in header)"
                else: genus_species_raw="Unknown Species (Name only in header)"; protein_name=second_part
            elif len(parts)==1: 
                 id_len = len(fasta_id_key)
                 first_space_after_id = description.find(' ', id_len)
                 if first_space_after_id != -1:
                     protein_name = description[id_len+1:].strip()
                     genus_species_raw = "Unknown Species (Space delimited header)"
                 else:
                     protein_name = "Unknown Protein (ID only header)"
                     genus_species_raw = "Unknown Species (ID only header)"
            genus_species_cleaned="Unknown Species"
            if genus_species_raw!="Unknown Species": 
                temp_name=genus_species_raw.replace('_', ' ').split()
                if len(temp_name)>=2: genus_species_cleaned=f"{temp_name[0]} {temp_name[1]}"
                elif len(temp_name)==1: genus_species_cleaned=temp_name[0] 
                else: genus_species_cleaned=genus_species_raw 
            header_info[fasta_id_key]={'Genus_Species': genus_species_cleaned, 'Protein_Name': protein_name}
            if count % 100000 == 0: logging.info(f"  Processed {count:,} FASTA records...")
    except Exception as e: logging.error(f"Error parsing FASTA file {fasta_file}: {e}", exc_info=True) 
    logging.info(f"Finished parsing. Extracted info for {len(header_info):,} sequences.")
    if skipped_count > 0: logging.warning(f"Skipped {skipped_count} records due to missing/unparseable IDs.")
    return header_info
# Helper function for plotting comparison bars (defined here for self-containment if needed)
def plot_comparison_bar(df_subset, df_baseline, column_name, title_suffix, color_map=None, category_orders=None):
    """Generates a grouped bar chart comparing percentages in a subset vs baseline."""
    if column_name not in df_subset.columns or column_name not in df_baseline.columns: print(f"  Skipping plot for '{column_name}': Column not found."); return
    subset_counts = df_subset[column_name].value_counts(normalize=True, dropna=False).mul(100); subset_counts.name = 'High-Quality Hits (%)'
    baseline_counts = df_baseline[column_name].value_counts(normalize=True, dropna=False).mul(100); baseline_counts.name = 'All Asgard (%)'
    df_compare = pd.concat([subset_counts, baseline_counts], axis=1).fillna(0).reset_index(); df_compare.rename(columns={df_compare.columns[0]: column_name}, inplace=True)
    df_melted = df_compare.melt(id_vars=column_name, var_name='Group', value_name='Percentage')
    xaxis_categoryorder=None; xaxis_categoryarray=None
    if category_orders and column_name in category_orders: cat_order = category_orders[column_name]; current_cats = df_melted[column_name].unique(); ordered_cats = [c for c in cat_order if c in current_cats]; missing_cats = [c for c in current_cats if c not in ordered_cats]; final_cat_order = ordered_cats + missing_cats; df_melted[column_name] = pd.Categorical(df_melted[column_name], categories=final_cat_order, ordered=True); df_melted = df_melted.sort_values(column_name); xaxis_categoryorder = 'array'; xaxis_categoryarray = final_cat_order
    else: default_order = baseline_counts.sort_values(ascending=False).index.tolist(); df_melted[column_name] = pd.Categorical(df_melted[column_name], categories=default_order, ordered=True); df_melted = df_melted.sort_values(column_name); xaxis_categoryorder = 'array'; xaxis_categoryarray = default_order
    print(f"\nComparison for '{column_name}':"); print(df_compare.round(1).to_markdown(index=False))
    fig = px.bar(df_melted, x=column_name, y='Percentage', color='Group', barmode='group', title=f'{column_name} Distribution: High-Quality Hits vs. All Asgard {title_suffix}', labels={'Percentage': '% of Proteins', column_name: column_name.replace('_', ' ').title()}, color_discrete_map={'High-Quality Hits (%)': arcadia_colors_manual.get('amber', '#F28360'), 'All Asgard (%)': arcadia_colors_manual.get('aegean', '#5088C5')})
    if xaxis_categoryorder == 'array': fig.update_xaxes(categoryorder=xaxis_categoryorder, categoryarray=xaxis_categoryarray)
    fig.show()

# --- Load Data ---
print(f"\n--- Loading Data from '{db_with_annots_path}' ---")
try:
    df_full = pd.read_csv(db_with_annots_path, low_memory=False)
    # Ensure ProteinID is string type right after loading
    if protein_id_col in df_full.columns:
         df_full[protein_id_col] = df_full[protein_id_col].astype(str)
    print(f"Successfully loaded database. Shape: {df_full.shape}")
except FileNotFoundError:
    print(f"ERROR: Database file not found at '{db_with_annots_path}'.")
    raise 
except Exception as e:
    print(f"An error occurred while loading the database: {e}")
    raise

# --- Create Color Maps for Plotting (after df_full is loaded) ---
print("\n--- Creating Color Maps ---")
# (Color map definitions remain the same)
# Asgard Phylum Colors
asgard_phylum_color_map = {}
if asgard_phylum_col in df_full.columns and group_col in df_full.columns:
    asgard_phyla_cat = df_full[df_full[group_col] == 'Asgard'][asgard_phylum_col].dropna().unique(); asgard_phyla_cat.sort() 
    if len(asgard_phyla_cat) > 0: asgard_phylum_color_map = {phylum: arcadia_primary_manual[i % len(arcadia_primary_manual)] for i, phylum in enumerate(asgard_phyla_cat)}
    asgard_phylum_color_map['Unknown Phylum'] = arcadia_colors_manual.get('gray', '#bdbdbd') 
print(f"Asgard phylum color map created for {len(asgard_phylum_color_map)} phyla.")
# Localization Colors
localization_color_map = {}
if localization_col in df_full.columns:
    all_localizations_cat = df_full[localization_col].astype('category').cat.categories.tolist(); localization_assignments = { 'Archaea: Cytoplasmic/Membrane (non-SP)': arcadia_colors_manual.get('aegean', '#5088C5'), 'Archaea: Membrane-associated (Lipoprotein/Pilin)': arcadia_colors_manual.get('amber', '#F28360'), 'Archaea: Secreted/Membrane (Sec/Tat pathway)': arcadia_colors_manual.get('seaweed', '#3B9886'), 'Host: Cytoplasm/Nucleus/Virus Factory': arcadia_colors_manual.get('vital', '#73B5E3'), 'Host: Membrane-associated (Lipoprotein/Pilin-like)': arcadia_colors_manual.get('tangerine', '#FFB984'), 'Host: Secretory Pathway (Secreted/Membrane/Organelle)': arcadia_colors_manual.get('lime', '#97CD78'), 'CYTOPLASMIC': arcadia_colors_manual.get('vital', '#73B5E3'), 'MEMBRANE': arcadia_colors_manual.get('tangerine', '#FFB984'), 'EXTRACELLULAR': arcadia_colors_manual.get('lime', '#97CD78'), 'Unknown': arcadia_colors_manual.get('gray', '#EBEDE8'),}; fallback_palette_loc = arcadia_neutrals_manual + arcadia_secondary_manual; fallback_idx_loc = 0
    for loc in all_localizations_cat:
        if loc not in localization_color_map: localization_color_map[loc] = localization_assignments.get(loc, fallback_palette_loc[fallback_idx_loc % len(fallback_palette_loc)])
        if loc not in localization_assignments: fallback_idx_loc += 1
print(f"Localization color map created for {len(localization_color_map)} categories.")
# Broad Functional Category Colors
broad_category_color_map = {}
if broad_func_cat_col in df_full.columns:
    all_broad_categories_cat = df_full[broad_func_cat_col].astype('category').cat.categories.tolist(); category_assignments = { 'Cytoskeleton': arcadia_colors_manual.get('aegean', '#5088C5'), 'Membrane Trafficking/Vesicles': arcadia_colors_manual.get('amber', '#F28360'), 'ESCRT/Endosomal Sorting': arcadia_colors_manual.get('seaweed', '#3B9886'), 'Ubiquitin System': arcadia_colors_manual.get('aster', '#7A77AB'), 'N-glycosylation': arcadia_colors_manual.get('rose', '#F898AE'), 'Nuclear Transport/Pore': arcadia_colors_manual.get('vital', '#73B5E3'), 'DNA Info Processing': arcadia_colors_manual.get('canary', '#F7B846'), 'RNA Info Processing': arcadia_colors_manual.get('lime', '#97CD78'), 'Translation': arcadia_colors_manual.get('tangerine', '#FFB984'), 'Signal Transduction': arcadia_colors_manual.get('aster', '#7A77AB'), 'Metabolism': arcadia_colors_manual.get('sky', '#C6E7F4'), 'Other Specific Annotation': arcadia_colors_manual.get('denim', '#B6C8D4'), 'Unknown/Unclassified': arcadia_colors_manual.get('gray', '#EBEDE8'),}; fallback_palette_cat = arcadia_primary_manual + arcadia_secondary_manual + arcadia_neutrals_manual; fallback_idx_cat = 0
    for category in all_broad_categories_cat:
        if category not in broad_category_color_map: broad_category_color_map[category] = category_assignments.get(category, fallback_palette_cat[fallback_idx_cat % len(fallback_palette_cat)])
        if category not in category_assignments: fallback_idx_cat += 1
print(f"Broad functional category color map created for {len(broad_category_color_map)} categories.")


# --- Filter for Asgard Proteins with Eukaryotic Hits (Initial Filter) ---
print(f"\n--- Filtering for Asgard Proteins with Eukaryotic DIAMOND Hits ---")
if group_col not in df_full.columns or hit_flag_col not in df_full.columns:
    print(f"ERROR: Required columns ('{group_col}' or '{hit_flag_col}') not found. Cannot proceed.")
    df_asgard_hits = pd.DataFrame()
else:
    df_asgard_hits = df_full[
        (df_full[group_col] == 'Asgard') & (df_full[hit_flag_col] == True)
    ].copy()
    print(f"Found {len(df_asgard_hits)} Asgard proteins with eukaryotic hits for initial review.")
    if df_asgard_hits.empty: print("No Asgard proteins with eukaryotic hits found.")

# --- Phase 1.1: Review E-value, Percent Identity ---
if not df_asgard_hits.empty:
    print(f"\n\n--- Phase 1.1: Reviewing E-value and Percent Identity for Asgard Eukaryotic Hits ---")
    # (Plots are commented out for brevity, uncomment if needed)
    print(f"\nAnalyzing distribution of '{euk_hit_evalue_col}'...")
    if euk_hit_evalue_col in df_asgard_hits.columns:
        print(df_asgard_hits[euk_hit_evalue_col].describe())
        # df_asgard_hits['log10_evalue'] = np.log10(df_asgard_hits[euk_hit_evalue_col] + 1e-200) 
        # fig_evalue_hist = px.histogram(...) fig_evalue_hist.show()
        # fig_evalue_box = px.box(...) fig_evalue_box.show()
    print(f"\nAnalyzing distribution of '{euk_hit_pident_col}'...")
    if euk_hit_pident_col in df_asgard_hits.columns:
        print(df_asgard_hits[euk_hit_pident_col].describe())
        # fig_pident_hist = px.histogram(...) fig_pident_hist.show()
        # fig_pident_box = px.box(...) fig_pident_box.show()
else:
    print("\nNo Asgard proteins with eukaryotic hits to analyze for Phase 1.1 (Evalue/Pident).")

print("\n--- Cell 1 (Setup & Evalue/Pident Review) Complete ---\n")

In [ ]:
# Cell 2: Add Coverage & Define High-Quality Hits
# UPDATED: Loads v2.0, adds only coverage columns, saves as v2.1
print("\n\n--- Cell 2: Add Coverage & Define High-Quality Hits ---")

# --- Step 2.1: Re-parse DIAMOND results to get alignment coordinates and subject lengths ---
print(f"\nRe-parsing DIAMOND results from folder '{diamond_results_folder_for_coverage}' to extract alignment details...")
best_hit_alignment_details = {} 
raw_diamond_files_coverage = glob.glob(os.path.join(diamond_results_folder_for_coverage, '*_diamond_hits.tsv'))

if not raw_diamond_files_coverage:
    logging.warning(f"No '*_diamond_hits.tsv' files found in '{diamond_results_folder_for_coverage}'. Cannot calculate coverage.")
else:
    logging.info(f"Found {len(raw_diamond_files_coverage)} DIAMOND result files to process for coverage.")
    df_diamond_raw_list_coverage = []
    # Use df_full (loaded in Cell 1) to get the list of ProteinIDs that need coverage info
    # This avoids parsing hits for proteins not in our main DB (e.g., GVs if running only Asgard analysis)
    target_protein_ids = set(df_full[protein_id_col].astype(str))
    logging.info(f"Will extract coverage details for up to {len(target_protein_ids)} target ProteinIDs.")
    
    processed_files = 0
    for f_path in raw_diamond_files_coverage:
        try:
            # Read only necessary columns
            df_temp = pd.read_csv(f_path, sep='\t', header=None, 
                                  names=diamond_col_names_for_coverage, 
                                  usecols=['qseqid', 'evalue', 'bitscore', 'qstart', 'qend', 'sstart', 'send', 'slen'])
            df_temp['qseqid'] = df_temp['qseqid'].astype(str) # Ensure qseqid is string
            # Filter early for relevant IDs and evalue threshold
            df_temp = df_temp[df_temp['qseqid'].isin(target_protein_ids) & (df_temp['evalue'] <= e_value_threshold_for_coverage)]
            if not df_temp.empty:
                 df_diamond_raw_list_coverage.append(df_temp)
            processed_files += 1
            if processed_files % 500 == 0:
                 logging.info(f"  Processed {processed_files}/{len(raw_diamond_files_coverage)} DIAMOND files...")

        except Exception as e:
            logging.error(f"  Error reading or processing file {f_path} for coverage: {e}")
            continue

    if df_diamond_raw_list_coverage:
        df_diamond_all_hits_coverage = pd.concat(df_diamond_raw_list_coverage, ignore_index=True)
        logging.info(f"  Combined {len(df_diamond_all_hits_coverage)} relevant DIAMOND alignments passing e-value for coverage analysis.")
        
        # No need to filter by e-value again, already done during read
        
        if not df_diamond_all_hits_coverage.empty:
            # Sort to get the best hit (lowest e-value, highest bitscore)
            df_diamond_all_hits_coverage.sort_values(by=['qseqid', 'evalue', 'bitscore'], ascending=[True, True, False], inplace=True)
            df_best_hits_for_coverage = df_diamond_all_hits_coverage.drop_duplicates(subset=['qseqid'], keep='first').copy()
            logging.info(f"  Identified {len(df_best_hits_for_coverage)} unique query proteins with best hit details for coverage.")
            
            # Store only the necessary details for coverage calculation
            for _, row in df_best_hits_for_coverage.iterrows():
                query_id = row['qseqid'] 
                # Convert coordinates to int, handle potential errors robustly
                try:
                    qstart = int(row['qstart'])
                    qend = int(row['qend'])
                    sstart = int(row['sstart'])
                    send = int(row['send'])
                    slen = int(row['slen'])
                    
                    best_hit_alignment_details[query_id] = {
                        euk_hit_qstart_col: qstart, 
                        euk_hit_qend_col: qend,
                        euk_hit_sstart_col: sstart, 
                        euk_hit_send_col: send,
                        euk_hit_slen_diamond_col: slen
                    }
                except (ValueError, TypeError) as conv_err:
                     logging.warning(f"Could not convert alignment details to int for qseqid {query_id}. Skipping. Error: {conv_err}. Row data: {row.to_dict()}")
                     continue
        else: 
            logging.info("  No DIAMOND hits passed the e-value threshold after combining files.")
    else: 
        logging.info("  No valid DIAMOND alignments loaded for coverage analysis.")


# --- Step 2.2: Merge Alignment Details into df_full (loaded from v2.0) ---
logging.info("\nMerging alignment details into main DataFrame...")
if best_hit_alignment_details:
    df_alignment_details_to_merge = pd.DataFrame.from_dict(best_hit_alignment_details, orient='index')
    df_alignment_details_to_merge.index.name = protein_id_col
    # Ensure index is string type for merging
    df_alignment_details_to_merge.index = df_alignment_details_to_merge.index.astype(str) 
    
    # Define the columns to merge 
    cols_to_add_from_diamond = [
        euk_hit_qstart_col, euk_hit_qend_col, euk_hit_sstart_col, euk_hit_send_col,
        euk_hit_slen_diamond_col 
    ]
    # Drop these columns from df_full if they already exist from a previous run
    cols_exist = [col for col in cols_to_add_from_diamond if col in df_full.columns]
    if cols_exist:
        logging.info(f"  Dropping existing alignment detail columns before merge: {cols_exist}")
        df_full = df_full.drop(columns=cols_exist)
            
    # Ensure ProteinID in df_full is also string before merging
    if protein_id_col in df_full.columns:
         df_full[protein_id_col] = df_full[protein_id_col].astype(str)
    else:
         logging.error(f"Critical Error: '{protein_id_col}' not found in df_full. Cannot merge.")
         sys.exit(1) # Stop execution

    df_full = df_full.merge(df_alignment_details_to_merge, on=protein_id_col, how='left')
    logging.info(f"  Merged alignment details. df_full shape: {df_full.shape}")
else:
    logging.warning("No alignment details found to merge. Coverage calculation will fail.")
    # Add empty columns if they weren't added
    for col in [euk_hit_qstart_col, euk_hit_qend_col, euk_hit_sstart_col, euk_hit_send_col, euk_hit_slen_diamond_col]:
        if col not in df_full.columns: df_full[col] = np.nan

# --- Step 2.3: Calculate Query and Subject Coverage ---
logging.info("\nCalculating Query and Subject Coverage...")
# Check if necessary columns exist
can_calc_q_coverage = all(c in df_full.columns for c in [euk_hit_qstart_col, euk_hit_qend_col, original_seq_length_col])
can_calc_s_coverage = all(c in df_full.columns for c in [euk_hit_sstart_col, euk_hit_send_col, euk_hit_slen_diamond_col])

# Initialize coverage columns if they don't exist
if query_coverage_col not in df_full.columns: df_full[query_coverage_col] = np.nan
if subject_coverage_col not in df_full.columns: df_full[subject_coverage_col] = np.nan

if can_calc_q_coverage:
    # Convert columns to numeric, coercing errors
    qstart = pd.to_numeric(df_full[euk_hit_qstart_col], errors='coerce')
    qend = pd.to_numeric(df_full[euk_hit_qend_col], errors='coerce')
    qlen = pd.to_numeric(df_full[original_seq_length_col], errors='coerce')
    
    # Calculate coverage only where all components are valid numbers and qlen > 0
    valid_q_mask = qstart.notna() & qend.notna() & qlen.notna() & (qlen > 0)
    # Ensure indices align before calculation using .loc
    df_full.loc[valid_q_mask, query_coverage_col] = (qend[valid_q_mask] - qstart[valid_q_mask] + 1) / qlen[valid_q_mask]
    logging.info(f"  Calculated '{query_coverage_col}'. NaN count: {df_full[query_coverage_col].isna().sum()}")
else:
    logging.warning(f"  Cannot calculate '{query_coverage_col}' due to missing columns.")
    

if can_calc_s_coverage:
    sstart = pd.to_numeric(df_full[euk_hit_sstart_col], errors='coerce')
    ssend = pd.to_numeric(df_full[euk_hit_send_col], errors='coerce')
    slen = pd.to_numeric(df_full[euk_hit_slen_diamond_col], errors='coerce')

    # Calculate coverage only where all components are valid numbers and slen > 0
    valid_s_mask = sstart.notna() & ssend.notna() & slen.notna() & (slen > 0)
    # Ensure indices align before calculation using .loc
    df_full.loc[valid_s_mask, subject_coverage_col] = (ssend[valid_s_mask] - sstart[valid_s_mask] + 1) / slen[valid_s_mask]
    logging.info(f"  Calculated '{subject_coverage_col}'. NaN count: {df_full[subject_coverage_col].isna().sum()}")
else:
    logging.warning(f"  Cannot calculate '{subject_coverage_col}' due to missing columns.")
    

# --- Step 2.4: Define High-Quality Hits DataFrame ---
logging.info(f"\n--- Defining High-Quality Hits DataFrame ---")
# Use the filter criteria defined in Cell 1
required_filter_cols = [group_col, hit_flag_col, query_coverage_col, subject_coverage_col, euk_hit_pident_col]
if all(col in df_full.columns for col in required_filter_cols):
     # Ensure coverage columns are numeric before filtering (might be redundant but safe)
     df_full[query_coverage_col] = pd.to_numeric(df_full[query_coverage_col], errors='coerce')
     df_full[subject_coverage_col] = pd.to_numeric(df_full[subject_coverage_col], errors='coerce')
     
     # Filter df_full (which now has coverage) to create df_high_quality_hits
     df_high_quality_hits = df_full[
        (df_full[group_col] == 'Asgard') & 
        (df_full[hit_flag_col] == True) &
        (df_full[query_coverage_col].notna()) & # Ensure coverage was calculated
        (df_full[subject_coverage_col].notna()) &
        (df_full[query_coverage_col] >= min_query_coverage_filter) &
        (df_full[subject_coverage_col] >= min_subject_coverage_filter) &
        (df_full[euk_hit_pident_col] >= min_pident_filter)
    ].copy() # IMPORTANT: Use .copy() to avoid SettingWithCopyWarning later
     logging.info(f"Defined 'df_high_quality_hits' with {len(df_high_quality_hits)} rows based on:")
     logging.info(f"  Query Coverage >= {min_query_coverage_filter*100:.0f}%")
     logging.info(f"  Subject Coverage >= {min_subject_coverage_filter*100:.0f}%")
     logging.info(f"  Percent Identity >= {min_pident_filter}%")
else:
     logging.error(f"Cannot define 'df_high_quality_hits', missing required columns: {[c for c in required_filter_cols if c not in df_full.columns]}")
     # Define as empty df to allow subsequent cells to check .empty
     df_high_quality_hits = pd.DataFrame(columns=df_full.columns if 'df_full' in locals() else []) 


# --- Step 2.5: Analyze Coverage Distributions (Optional Plotting) ---
if not df_high_quality_hits.empty:
    logging.info(f"\n--- Analyzing Alignment Coverage Distributions for High-Quality Hits---")
    # (Plots are commented out for brevity, uncomment if needed)
    logging.info(f"\nDistribution of '{query_coverage_col}':")
    print(df_high_quality_hits[query_coverage_col].describe())
    # fig_qcov_hist = px.histogram(...) fig_qcov_hist.show()
    logging.info(f"\nDistribution of '{subject_coverage_col}':")
    print(df_high_quality_hits[subject_coverage_col].describe())
    # fig_scov_hist = px.histogram(...) fig_scov_hist.show()
else:
    logging.warning("No high-quality hits defined, skipping coverage distribution analysis.")

# --- Step 2.6: Save the updated df_full with coverage ---
# This DataFrame now contains the original v2.0 info PLUS the coverage columns
logging.info(f"\nSaving updated database with coverage info to: {output_db_with_coverage_path}")
try:
    # Ensure output directory exists - Use Path object methods
    output_db_with_coverage_path.parent.mkdir(parents=True, exist_ok=True)
    # Save the df_full dataframe which contains ALL proteins, now updated with coverage
    df_full.to_csv(output_db_with_coverage_path, index=False)
    logging.info(f"Successfully saved updated database (df_full) to '{output_db_with_coverage_path}'.")
except Exception as e:
    logging.error(f"Failed to save updated database with coverage: {e}")


print("\n\n--- Cell 2 (Add Coverage & Define High-Quality Hits) Complete ---")
print("The DataFrame 'df_full' has been updated with alignment details and coverage.")
print("The DataFrame 'df_high_quality_hits' is now defined based on filtering criteria.")
print(f"The fully updated database (df_full) is saved as '{output_db_with_coverage_path}'.")
print("Subsequent cells can now use 'df_high_quality_hits'.")

In [ ]:
# Cell 3: Phase 2.3 - Examine Taxonomic Distribution of Eukaryotic Hits

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time
 
# --- Plotly Imports ---
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

print("\n\n--- Cell 3: Phase 2.3 - Examine Taxonomic Distribution of Eukaryotic Hits ---")

# --- Assumptions ---
# This cell assumes Cell 1 and Cell 2 have been run successfully.
# It relies on the following DataFrames being available and populated:
#   - df_full: The main DataFrame loaded in Cell 1.
#   - df_asgard_hits: Asgard proteins with any hit (from Cell 1/2).
#   - df_high_quality_hits: Asgard proteins filtered for hit quality in Cell 2.
# It also relies on column name variables (e.g., euk_hit_organism_col, esp_col),
# color palettes (e.g., arcadia_primary_manual, arcadia_secondary_manual),
# and the output directory variables (e.g., output_summary_dir_phase1) defined in Cell 1.

# --- Check if necessary DataFrames exist ---
if 'df_high_quality_hits' not in locals():
    print("ERROR: DataFrame 'df_high_quality_hits' not found. Please run Cell 2 first.")
    if 'df_asgard_hits' in locals() and not df_asgard_hits.empty:
        print("WARNING: Using 'df_asgard_hits' (all hits) for taxonomic analysis as 'df_high_quality_hits' is missing.")
        df_to_analyze_taxa = df_asgard_hits
    else:
        print("ERROR: Neither 'df_high_quality_hits' nor 'df_asgard_hits' are available. Skipping taxonomic analysis.")
        df_to_analyze_taxa = pd.DataFrame() 
elif df_high_quality_hits.empty:
     print("WARNING: DataFrame 'df_high_quality_hits' is empty (likely due to filtering criteria in Cell 2 or no initial hits).")
     print("         Attempting to use 'df_asgard_hits' if available, otherwise skipping taxonomic analysis.")
     if 'df_asgard_hits' in locals() and not df_asgard_hits.empty:
        df_to_analyze_taxa = df_asgard_hits
        print("         Using 'df_asgard_hits' (all hits passing e-value) for taxonomic analysis.")
     else:
        df_to_analyze_taxa = pd.DataFrame() 
else:
    df_to_analyze_taxa = df_high_quality_hits
    print("Using 'df_high_quality_hits' (filtered by coverage/pident) for taxonomic analysis.")

# --- Perform Taxonomic Analysis ---
if not df_to_analyze_taxa.empty and euk_hit_organism_col in df_to_analyze_taxa.columns:
    print(f"\nAnalyzing distribution of '{euk_hit_organism_col}' for {len(df_to_analyze_taxa)} Asgard hits...")

    # Clean organism names (remove potential leading/trailing spaces)
    df_to_analyze_taxa[euk_hit_organism_col] = df_to_analyze_taxa[euk_hit_organism_col].astype(str).str.strip()
    
    # Replace empty strings or common placeholders with 'Unknown'
    df_to_analyze_taxa[euk_hit_organism_col] = df_to_analyze_taxa[euk_hit_organism_col].replace(['', 'nan', 'None', 'Unknown'], 'Unknown Organism')

    # 1. Overall Top Hit Organisms
    top_n_orgs = 30 # How many top organisms to show/color
    organism_counts = df_to_analyze_taxa[euk_hit_organism_col].value_counts()
    
    print(f"\nTop {top_n_orgs} Eukaryotic Hit Organisms (from selected hits):")
    try:
        print(organism_counts.head(top_n_orgs).to_markdown())
    except ImportError:
        print(organism_counts.head(top_n_orgs))

    # --- Create Color Map for Top Eukaryotic Organisms ---
    print(f"\nCreating color map for top {top_n_orgs} eukaryotic organisms...")
    euk_organism_color_map = {}
    top_org_list = organism_counts.head(top_n_orgs).index.tolist()
    # Combine primary and secondary palettes for more colors
    euk_palette = arcadia_primary_manual + arcadia_secondary_manual 
    
    for i, org_name in enumerate(top_org_list):
        if org_name == 'Unknown Organism':
             # Assign a specific neutral color to 'Unknown Organism' if it's in the top list
             euk_organism_color_map[org_name] = arcadia_colors_manual.get('gray', '#bdbdbd')
        else:
            # Cycle through the combined palette
            euk_organism_color_map[org_name] = euk_palette[i % len(euk_palette)]
            
    # Assign a default color for organisms not in the top N (optional, Plotly handles this)
    # default_other_color = arcadia_colors_manual.get('chateau', '#BAB0A8') 
    print(f"Color map created for {len(euk_organism_color_map)} organisms.")

    # Plot Overall Distribution using the color map
    df_org_plot = organism_counts.head(top_n_orgs).reset_index()
    df_org_plot.columns = [euk_hit_organism_col, 'Count'] 

    fig_org_overall = px.bar(
        df_org_plot,
        x=euk_hit_organism_col,
        y='Count',
        color=euk_hit_organism_col, # Color bars by organism
        color_discrete_map=euk_organism_color_map, # Apply the created color map
        title=f'Top {top_n_orgs} Eukaryotic Hit Organisms for Selected Asgard Hits',
        labels={euk_hit_organism_col: 'Eukaryotic Organism (Best Hit)', 'Count': 'Number of Asgard Proteins Hit'}
    )
    fig_org_overall.update_xaxes(tickangle=45, categoryorder='total descending') 
    fig_org_overall.update_layout(showlegend=False) # Hide legend if too many items
    fig_org_overall.show()

    # 2. Top Hit Organisms for ESPs
    if esp_col in df_to_analyze_taxa.columns:
        df_esps_hits_subset = df_to_analyze_taxa[df_to_analyze_taxa[esp_col] == True]
        if not df_esps_hits_subset.empty:
            print(f"\nAnalyzing distribution of '{euk_hit_organism_col}' for {len(df_esps_hits_subset)} Asgard ESP hits...")
            organism_counts_esps = df_esps_hits_subset[euk_hit_organism_col].value_counts()
            
            print(f"\nTop {top_n_orgs} Eukaryotic Hit Organisms (Selected ESP Hits):")
            try:
                print(organism_counts_esps.head(top_n_orgs).to_markdown())
            except ImportError:
                print(organism_counts_esps.head(top_n_orgs))

            # Plot ESP Distribution using the SAME color map for consistency
            df_org_plot_esps = organism_counts_esps.head(top_n_orgs).reset_index()
            df_org_plot_esps.columns = [euk_hit_organism_col, 'Count']

            fig_org_esps = px.bar(
                df_org_plot_esps,
                x=euk_hit_organism_col,
                y='Count',
                color=euk_hit_organism_col, # Color bars by organism
                color_discrete_map=euk_organism_color_map, # Apply the created color map
                title=f'Top {top_n_orgs} Eukaryotic Hit Organisms for Selected Asgard ESP Hits',
                labels={euk_hit_organism_col: 'Eukaryotic Organism (Best Hit)', 'Count': 'Number of Asgard ESPs Hit'}
            )
            fig_org_esps.update_xaxes(tickangle=45, categoryorder='total descending')
            fig_org_esps.update_layout(showlegend=False) # Hide legend if too many items
            fig_org_esps.show()

            # Save ESP organism counts
            esp_org_counts_filename = os.path.join(output_summary_dir_phase1, "esp_euk_hit_organism_counts.csv")
            try:
                # Save the full list, not just top N
                organism_counts_esps.reset_index().rename(columns={'index':euk_hit_organism_col, euk_hit_organism_col:'Count'}).to_csv(esp_org_counts_filename, index=False)
                print(f"Saved ESP hit organism counts to '{esp_org_counts_filename}'")
            except Exception as e:
                print(f"Error saving ESP hit organism counts: {e}")
        else:
            print("\nNo hits found for Asgard ESPs in the selected subset to analyze taxonomically.")
    else:
        print(f"\nSkipping ESP-specific taxonomic analysis (column '{esp_col}' not found in the selected hits).")

    # Save Overall organism counts
    overall_org_counts_filename = os.path.join(output_summary_dir_phase1, "overall_euk_hit_organism_counts.csv")
    try:
        # Save the full list
        organism_counts.reset_index().rename(columns={'index':euk_hit_organism_col, euk_hit_organism_col:'Count'}).to_csv(overall_org_counts_filename, index=False)
        print(f"Saved overall hit organism counts to '{overall_org_counts_filename}'")
    except Exception as e:
        print(f"Error saving overall hit organism counts: {e}")

else:
    print("\nNo Asgard hits available in the selected DataFrame ('df_to_analyze_taxa') to analyze eukaryotic organism distribution.")


print("\n\n--- Cell 3 (Taxonomic Distribution Analysis) Complete ---")
print("Next steps from the plan would involve:")
print("  - Phase 2.1: Comparing Domain Architectures (requires Euk hit domain info).")
print("  - Phase 2.2: Analyzing Functional Annotations (requires fetching Euk hit functions).")
print("  - Phase 2.4: Cross-referencing with external orthology data.")



In [ ]:
# Cell 4: Statistical Profiling of Asgard Proteins with High-Quality Eukaryotic Hits

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time

# --- Plotly Imports ---
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

print("\n\n--- Cell 4: Profiling Asgard Proteins with High-Quality Eukaryotic Hits ---")

# --- Assumptions ---
# This cell assumes Cell 1, 2, and 3 have been run successfully.
# It relies on the following DataFrames being available and populated:
#   - df_full: The main DataFrame loaded in Cell 1.
#   - df_high_quality_hits: Asgard proteins filtered for hit quality in Cell 2.
#   - df_asgard_all: Baseline Asgard proteins from df_full (created in this cell if needed)
# It also relies on column name variables (e.g., group_col, structurally_dark_col, 
# broad_func_cat_col, esp_col, num_domains_col) and color maps (arcadia_colors_manual) defined in Cell 1.

# --- Check if necessary DataFrames exist ---
if 'df_high_quality_hits' not in locals():
    print("ERROR: DataFrame 'df_high_quality_hits' not found. Please run Cell 2 first.")
    # Stop or create empty df to prevent errors
    df_high_quality_hits = pd.DataFrame() 
elif df_high_quality_hits.empty:
     print("WARNING: DataFrame 'df_high_quality_hits' is empty. Profiling results will be empty.")
     # Continue with empty df for now, but analysis will show 0 counts

if 'df_full' not in locals():
     print("ERROR: DataFrame 'df_full' not found. Cannot calculate baseline Asgard profiles. Please run Cell 1.")
     # Stop or create empty df
     df_asgard_all = pd.DataFrame()
else:
     # Get the baseline Asgard DataFrame if not already done
     if 'df_asgard_all' not in locals() or df_asgard_all.empty:
          print("Creating baseline df_asgard_all from df_full.")
          df_asgard_all = df_full[df_full[group_col] == 'Asgard'].copy()
     
     if df_asgard_all.empty:
         print("WARNING: No Asgard proteins found in the main DataFrame ('df_full'). Baseline comparison not possible.")


# --- Helper Function for Comparative Profiling (Corrected) ---
def plot_comparison_bar(df_subset, df_baseline, column_name, title_suffix, color_map=None, category_orders=None):
    """Generates a grouped bar chart comparing percentages in a subset vs baseline."""
    # Ensure color_map uses the correct keys if provided (it's not used in px.bar here, but good practice)
    # This function now ignores the passed color_map for the px.bar call itself,
    # as coloring is fixed to the 'Group' column.
    
    if column_name not in df_subset.columns or column_name not in df_baseline.columns:
        print(f"  Skipping plot for '{column_name}': Column not found in one or both DataFrames.")
        return

    # Calculate percentages for the subset (high-quality hits)
    subset_counts = df_subset[column_name].value_counts(normalize=True, dropna=False).mul(100)
    subset_counts.name = 'High-Quality Hits (%)'

    # Calculate percentages for the baseline (all Asgard)
    baseline_counts = df_baseline[column_name].value_counts(normalize=True, dropna=False).mul(100)
    baseline_counts.name = 'All Asgard (%)'

    # Combine the percentages
    df_compare = pd.concat([subset_counts, baseline_counts], axis=1).fillna(0).reset_index()
    # Rename index column safely
    df_compare.rename(columns={df_compare.columns[0]: column_name}, inplace=True)
    
    # Melt for plotting with plotly express
    df_melted = df_compare.melt(id_vars=column_name, var_name='Group', value_name='Percentage')

    # Sort categories if order provided, otherwise sort by baseline percentage
    xaxis_categoryorder = None
    xaxis_categoryarray = None
    if category_orders and column_name in category_orders:
         cat_order = category_orders[column_name]
         # Ensure all values in df_melted are in the category order list
         current_cats = df_melted[column_name].unique()
         ordered_cats = [c for c in cat_order if c in current_cats]
         # Add any missing categories from data to the end
         missing_cats = [c for c in current_cats if c not in ordered_cats]
         final_cat_order = ordered_cats + missing_cats
         
         df_melted[column_name] = pd.Categorical(df_melted[column_name], categories=final_cat_order, ordered=True)
         df_melted = df_melted.sort_values(column_name) # Sort based on the defined category order
         xaxis_categoryorder = 'array' 
         xaxis_categoryarray = final_cat_order # Use the final computed order
    else:
        # Default sort: Order categories by the baseline percentage descending for better viz
        default_order = baseline_counts.sort_values(ascending=False).index.tolist()
        df_melted[column_name] = pd.Categorical(df_melted[column_name], categories=default_order, ordered=True)
        df_melted = df_melted.sort_values(column_name)
        xaxis_categoryorder = 'array' # Use the order defined by the categorical type
        xaxis_categoryarray = default_order


    print(f"\nComparison for '{column_name}':")
    try:
        print(df_compare.round(1).to_markdown(index=False))
    except ImportError:
        print(df_compare.round(1))

    # Create the plot (Corrected: Removed second color_discrete_map)
    fig = px.bar(
        df_melted,
        x=column_name,
        y='Percentage',
        color='Group', # Colors bars based on 'Group' column
        barmode='group',
        title=f'{column_name} Distribution: High-Quality Hits vs. All Asgard {title_suffix}',
        labels={'Percentage': '% of Proteins', column_name: column_name.replace('_', ' ').title()}, # Nicer axis label
        color_discrete_map={'High-Quality Hits (%)': arcadia_colors_manual.get('amber', '#F28360'), 
                            'All Asgard (%)': arcadia_colors_manual.get('aegean', '#5088C5')}
        # Removed the problematic line: **({f'color_discrete_map': color_map} ... )
    )
    
    # Apply category order to x-axis if needed (overrides default plotly sorting)
    if xaxis_categoryorder == 'array':
         fig.update_xaxes(categoryorder=xaxis_categoryorder, categoryarray=xaxis_categoryarray)
    
    fig.show()


# --- Perform Profiling ---

if not df_high_quality_hits.empty and not df_asgard_all.empty:
    print(f"\n--- Profiling {len(df_high_quality_hits)} Asgard proteins with high-quality eukaryotic hits ---")

    # 1. Structural Darkness Profile
    if structurally_dark_col in df_high_quality_hits.columns:
        plot_comparison_bar(
            df_subset=df_high_quality_hits,
            df_baseline=df_asgard_all,
            column_name=structurally_dark_col,
            title_suffix="(True = Dark)"
            # No color_map needed here as x-axis is boolean
        )
    else:
        print(f"Skipping Structural Darkness profile: Column '{structurally_dark_col}' not found.")

    # 2. Broad Functional Category Profile
    if broad_func_cat_col in df_high_quality_hits.columns:
        # Get category order based on baseline abundance
        baseline_cat_order = df_asgard_all[broad_func_cat_col].value_counts().index.tolist()
        plot_comparison_bar(
            df_subset=df_high_quality_hits,
            df_baseline=df_asgard_all,
            column_name=broad_func_cat_col,
            title_suffix="",
            # color_map=broad_category_color_map, # Not used by px.bar in this setup
            category_orders = {broad_func_cat_col: baseline_cat_order} # Pass the order for sorting x-axis
        )
    else:
        print(f"Skipping Broad Functional Category profile: Column '{broad_func_cat_col}' not found.")

    # 3. ESP Profile
    if esp_col in df_high_quality_hits.columns:
        plot_comparison_bar(
            df_subset=df_high_quality_hits,
            df_baseline=df_asgard_all,
            column_name=esp_col,
            title_suffix="(True = ESP)"
            # No color_map needed here
        )
    else:
        print(f"Skipping ESP profile: Column '{esp_col}' not found.")

    # 4. Domain-less Profile
    if num_domains_col in df_high_quality_hits.columns:
        # Create a temporary boolean column: True if domain-less (NaN), False otherwise
        domainless_col_temp = 'Is_DomainLess'
        # Ensure the temporary column is added before calling the plot function
        if domainless_col_temp not in df_high_quality_hits.columns:
             df_high_quality_hits[domainless_col_temp] = df_high_quality_hits[num_domains_col].isna()
        if domainless_col_temp not in df_asgard_all.columns:
             df_asgard_all[domainless_col_temp] = df_asgard_all[num_domains_col].isna()
        
        plot_comparison_bar(
            df_subset=df_high_quality_hits,
            df_baseline=df_asgard_all,
            column_name=domainless_col_temp,
            title_suffix="(True = Domain-less)"
            # No color_map needed here
        )
        # Clean up temporary column if desired (optional)
        # if domainless_col_temp in df_high_quality_hits.columns:
        #     df_high_quality_hits.drop(columns=[domainless_col_temp], inplace=True)
        # if domainless_col_temp in df_asgard_all.columns:
        #     df_asgard_all.drop(columns=[domainless_col_temp], inplace=True)
    else:
        print(f"Skipping Domain-less profile: Column '{num_domains_col}' not found.")

else:
     print("\nSkipping profiling analysis because either 'df_high_quality_hits' or 'df_asgard_all' is empty or unavailable.")


print("\n\n--- Cell 4 (Statistical Profiling) Complete ---")
print("This cell compared the profiles of Asgard proteins with high-quality eukaryotic hits")
print("against the overall Asgard proteome for key characteristics.")
print("Next steps involve deeper functional and domain comparisons (Phase 2.1, 2.2, 2.4).")



In [ ]:
# Cell 5: Extract Unique Eukaryotic Hit IDs for External API Script

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os

print("\n\n--- Cell 5: Extract Eukaryotic Hit IDs for External API Script ---")

# --- Assumptions ---
# This cell assumes Cell 1 and Cell 2 have been run successfully.
# It relies on 'df_high_quality_hits' being available and populated from Cell 2.
# It also relies on column name variables (e.g., euk_hit_sseqid_col) and 
# output directory variables (e.g., output_summary_dir_phase1) defined in Cell 1.

# --- Check if necessary DataFrame exists ---
if 'df_high_quality_hits' not in locals() or df_high_quality_hits.empty:
    print("ERROR: DataFrame 'df_high_quality_hits' not found or is empty.")
    print("       Cannot extract IDs. Please ensure Cell 2 ran successfully and generated results.")
    ids_to_save = [] # Ensure variable exists but is empty
else:
    print(f"Using 'df_high_quality_hits' (shape: {df_high_quality_hits.shape}) to extract IDs.")
    
    # --- Extract Unique Eukaryotic SSEQIDs ---
    if euk_hit_sseqid_col in df_high_quality_hits.columns:
        unique_euk_sseqids = df_high_quality_hits[euk_hit_sseqid_col].dropna().unique()
        ids_to_save = unique_euk_sseqids.tolist()
        print(f"Found {len(ids_to_save)} unique eukaryotic SSEQIDs to save for mapping.")
    else:
        print(f"ERROR: Column '{euk_hit_sseqid_col}' not found in df_high_quality_hits.")
        ids_to_save = []

# --- Save IDs to File ---
if ids_to_save:
    output_id_list_filename = os.path.join(output_summary_dir_phase1, "eukaryotic_hit_sseqids_to_map.txt")
    try:
        with open(output_id_list_filename, 'w') as f_out:
            for prot_id in ids_to_save:
                f_out.write(f"{prot_id}\n")
        print(f"\nSuccessfully saved {len(ids_to_save)} unique eukaryotic hit IDs to:")
        print(f"  '{output_id_list_filename}'")
        print("\nNext Steps:")
        print(f"1. Modify your 'uniprot_api_lookup.py' script:")
        print(f"   - Change the input loading section to read IDs from '{output_id_list_filename}'.")
        print(f"   - Change the 'FROM_DB' variable to 'P_RefSeq_AC' (if IDs are XP_/NP_).")
        print(f"   - Adjust 'OUTPUT_MAPPING_FILE' path if desired.")
        print(f"2. Run the modified script from your terminal.")
        print(f"3. Use the resulting mapping file for the next step: fetching annotations.")

    except Exception as e:
        print(f"\nERROR: Could not save the ID list to file. Error: {e}")
else:
    print("\nNo unique eukaryotic hit IDs found or extracted to save.")


print("\n\n--- Cell 5 (Extract Euk Hit IDs) Complete ---")



In [ ]:
# Cell 6: Summary Statistics Visualization for Eukaryotic Hits in Asgard Subsets

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time

# --- Plotly Imports ---
# Assuming plotly express and graph_objects were imported as px and go in Cell 1
# import plotly.express as px 
# import plotly.graph_objects as go 

print("\n\n--- Cell 6: Summary Statistics Visualization ---")

# --- Assumptions ---
# This cell assumes Cell 1 has been run successfully.
# It relies on 'df_full' being available and populated with columns:
#   group_col, hit_flag_col, esp_col, structurally_dark_col, 
#   num_domains_col, broad_func_cat_col
# It also relies on color palettes (e.g., arcadia_primary_manual) defined in Cell 1.

# --- Check if necessary DataFrame exists ---
if 'df_full' not in locals() or df_full.empty:
    print("ERROR: DataFrame 'df_full' not found or is empty. Please run Cell 1 first.")
    # Stop or create empty df to prevent errors
    df_full = pd.DataFrame() 
    df_asgard_all = pd.DataFrame() # Ensure this is also empty
else:
    # Ensure df_asgard_all exists from Cell 1/4 or create it
    if 'df_asgard_all' not in locals() or df_asgard_all.empty:
        print("Creating baseline df_asgard_all from df_full for summary.")
        if group_col in df_full.columns:
            df_asgard_all = df_full[df_full[group_col] == 'Asgard'].copy()
        else:
            print(f"ERROR: Cannot create df_asgard_all, missing '{group_col}'.")
            df_asgard_all = pd.DataFrame() # Ensure it's defined but empty
            
    if df_asgard_all.empty:
         print("WARNING: No Asgard proteins found in 'df_full'. Summary will be empty.")


# --- Define and Analyze Subsets ---
summary_data = []

if not df_asgard_all.empty:
    print(f"Analyzing {len(df_asgard_all)} total Asgard proteins...")
    
    # Define subsets based on columns
    subsets_to_analyze = {
        "Overall Asgard": df_asgard_all,
        "Asgard ESPs": df_asgard_all[df_asgard_all[esp_col] == True] if esp_col in df_asgard_all.columns else pd.DataFrame(),
        "Structurally Dark": df_asgard_all[df_asgard_all[structurally_dark_col] == True] if structurally_dark_col in df_asgard_all.columns else pd.DataFrame(),
        "Domain-less": df_asgard_all[df_asgard_all[num_domains_col].isna()] if num_domains_col in df_asgard_all.columns else pd.DataFrame(),
        "Unknown/Unclassified": df_asgard_all[df_asgard_all[broad_func_cat_col] == 'Unknown/Unclassified'] if broad_func_cat_col in df_asgard_all.columns else pd.DataFrame()
    }

    # Calculate stats for each subset
    for name, df_subset in subsets_to_analyze.items():
        print(f"\nProcessing subset: {name}")
        
        # Check if subset DataFrame is valid and has the hit flag column
        if df_subset.empty or hit_flag_col not in df_subset.columns:
            if df_subset.empty:
                 print(f"  Subset '{name}' is empty (possibly due to missing definition column or no matching data). Skipping.")
            else:
                 print(f"  Skipping subset '{name}': Missing '{hit_flag_col}' column.")
            total_subset = 0
            hits_in_subset = 0
            percent_hits = 0.0
        else:
            total_subset = len(df_subset)
            hits_in_subset = df_subset[hit_flag_col].sum()
            percent_hits = (hits_in_subset / total_subset * 100) if total_subset > 0 else 0.0
            print(f"  Total Proteins: {total_subset}")
            print(f"  With Euk Hit: {hits_in_subset}")
            print(f"  Percentage: {percent_hits:.1f}%")
            
        summary_data.append({
            'Category': name,
            'Total Proteins': total_subset,
            'Proteins with Euk Hit': hits_in_subset,
            'Percentage with Euk Hit (%)': percent_hits
        })

else:
    print("Skipping summary calculation as no Asgard proteins were found.")

# --- Create and Display Summary DataFrame ---
if summary_data:
    df_summary = pd.DataFrame(summary_data)
    
    print("\n\n--- Summary Table: Eukaryotic Hits in Asgard Subsets ---")
    try:
        # Format the table nicely
        df_display_summary = df_summary.copy()
        df_display_summary['Total Proteins'] = df_display_summary['Total Proteins'].map('{:,.0f}'.format)
        df_display_summary['Proteins with Euk Hit'] = df_display_summary['Proteins with Euk Hit'].map('{:,.0f}'.format)
        df_display_summary['Percentage with Euk Hit (%)'] = df_display_summary['Percentage with Euk Hit (%)'].map('{:.1f}%'.format)
        print(df_display_summary.to_markdown(index=False))
    except ImportError:
        print(df_summary) # Fallback if tabulate not installed

    # --- Generate Plot ---
    print("\n--- Generating Plot ---")
    
    # Define a color map for the categories using Arcadia palettes
    # Ensure arcadia_primary_manual is defined (should be from Cell 1)
    if 'arcadia_primary_manual' not in locals():
        print("Warning: Arcadia color palette not found. Using default Plotly colors.")
        category_color_map = None
    else:
         # Assign colors, maybe cycle through primary then secondary
         plot_palette = arcadia_primary_manual + arcadia_secondary_manual
         category_color_map = {
             category: plot_palette[i % len(plot_palette)] 
             for i, category in enumerate(df_summary['Category'])
         }

    fig_summary = px.bar(
        df_summary,
        x='Category',
        y='Percentage with Euk Hit (%)',
        color='Category', # Color bars by category
        color_discrete_map=category_color_map, # Apply the custom color map
        title='Percentage of Asgard Protein Subsets with Eukaryotic DIAMOND Hits',
        labels={'Percentage with Euk Hit (%)': '% Proteins with Eukaryotic Hit'},
        text='Percentage with Euk Hit (%)' # Add percentage text on bars
    )
    
    fig_summary.update_traces(texttemplate='%{text:.1f}%', textposition='outside') # Format text
    fig_summary.update_layout(
        xaxis_title=None, # Remove x-axis title if categories are clear
        yaxis_title='% Proteins with Eukaryotic Hit',
        yaxis_range=[0, df_summary['Percentage with Euk Hit (%)'].max() * 1.15], # Set y-axis range slightly above max
        showlegend=False, # Hide legend as colors match x-axis labels
        uniformtext_minsize=8, 
        uniformtext_mode='hide'
    )
    fig_summary.update_xaxes(categoryorder='array', categoryarray=df_summary['Category']) # Keep original order
    fig_summary.show()
    
    # Save plot
    plot_filename = os.path.join(output_plot_dir_phase1, "asgard_subsets_euk_hit_percentage.html")
    try:
        fig_summary.write_html(plot_filename)
        print(f"Saved summary plot to '{plot_filename}'")
    except Exception as e:
        print(f"Error saving summary plot: {e}")

else:
    print("\nNo summary data generated to display or plot.")


print("\n\n--- Cell 6 (Summary Visualization) Complete ---")



In [ ]:
# Cell 7: Comparison of High-Quality Eukaryotic Hits: ESPs vs. Non-ESPs

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time
from collections import Counter

# --- Plotly Imports ---
# Assuming plotly express and graph_objects were imported as px and go in Cell 1
# import plotly.express as px 
# import plotly.graph_objects as go 

print("\n\n--- Cell 7: ESP vs. Non-ESP High-Quality Hit Comparison ---")

# --- Assumptions ---
# This cell assumes Cell 1 and Cell 2 have been run successfully.
# It relies on the following DataFrames being available and populated:
#   - df_full: The main DataFrame loaded in Cell 1.
#   - df_high_quality_hits: Asgard proteins filtered for hit quality in Cell 2.
# It also relies on column name variables (e.g., group_col, esp_col, euk_hit_pident_col, etc.) 
# and color maps (arcadia_colors_manual, broad_category_color_map, etc.) defined in Cell 1.

# --- Check if necessary DataFrames exist ---
if 'df_high_quality_hits' not in locals() or df_high_quality_hits.empty:
    print("ERROR: DataFrame 'df_high_quality_hits' not found or is empty.")
    print("       Cannot perform ESP comparison. Please ensure Cell 2 ran successfully and generated results.")
    # Create empty df to prevent downstream errors
    df_high_quality_hits = pd.DataFrame() 

if 'df_full' not in locals() or df_full.empty:
     print("ERROR: DataFrame 'df_full' not found. Needed for context if issues arise.")
     # Allow proceeding but some context might be missing if df_high_quality_hits is also empty

# --- ESP Definition ---
print("\n--- ESP Definition Used ---")
print(f"ESPs are defined based on the '{esp_col}' column in the DataFrame.")
print("This column was populated in Cell 1 of the 'integrate_diamond_hits' notebook")
print(f"by checking if an Asgard protein's '{orthogroup_col}' was present in the list")
print(f"loaded from 'all_esp_orthogroup_list_v4.txt'.")

# --- Subset Data into ESPs and Non-ESPs ---
df_hq_esps = pd.DataFrame()
df_hq_non_esps = pd.DataFrame()

if not df_high_quality_hits.empty and esp_col in df_high_quality_hits.columns:
    df_hq_esps = df_high_quality_hits[df_high_quality_hits[esp_col] == True].copy()
    df_hq_non_esps = df_high_quality_hits[df_high_quality_hits[esp_col] == False].copy()
    print(f"\nSubsetting high-quality hits:")
    print(f"  Found {len(df_hq_esps)} ESPs with high-quality eukaryotic hits.")
    print(f"  Found {len(df_hq_non_esps)} Non-ESPs with high-quality eukaryotic hits.")
elif not df_high_quality_hits.empty:
     print(f"ERROR: ESP column '{esp_col}' not found in df_high_quality_hits. Cannot subset.")
else:
     print("Skipping subsetting as df_high_quality_hits is empty.")


# --- Perform Comparative Analyses ---

# Helper function for comparative histograms/box plots
def plot_comparison_dist(df1, df2, column, df1_name, df2_name, title, log_scale=False):
    """Plots overlapping histograms and box plots for a column from two dataframes."""
    if column not in df1.columns or column not in df2.columns:
        print(f"  Skipping distribution plot for '{column}': Column missing in one or both dataframes.")
        return
        
    if df1[column].isna().all() or df2[column].isna().all():
        print(f"  Skipping distribution plot for '{column}': All values are NaN in one or both dataframes.")
        return

    fig = go.Figure()
    # Histogram for df1
    fig.add_trace(go.Histogram(x=df1[column], name=df1_name, opacity=0.75, 
                               marker_color=arcadia_colors_manual.get('amber', '#F28360')))
    # Histogram for df2
    fig.add_trace(go.Histogram(x=df2[column], name=df2_name, opacity=0.75,
                               marker_color=arcadia_colors_manual.get('aegean', '#5088C5')))
    
    fig.update_layout(
        barmode='overlay',
        title_text=f'Distribution Comparison: {title}',
        xaxis_title_text=column.replace('_', ' ').title(),
        yaxis_title_text='Count'
    )
    if log_scale:
        # Handle potential non-positive values for log scale if needed
        # This simple version assumes positive values or uses Plotly's default handling
        fig.update_xaxes(type="log") 
        fig.update_layout(xaxis_title_text=f"Log({column.replace('_', ' ').title()})")

    fig.show()

    # Box plots
    df_combined = pd.concat([
        df1[[column]].assign(Group=df1_name),
        df2[[column]].assign(Group=df2_name)
    ], ignore_index=True)

    fig_box = px.box(
        df_combined.dropna(subset=[column]), 
        x='Group', 
        y=column, 
        color='Group',
        title=f'Box Plot Comparison: {title}',
        labels={column: column.replace('_', ' ').title()},
        color_discrete_map={df1_name: arcadia_colors_manual.get('amber', '#F28360'), 
                            df2_name: arcadia_colors_manual.get('aegean', '#5088C5')}
    )
    if log_scale:
         fig_box.update_yaxes(type="log")
         fig_box.update_layout(yaxis_title_text=f"Log({column.replace('_', ' ').title()})")
         
    fig_box.show()


if not df_hq_esps.empty and not df_hq_non_esps.empty:
    print("\n--- Comparing ESP vs Non-ESP High-Quality Hits ---")

    # 1. Hit Quality Comparison
    print("\n--- 1. Hit Quality (PIDENT, E-value) ---")
    plot_comparison_dist(df_hq_esps, df_hq_non_esps, euk_hit_pident_col, 'ESPs', 'Non-ESPs', 'Euk Hit Percent Identity')
    # For E-value, use the log10 version if calculated in Cell 1, otherwise calculate here
    if 'log10_evalue' not in df_hq_esps.columns and euk_hit_evalue_col in df_hq_esps.columns:
         df_hq_esps['log10_evalue'] = np.log10(df_hq_esps[euk_hit_evalue_col] + 1e-200)
    if 'log10_evalue' not in df_hq_non_esps.columns and euk_hit_evalue_col in df_hq_non_esps.columns:
         df_hq_non_esps['log10_evalue'] = np.log10(df_hq_non_esps[euk_hit_evalue_col] + 1e-200)
         
    if 'log10_evalue' in df_hq_esps.columns and 'log10_evalue' in df_hq_non_esps.columns:
        plot_comparison_dist(df_hq_esps, df_hq_non_esps, 'log10_evalue', 'ESPs', 'Non-ESPs', 'Log10(Euk Hit E-value)')
    else:
        print(f"  Skipping E-value comparison: Column '{euk_hit_evalue_col}' or 'log10_evalue' missing.")

    # 2. Alignment Coverage Comparison
    print("\n--- 2. Alignment Coverage (Query, Subject) ---")
    plot_comparison_dist(df_hq_esps, df_hq_non_esps, query_coverage_col, 'ESPs', 'Non-ESPs', 'Query (Asgard) Coverage')
    plot_comparison_dist(df_hq_esps, df_hq_non_esps, subject_coverage_col, 'ESPs', 'Non-ESPs', 'Subject (Eukaryote) Coverage')

    # 3. Taxonomic Distribution Comparison
    print("\n--- 3. Taxonomic Distribution of Hits ---")
    top_n_orgs_comp = 20 # Show fewer for comparison clarity
    
    org_counts_esps = df_hq_esps[euk_hit_organism_col].value_counts().head(top_n_orgs_comp)
    org_counts_non_esps = df_hq_non_esps[euk_hit_organism_col].value_counts().head(top_n_orgs_comp)

    df_org_comp = pd.DataFrame({
        'ESP Hits': org_counts_esps,
        'Non-ESP Hits': org_counts_non_esps
    }).fillna(0).astype(int).reset_index()
    df_org_comp.rename(columns={'index': euk_hit_organism_col}, inplace=True)
    
    df_org_melted = df_org_comp.melt(id_vars=euk_hit_organism_col, var_name='Group', value_name='Count')

    print(f"\nTop {top_n_orgs_comp} Eukaryotic Hit Organisms (ESP vs Non-ESP):")
    # Display combined counts for top overall organisms
    combined_top_orgs = df_org_comp.sort_values(['ESP Hits', 'Non-ESP Hits'], ascending=False).head(top_n_orgs_comp)
    try:
        print(combined_top_orgs.to_markdown(index=False))
    except ImportError:
        print(combined_top_orgs)

    fig_org_comp = px.bar(
        df_org_melted,
        x=euk_hit_organism_col,
        y='Count',
        color='Group',
        barmode='group',
        title=f'Top {top_n_orgs_comp} Eukaryotic Hit Organisms: ESPs vs. Non-ESPs',
        labels={'Count': 'Number of Asgard Proteins Hit', euk_hit_organism_col: 'Eukaryotic Organism'},
        category_orders={euk_hit_organism_col: combined_top_orgs[euk_hit_organism_col].tolist()}, # Order by combined rank
        color_discrete_map={'ESP Hits': arcadia_colors_manual.get('amber', '#F28360'), 
                            'Non-ESP Hits': arcadia_colors_manual.get('aegean', '#5088C5')}
    )
    fig_org_comp.update_xaxes(tickangle=45)
    fig_org_comp.show()

    # 4. Functional Category Comparison (within ESPs and within Non-ESPs)
    print("\n--- 4. Broad Functional Category Distribution ---")
    if broad_func_cat_col in df_hq_esps.columns and broad_func_cat_col in df_hq_non_esps.columns:
        esp_cat_counts = df_hq_esps[broad_func_cat_col].value_counts(normalize=True).mul(100)
        non_esp_cat_counts = df_hq_non_esps[broad_func_cat_col].value_counts(normalize=True).mul(100)

        df_cat_comp = pd.DataFrame({'ESPs (%)': esp_cat_counts, 'Non-ESPs (%)': non_esp_cat_counts}).fillna(0)
        # Order by ESP percentage for focus
        df_cat_comp = df_cat_comp.sort_values('ESPs (%)', ascending=False)
        
        print("\nBroad Functional Category Distribution within Hit Groups:")
        try:
             print(df_cat_comp.round(1).to_markdown())
        except ImportError:
             print(df_cat_comp.round(1))
             
        # Plotting - maybe side-by-side? Or separate plots? Let's do separate for clarity.
        fig_cat_esp = px.bar(
            esp_cat_counts.reset_index(), x=broad_func_cat_col, y='proportion',
            title='Functional Categories of ESPs with High-Quality Euk Hits',
            labels={'proportion': '% of ESP Hits', broad_func_cat_col: 'Category'},
            color=broad_func_cat_col, color_discrete_map=broad_category_color_map
        )
        fig_cat_esp.update_layout(showlegend=False)
        fig_cat_esp.update_xaxes(categoryorder='total descending')
        fig_cat_esp.show()

        fig_cat_non_esp = px.bar(
            non_esp_cat_counts.reset_index(), x=broad_func_cat_col, y='proportion',
            title='Functional Categories of Non-ESPs with High-Quality Euk Hits',
            labels={'proportion': '% of Non-ESP Hits', broad_func_cat_col: 'Category'},
            color=broad_func_cat_col, color_discrete_map=broad_category_color_map
        )
        fig_cat_non_esp.update_layout(showlegend=False)
        fig_cat_non_esp.update_xaxes(categoryorder='total descending')
        fig_cat_non_esp.show()
        
    else:
        print(f"Skipping Functional Category comparison: Column '{broad_func_cat_col}' missing.")

    # 5. Structural Darkness Comparison
    print("\n--- 5. Structural Darkness Comparison ---")
    if structurally_dark_col in df_hq_esps.columns and structurally_dark_col in df_hq_non_esps.columns:
        plot_comparison_bar(df_hq_esps, df_hq_non_esps, structurally_dark_col, 'ESP vs Non-ESP Hits (True=Dark)')
    else:
        print(f"Skipping Structural Darkness comparison: Column '{structurally_dark_col}' missing.")
        
    # 6. Domain-less Comparison
    print("\n--- 6. Domain-less Comparison ---")
    if num_domains_col in df_hq_esps.columns and num_domains_col in df_hq_non_esps.columns:
        domainless_col_temp = 'Is_DomainLess'
        df_hq_esps[domainless_col_temp] = df_hq_esps[num_domains_col].isna()
        df_hq_non_esps[domainless_col_temp] = df_hq_non_esps[num_domains_col].isna()
        plot_comparison_bar(df_hq_esps, df_hq_non_esps, domainless_col_temp, 'ESP vs Non-ESP Hits (True=Domain-less)')
    else:
        print(f"Skipping Domain-less comparison: Column '{num_domains_col}' missing.")

elif df_high_quality_hits.empty:
    print("Skipping ESP vs Non-ESP comparison as 'df_high_quality_hits' is empty.")
else:
    print("Skipping ESP vs Non-ESP comparison due to missing ESP column or empty subsets.")

print("\n\n--- Cell 7 (ESP vs Non-ESP Comparison) Complete ---")
print("This cell compared key characteristics between Asgard ESPs and Non-ESPs")
print("that have high-quality eukaryotic hits.")



In [ ]:
# Cell 8: Profiling Domains and Functions of Non-ESPs with High-Quality Eukaryotic Hits

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time
from collections import Counter
import re # For parsing IPRs

# --- Plotly Imports ---
# Assuming plotly express and graph_objects were imported as px and go in Cell 1
# import plotly.express as px 
# import plotly.graph_objects as go 

print("\n\n--- Cell 8: Profiling Non-ESPs with High-Quality Eukaryotic Hits ---")

# --- Assumptions ---
# This cell assumes Cell 1, 2, and subsequent cells defining df_high_quality_hits have been run.
# It relies on the DataFrame 'df_high_quality_hits'.
# It also relies on column names (esp_col, domain_arch_col, ipr_col, broad_func_cat_col) 
# and helper functions/variables (ipr_lookup, translate_architecture, output_summary_dir_phase1, etc.) defined in Cell 1.

# --- Check if necessary DataFrames/Variables exist ---
if 'df_high_quality_hits' not in locals() or df_high_quality_hits.empty:
    print("ERROR: DataFrame 'df_high_quality_hits' not found or is empty.")
    print("       Cannot perform Non-ESP analysis. Please ensure Cell 2 ran successfully.")
    df_hq_non_esps = pd.DataFrame() # Create empty df to prevent errors
else:
    # Subset for Non-ESPs with high-quality hits
    if esp_col in df_high_quality_hits.columns:
        df_hq_non_esps = df_high_quality_hits[df_high_quality_hits[esp_col] == False].copy()
        print(f"Analyzing {len(df_hq_non_esps)} Non-ESP Asgard proteins with high-quality eukaryotic hits.")
        if df_hq_non_esps.empty:
            print("No Non-ESP proteins found within the high-quality hits subset.")
    else:
        print(f"ERROR: ESP column '{esp_col}' not found in df_high_quality_hits. Cannot isolate Non-ESPs.")
        df_hq_non_esps = pd.DataFrame()

# --- Perform Profiling on Non-ESP Subset ---

if not df_hq_non_esps.empty:

    # 1. Analyze Domain Architectures for Non-ESPs with Hits
    print(f"\n--- 1. Domain Architectures (Top 20) for Non-ESPs with High-Quality Euk Hits ---")
    if domain_arch_col in df_hq_non_esps.columns:
        # Fill NA with a placeholder for value_counts if needed
        non_esp_arch_counts = df_hq_non_esps[domain_arch_col].fillna('No_Domain').value_counts()
        
        df_non_esp_arch_plot = non_esp_arch_counts.head(20).reset_index()
        df_non_esp_arch_plot.columns = ['Architecture', 'Count']
        
        # Translate architectures for readability in the plot hover/table
        if 'translate_architecture' in locals() and 'ipr_lookup' in locals():
             df_non_esp_arch_plot['Translated Architecture'] = df_non_esp_arch_plot['Architecture'].apply(translate_architecture)
             hover_col = 'Translated Architecture'
        else:
             print("Warning: 'translate_architecture' function or 'ipr_lookup' not found. Using raw architectures.")
             hover_col = 'Architecture'
        
        print("\nTop 20 Domain Architectures:")
        try:
             print(df_non_esp_arch_plot.to_markdown(index=False))
        except ImportError:
             print(df_non_esp_arch_plot)
             
        # Save summary
        arch_filename = os.path.join(output_summary_dir_phase1, "non_esp_hq_hits_top_architectures.csv")
        try:
             non_esp_arch_counts.reset_index().rename(columns={'index':'Architecture', domain_arch_col:'Count'}).to_csv(arch_filename, index=False)
             print(f"Saved architecture counts to '{arch_filename}'")
        except Exception as e:
             print(f"Error saving architecture counts: {e}")

        # Plot
        fig_arch = px.bar(
            df_non_esp_arch_plot,
            x='Architecture',
            y='Count',
            title='Top 20 Domain Architectures for Non-ESP Asgard Proteins with High-Quality Euk Hits',
            hover_data=[hover_col],
            labels={'Architecture': 'Domain Architecture (Raw)', 'Count': 'Number of Proteins'}
        )
        fig_arch.update_xaxes(tickangle=45, categoryorder='total descending', 
                              # Truncate long x-axis labels if needed
                              ticktext = [truncate_string(s, 50) for s in df_non_esp_arch_plot['Architecture']],
                              tickvals = df_non_esp_arch_plot['Architecture'])
        fig_arch.show()
    else:
        print(f"Skipping Domain Architecture analysis: Column '{domain_arch_col}' not found.")
        
    # 2. Analyze Individual IPR Signatures for Non-ESPs with Hits
    print(f"\n--- 2. Individual IPR Signatures (Top 30) for Non-ESPs with High-Quality Euk Hits ---")
    if ipr_col in df_hq_non_esps.columns and 'ipr_lookup' in locals():
        all_iprs_non_esp_list = []
        for ipr_string_raw in df_hq_non_esps[ipr_col].dropna():
            processed_ipr_string = str(ipr_string_raw).replace('|', ';')
            individual_iprs = processed_ipr_string.split(';')
            for ipr_item in individual_iprs:
                stripped_ipr = ipr_item.strip()
                if stripped_ipr: 
                    all_iprs_non_esp_list.append(stripped_ipr)
        
        ipr_counts_non_esp = Counter(all_iprs_non_esp_list)
        
        if ipr_counts_non_esp:
            top_n_iprs = 30
            df_ipr_counts = pd.DataFrame(ipr_counts_non_esp.most_common(top_n_iprs), columns=['IPR_ID', 'Count'])
            
            # Add descriptions
            df_ipr_counts['Description'] = df_ipr_counts['IPR_ID'].apply(lambda x: ipr_lookup.get(x, {}).get('Name', 'N/A'))
            df_ipr_counts['Type'] = df_ipr_counts['IPR_ID'].apply(lambda x: ipr_lookup.get(x, {}).get('Type', 'N/A'))
            
            # Create a combined label for plotting
            df_ipr_counts['Label'] = df_ipr_counts['Description'].apply(lambda x: truncate_string(x, 60)) + " (" + df_ipr_counts['IPR_ID'] + ")"

            print(f"\nTop {top_n_iprs} IPR Signatures:")
            try:
                 print(df_ipr_counts[['IPR_ID', 'Count', 'Description']].to_markdown(index=False))
            except ImportError:
                 print(df_ipr_counts[['IPR_ID', 'Count', 'Description']])

            # Save summary
            ipr_filename = os.path.join(output_summary_dir_phase1, "non_esp_hq_hits_top_iprs.csv")
            try:
                 # Save full list sorted by count
                 full_ipr_df = pd.DataFrame(ipr_counts_non_esp.items(), columns=['IPR_ID', 'Count']).sort_values('Count', ascending=False)
                 full_ipr_df['Description'] = full_ipr_df['IPR_ID'].apply(lambda x: ipr_lookup.get(x, {}).get('Name', 'N/A'))
                 full_ipr_df['Type'] = full_ipr_df['IPR_ID'].apply(lambda x: ipr_lookup.get(x, {}).get('Type', 'N/A'))
                 full_ipr_df[['IPR_ID', 'Count', 'Description', 'Type']].to_csv(ipr_filename, index=False)
                 print(f"Saved IPR counts to '{ipr_filename}'")
            except Exception as e:
                 print(f"Error saving IPR counts: {e}")

            # Plot
            fig_ipr = px.bar(
                df_ipr_counts,
                x='Label',
                y='Count',
                title=f'Top {top_n_iprs} IPR Signatures for Non-ESP Asgard Proteins with High-Quality Euk Hits',
                hover_data=['IPR_ID', 'Description', 'Type'],
                labels={'Label': 'IPR Signature', 'Count': 'Number of Proteins'}
            )
            fig_ipr.update_xaxes(tickangle=45, categoryorder='total descending')
            fig_ipr.show()
            
        else:
            print("No IPR signatures found to analyze in this subset.")
            
    elif ipr_col not in df_hq_non_esps.columns:
        print(f"Skipping IPR analysis: Column '{ipr_col}' not found.")
    else: # ipr_lookup missing
        print(f"Skipping IPR analysis: 'ipr_lookup' dictionary not found (load from Cell 1).")

    # 3. Analyze Broad Functional Categories for Non-ESPs with Hits
    print(f"\n--- 3. Broad Functional Categories for Non-ESPs with High-Quality Euk Hits ---")
    if broad_func_cat_col in df_hq_non_esps.columns:
        non_esp_cat_counts = df_hq_non_esps[broad_func_cat_col].value_counts()
        df_non_esp_cat_plot = non_esp_cat_counts.reset_index()
        df_non_esp_cat_plot.columns = ['Category', 'Count']

        print("\nFunctional Category Distribution:")
        try:
            print(non_esp_cat_counts.to_markdown())
        except ImportError:
            print(non_esp_cat_counts)
            
        # Save summary
        cat_filename = os.path.join(output_summary_dir_phase1, "non_esp_hq_hits_category_counts.csv")
        try:
             non_esp_cat_counts.reset_index().rename(columns={'index':'Category', broad_func_cat_col:'Count'}).to_csv(cat_filename, index=False)
             print(f"Saved category counts to '{cat_filename}'")
        except Exception as e:
             print(f"Error saving category counts: {e}")

        # Plot
        fig_cat_non_esp_only = px.bar(
            df_non_esp_cat_plot,
            x='Category',
            y='Count',
            title='Functional Categories of Non-ESPs with High-Quality Euk Hits',
            labels={'Count': 'Number of Proteins', 'Category': 'Broad Functional Category'},
            color='Category',
            color_discrete_map=broad_category_color_map # Use map from Cell 1
        )
        fig_cat_non_esp_only.update_layout(showlegend=False)
        fig_cat_non_esp_only.update_xaxes(categoryorder='total descending')
        fig_cat_non_esp_only.show()
    else:
         print(f"Skipping Functional Category analysis: Column '{broad_func_cat_col}' not found.")


else:
    print("Skipping detailed Non-ESP profiling as the 'df_hq_non_esps' DataFrame is empty.")


print("\n\n--- Cell 8 (Non-ESP Hit Profiling) Complete ---")
print("This cell analyzed the domain architectures, IPR signatures, and functional categories")
print("for the subset of Non-ESP Asgard proteins that have high-quality eukaryotic hits.")
print("Compare these results (e.g., top IPRs/Architectures) to typical ESP functions")
print("to investigate the hypothesis that some might be functionally related.")
print("The next crucial step is obtaining annotations for the eukaryotic hits themselves.")


In [ ]:
# Cell 10: Domain Architecture Comparison (ESP vs. Non-ESP High-Quality Hits)

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time
from collections import Counter
import re # For parsing IPRs

# --- Plotly Imports ---
# Assuming plotly express and graph_objects were imported as px and go in Cell 1
# import plotly.express as px 
# import plotly.graph_objects as go 

print("\n\n--- Cell 10: Domain Architecture Comparison (ESP vs Non-ESP High-Quality Hits) ---")

# --- Assumptions ---
# This cell assumes Cell 1, 2, and 7 have been run successfully.
# It relies on the following DataFrames being available and populated:
#   - df_hq_esps: ESPs with high-quality hits (from Cell 7).
#   - df_hq_non_esps: Non-ESPs with high-quality hits (from Cell 7).
# It also relies on column names (domain_arch_col), helper functions (translate_architecture),
# variables (ipr_lookup, arcadia_colors_manual), and output directories (output_summary_dir_phase1) defined in Cell 1.

# --- Configuration ---
top_n_arch_display = 30 # How many top architectures to show in tables/plots
# Placeholder list of IPR IDs often associated with ESP functions 
# (Customize this list based on your project's definition and literature)
esp_like_ipr_keywords = [
    "IPR001680", # WD40 repeat
    "IPR002110", # TPR repeat
    "IPR002110", # Ankyrin repeat # Note: Duplicate IPR002110, likely meant IPR002110 (TPR) and IPR002068 (Ankyrin) or similar - check list
    "IPR001806", # Ras family
    "IPR000159", # RhoGAP domain
    "IPR001895", # RhoGEF domain
    "IPR000719", # Protein kinase domain
    "IPR001245", # Protein kinase, catalytic domain
    "IPR011009", # Protein kinase-like domain
    "IPR000387", # Protein tyrosine kinase
    "IPR000980", # Protein phosphatase 2C
    "IPR020633", # Dual specificity phosphatase, catalytic domain
    "IPR001628", # Protein tyrosine phosphatase
    "IPR000626", # Ubiquitin-conjugating enzyme E2
    "IPR000569", # Ubiquitin carboxyl-terminal hydrolase
    "IPR001342", # Ubiquitin domain
    "IPR019744", # Ubiquitin-like domain superfamily
    "IPR011080", # ESCRT-I complex Vps28 C-terminal domain
    "IPR017946", # ESCRT-II Vps25
    "IPR017944", # ESCRT-III/Vps20/Vps32
    "IPR001478", # Clathrin, heavy chain
    "IPR016024", # Clathrin adaptor complex, medium subunit family
    "IPR008948", # SNARE domain
    "IPR001007", # Dynein heavy chain region
    "IPR001613", # Kinesin motor domain
    "IPR004000", # Actin
    "IPR001603", # Tubulin
    # Add more relevant IPR IDs based on your ESP definitions...
]

# --- Check if necessary DataFrames/Variables exist ---
if 'df_hq_esps' not in locals() or 'df_hq_non_esps' not in locals():
    print("ERROR: DataFrames 'df_hq_esps' and/or 'df_hq_non_esps' not found.")
    print("       Cannot perform comparison. Please ensure Cell 7 ran successfully.")
    if 'df_hq_esps' not in locals(): df_hq_esps = pd.DataFrame()
    if 'df_hq_non_esps' not in locals(): df_hq_non_esps = pd.DataFrame()
    
if 'domain_arch_col' not in locals():
     print("ERROR: Variable 'domain_arch_col' not defined. Please ensure Cell 1 ran.")
     domain_arch_col = 'Domain_Architecture' # Assign dummy - analysis will likely fail later

if 'translate_architecture' not in locals() or 'ipr_lookup' not in locals():
     print("WARNING: Helper function 'translate_architecture' or 'ipr_lookup' not found.")
     print("         Plots and tables will show raw IPR IDs only.")
     if 'translate_architecture' not in locals():
         def translate_architecture(arch_string, lookup_dict=None): return arch_string
     if 'ipr_lookup' not in locals():
         ipr_lookup = {}
         
if 'arcadia_colors_manual' not in locals():
     print("WARNING: Arcadia color palette 'arcadia_colors_manual' not found. Using default colors.")
     arcadia_colors_manual = {} # Define empty to avoid errors

# --- Analyze and Compare Domain Architectures ---
if (not df_hq_esps.empty or not df_hq_non_esps.empty):
    # Ensure domain_arch_col exists before proceeding
    if domain_arch_col not in df_hq_esps.columns or domain_arch_col not in df_hq_non_esps.columns:
         print(f"ERROR: Column '{domain_arch_col}' missing from ESP or Non-ESP DataFrame. Cannot proceed with architecture analysis.")
    else:
        # 1. Calculate Architecture Counts for each group
        esp_arch_counts = df_hq_esps[domain_arch_col].fillna('No_Domain').value_counts()
        non_esp_arch_counts = df_hq_non_esps[domain_arch_col].fillna('No_Domain').value_counts()

        # 2. Combine Counts into a Comparison DataFrame
        df_arch_compare = pd.DataFrame({
            'ESP_Count': esp_arch_counts,
            'Non_ESP_Count': non_esp_arch_counts
        }).fillna(0).astype(int)
        
        # Add a total count column for sorting
        df_arch_compare['Total_Count'] = df_arch_compare['ESP_Count'] + df_arch_compare['Non_ESP_Count']
        df_arch_compare = df_arch_compare.sort_values('Total_Count', ascending=False)
        df_arch_compare.index.name = 'Architecture' # Name the index
        
        # Add translated architecture
        df_arch_compare['Translated Architecture'] = df_arch_compare.index.map(lambda x: translate_architecture(x, lookup_dict=ipr_lookup))

        print(f"\n--- Top {top_n_arch_display} Domain Architectures Overall (ESPs + Non-ESPs with High-Quality Euk Hits) ---")
        try:
             # Select columns for display
             display_cols_overall = ['ESP_Count', 'Non_ESP_Count', 'Total_Count', 'Translated Architecture']
             print(df_arch_compare[display_cols_overall].head(top_n_arch_display).to_markdown())
        except ImportError:
             print(df_arch_compare[display_cols_overall].head(top_n_arch_display))
             
        # Save combined summary
        combined_arch_filename = os.path.join(output_summary_dir_phase1, "esp_vs_non_esp_hq_hits_architectures.csv")
        try:
             df_arch_compare.reset_index().to_csv(combined_arch_filename, index=False)
             print(f"\nSaved combined architecture counts to '{combined_arch_filename}'")
        except Exception as e:
             print(f"Error saving combined architecture counts: {e}")

        # 3. Identify Architectures with ESP-like Domains in Non-ESPs
        print(f"\n--- Identifying Non-ESP Architectures with Potential ESP-like Domains ---")
        
        non_esp_arch_with_esp_like_domains = []
        
        # Iterate through architectures found in Non-ESPs
        for architecture, counts in df_arch_compare[df_arch_compare['Non_ESP_Count'] > 0].iterrows():
            if architecture == 'No_Domain': continue # Skip the 'No_Domain' placeholder
            
            # Get IPR IDs in the current architecture
            current_iprs = set(str(architecture).replace('|', ';').split(';')) - {''}
            
            # Check for overlap with ESP-like keywords
            found_esp_like = False
            for esp_ipr in esp_like_ipr_keywords:
                if esp_ipr in current_iprs:
                    found_esp_like = True
                    break # Found one, no need to check further for this architecture
                    
            if found_esp_like:
                non_esp_arch_with_esp_like_domains.append({
                    'Architecture': architecture,
                    'ESP_Count': counts['ESP_Count'],
                    'Non_ESP_Count': counts['Non_ESP_Count'],
                    'Total_Count': counts['Total_Count'],
                    'Translated Architecture': counts['Translated Architecture']
                })

        if non_esp_arch_with_esp_like_domains:
            df_non_esp_esp_like = pd.DataFrame(non_esp_arch_with_esp_like_domains).sort_values('Non_ESP_Count', ascending=False)
            print(f"Found {len(df_non_esp_esp_like)} architectures in Non-ESPs (with Euk hits) containing potential ESP-like domains:")
            try:
                print(df_non_esp_esp_like.to_markdown(index=False))
            except ImportError:
                print(df_non_esp_esp_like)
                
            # Save this list
            esp_like_filename = os.path.join(output_summary_dir_phase1, "non_esp_hq_hits_with_esp_like_domains.csv")
            try:
                 df_non_esp_esp_like.to_csv(esp_like_filename, index=False)
                 print(f"\nSaved list of Non-ESP architectures with ESP-like domains to '{esp_like_filename}'")
            except Exception as e:
                 print(f"Error saving ESP-like domain list: {e}")
                 
        else:
            print("No architectures containing the specified ESP-like IPRs were found among the Non-ESPs with high-quality hits.")
            
        # 4. Visualize Comparison (Focus on architectures present in both)
        print(f"\n--- Visualizing Top Architectures Present in Both ESPs and Non-ESPs ---")
        
        # Filter for architectures present in both groups
        df_arch_common = df_arch_compare[(df_arch_compare['ESP_Count'] > 0) & (df_arch_compare['Non_ESP_Count'] > 0)].copy()
        df_arch_common = df_arch_common.sort_values('Total_Count', ascending=False) # Keep all common ones for now
        
        if not df_arch_common.empty:
            # Melt for plotting
            df_common_melt = df_arch_common[['ESP_Count', 'Non_ESP_Count']].reset_index().melt(
                id_vars='Architecture', 
                var_name='Group', 
                value_name='Count'
            )
            # Add translated architecture for hover
            df_common_melt = df_common_melt.merge(
                df_arch_compare[['Translated Architecture']].reset_index(), 
                on='Architecture', 
                how='left'
            )

            # --- Plot 1: Including "No_Domain" ---
            df_plot_data_with_nodomain = df_common_melt[df_common_melt['Architecture'].isin(df_arch_common.head(top_n_arch_display).index)]
            
            fig_arch_common_with_nodomain = px.bar(
                df_plot_data_with_nodomain,
                x='Architecture',
                y='Count',
                color='Group',
                barmode='group',
                title=f'Top {top_n_arch_display} Common Architectures (Including No_Domain)',
                hover_data=['Translated Architecture'],
                labels={'Architecture': 'Domain Architecture (Raw)', 'Count': 'Number of Proteins'},
                color_discrete_map={'ESP_Count': arcadia_colors_manual.get('amber', '#F28360'), 
                                    'Non_ESP_Count': arcadia_colors_manual.get('aegean', '#5088C5')}
            )
            fig_arch_common_with_nodomain.update_xaxes(
                tickangle=45, 
                categoryorder='array', # Use the order from the sorted df_arch_common
                categoryarray=df_arch_common.head(top_n_arch_display).index.tolist(), 
                ticktext = [truncate_string(s, 50) for s in df_arch_common.head(top_n_arch_display).index], 
                tickvals = df_arch_common.head(top_n_arch_display).index
            )
            fig_arch_common_with_nodomain.show()

            # --- Plot 2: Excluding "No_Domain" ---
            df_arch_common_no_nodomain = df_arch_common[df_arch_common.index != 'No_Domain'].head(top_n_arch_display)
            
            if not df_arch_common_no_nodomain.empty:
                df_plot_data_no_nodomain = df_common_melt[df_common_melt['Architecture'].isin(df_arch_common_no_nodomain.index)]

                fig_arch_common_no_nodomain = px.bar(
                    df_plot_data_no_nodomain,
                    x='Architecture',
                    y='Count',
                    color='Group',
                    barmode='group',
                    title=f'Top {top_n_arch_display} Common Architectures (Excluding No_Domain)',
                    hover_data=['Translated Architecture'],
                    labels={'Architecture': 'Domain Architecture (Raw)', 'Count': 'Number of Proteins'},
                    color_discrete_map={'ESP_Count': arcadia_colors_manual.get('amber', '#F28360'), 
                                        'Non_ESP_Count': arcadia_colors_manual.get('aegean', '#5088C5')}
                )
                fig_arch_common_no_nodomain.update_xaxes(
                    tickangle=45, 
                    categoryorder='array', # Use the order from the sorted df_arch_common_no_nodomain
                    categoryarray=df_arch_common_no_nodomain.index.tolist(),
                    ticktext = [truncate_string(s, 50) for s in df_arch_common_no_nodomain.index], 
                    tickvals = df_arch_common_no_nodomain.index
                )
                fig_arch_common_no_nodomain.show()
            else:
                print("\nNo common architectures found after excluding 'No_Domain'.")

        else:
            print("No common architectures found between ESPs and Non-ESPs in the high-quality hits subset.")

else:
    print("\nSkipping architecture comparison: Input DataFrames empty or required columns missing.")


print("\n\n--- Cell 10 (Domain Architecture Comparison) Complete ---")
print("This cell compared domain architectures between ESPs and Non-ESPs with high-quality eukaryotic hits.")
print("Review the tables and plots, especially the list of Non-ESP architectures containing ESP-like domains.")



In [ ]:
# Cell 11: IPR Overlap Analysis (ESP vs. Non-ESP High-Quality Hits)

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time
from collections import Counter
import re # For parsing IPRs

# --- Plotly Imports ---
# Assuming plotly express and graph_objects were imported as px and go in Cell 1
# import plotly.express as px 
# import plotly.graph_objects as go 

print("\n\n--- Cell 11: IPR Overlap Analysis (ESP vs Non-ESP High-Quality Hits) ---")

# --- Assumptions ---
# This cell assumes Cell 1, 2, and 7 have been run successfully.
# It relies on the following DataFrames being available and populated:
#   - df_hq_esps: ESPs with high-quality hits (from Cell 7).
#   - df_hq_non_esps: Non-ESPs with high-quality hits (from Cell 7).
# It also relies on column names (ipr_col), helper functions (translate_architecture - though not strictly needed here),
# variables (ipr_lookup), and output directories (output_summary_dir_phase1) defined in Cell 1.

# --- Configuration ---
top_n_ipr_compare = 25 # How many top shared/unique IPRs to show

# --- Check if necessary DataFrames/Variables exist ---
if 'df_hq_esps' not in locals() or 'df_hq_non_esps' not in locals():
    print("ERROR: DataFrames 'df_hq_esps' and/or 'df_hq_non_esps' not found.")
    print("       Cannot perform comparison. Please ensure Cell 7 ran successfully.")
    if 'df_hq_esps' not in locals(): df_hq_esps = pd.DataFrame()
    if 'df_hq_non_esps' not in locals(): df_hq_non_esps = pd.DataFrame()

if 'ipr_col' not in locals():
     print("ERROR: Variable 'ipr_col' not defined. Please ensure Cell 1 ran.")
     # Exit or assign dummy value if you want to test structure
     # sys.exit()
     ipr_col = 'IPR_Signatures' # Assign dummy - analysis will likely fail later
     
if 'ipr_lookup' not in locals():
     print("WARNING: 'ipr_lookup' dictionary not found. Descriptions will not be available.")
     ipr_lookup = {}

# --- Helper function to get IPR counts from a DataFrame ---
def get_ipr_counts(df, ipr_column_name):
    """Parses IPR strings and returns a Counter object of IPR frequencies."""
    if ipr_column_name not in df.columns:
        print(f"Warning: IPR column '{ipr_column_name}' not found in DataFrame. Returning empty Counter.")
        return Counter()
        
    all_iprs_list = []
    for ipr_string_raw in df[ipr_column_name].dropna():
        processed_ipr_string = str(ipr_string_raw).replace('|', ';')
        individual_iprs = processed_ipr_string.split(';')
        for ipr_item in individual_iprs:
            stripped_ipr = ipr_item.strip()
            if stripped_ipr and stripped_ipr.startswith("IPR"): # Basic check for IPR format
                all_iprs_list.append(stripped_ipr)
            # else: # Optional: log skipped non-IPR items
            #    if stripped_ipr: logging.debug(f"Skipping non-IPR item: {stripped_ipr}")
                
    return Counter(all_iprs_list)

# --- Calculate IPR Counts for Both Groups ---
ipr_counts_esp = Counter()
ipr_counts_non_esp = Counter()

if not df_hq_esps.empty:
    print("Calculating IPR counts for ESPs with high-quality hits...")
    ipr_counts_esp = get_ipr_counts(df_hq_esps, ipr_col)
    print(f"  Found {len(ipr_counts_esp)} unique IPR IDs in ESPs.")
else:
    print("ESP subset is empty, cannot calculate IPR counts.")

if not df_hq_non_esps.empty:
    print("Calculating IPR counts for Non-ESPs with high-quality hits...")
    ipr_counts_non_esp = get_ipr_counts(df_hq_non_esps, ipr_col)
    print(f"  Found {len(ipr_counts_non_esp)} unique IPR IDs in Non-ESPs.")
else:
    print("Non-ESP subset is empty, cannot calculate IPR counts.")

# --- Analyze Overlap ---
if ipr_counts_esp and ipr_counts_non_esp:
    print("\n--- Analyzing IPR Overlap ---")
    
    # Convert Counters to DataFrames for easier merging
    df_ipr_esp = pd.DataFrame(ipr_counts_esp.items(), columns=['IPR_ID', 'ESP_Count']).set_index('IPR_ID')
    df_ipr_non_esp = pd.DataFrame(ipr_counts_non_esp.items(), columns=['IPR_ID', 'Non_ESP_Count']).set_index('IPR_ID')
    
    # Outer merge to get all IPRs and their counts in each group
    df_ipr_compare = pd.merge(df_ipr_esp, df_ipr_non_esp, left_index=True, right_index=True, how='outer').fillna(0).astype(int)
    df_ipr_compare['Total_Count'] = df_ipr_compare['ESP_Count'] + df_ipr_compare['Non_ESP_Count']
    
    # Add descriptions
    df_ipr_compare['Description'] = df_ipr_compare.index.map(lambda x: ipr_lookup.get(x, {}).get('Name', 'N/A'))
    df_ipr_compare['Type'] = df_ipr_compare.index.map(lambda x: ipr_lookup.get(x, {}).get('Type', 'N/A'))
    
    df_ipr_compare = df_ipr_compare.sort_values('Total_Count', ascending=False)
    
    # 1. Common IPRs
    df_ipr_common = df_ipr_compare[(df_ipr_compare['ESP_Count'] > 0) & (df_ipr_compare['Non_ESP_Count'] > 0)].copy()
    print(f"\nFound {len(df_ipr_common)} IPR signatures common to both ESP and Non-ESP hits.")
    
    if not df_ipr_common.empty:
        print(f"\nTop {top_n_ipr_compare} Common IPR Signatures (Sorted by Total Count):")
        display_cols_common = ['ESP_Count', 'Non_ESP_Count', 'Total_Count', 'Description', 'Type']
        try:
            print(df_ipr_common[display_cols_common].head(top_n_ipr_compare).to_markdown())
        except ImportError:
            print(df_ipr_common[display_cols_common].head(top_n_ipr_compare))
            
        # Visualize top common IPRs
        df_common_plot_data = df_ipr_common.head(top_n_ipr_compare).reset_index()
        df_common_plot_data['Label'] = df_common_plot_data['Description'].apply(lambda x: truncate_string(x, 40)) + " (" + df_common_plot_data['IPR_ID'] + ")"
        
        df_common_melt = df_common_plot_data.melt(
            id_vars=['IPR_ID', 'Label', 'Description', 'Type'], 
            value_vars=['ESP_Count', 'Non_ESP_Count'],
            var_name='Group', value_name='Count'
        )
        
        fig_ipr_common = px.bar(
            df_common_melt,
            x='Label',
            y='Count',
            color='Group',
            barmode='group',
            title=f'Top {top_n_ipr_compare} Common IPR Frequencies (ESP vs Non-ESP Hits)',
            hover_data=['IPR_ID', 'Description', 'Type'],
            labels={'Label': 'IPR Signature', 'Count': 'Number of Proteins'},
            color_discrete_map={'ESP_Count': arcadia_colors_manual.get('amber', '#F28360'), 
                                'Non_ESP_Count': arcadia_colors_manual.get('aegean', '#5088C5')}
        )
        fig_ipr_common.update_xaxes(tickangle=45, categoryorder='total descending') # Order by total count implicitly via melt order
        fig_ipr_common.show()
        
    # 2. Unique IPRs
    df_ipr_esp_unique = df_ipr_compare[(df_ipr_compare['ESP_Count'] > 0) & (df_ipr_compare['Non_ESP_Count'] == 0)].copy()
    df_ipr_non_esp_unique = df_ipr_compare[(df_ipr_compare['ESP_Count'] == 0) & (df_ipr_compare['Non_ESP_Count'] > 0)].copy()

    print(f"\nFound {len(df_ipr_esp_unique)} IPR signatures unique to ESP hits.")
    if not df_ipr_esp_unique.empty:
        print(f"Top {top_n_ipr_compare} Unique IPRs in ESP Hits:")
        display_cols_unique = ['ESP_Count', 'Description', 'Type']
        try:
            print(df_ipr_esp_unique[display_cols_unique].head(top_n_ipr_compare).to_markdown())
        except ImportError:
            print(df_ipr_esp_unique[display_cols_unique].head(top_n_ipr_compare))

    print(f"\nFound {len(df_ipr_non_esp_unique)} IPR signatures unique to Non-ESP hits.")
    if not df_ipr_non_esp_unique.empty:
        print(f"Top {top_n_ipr_compare} Unique IPRs in Non-ESP Hits:")
        display_cols_unique_non = ['Non_ESP_Count', 'Description', 'Type']
        try:
            print(df_ipr_non_esp_unique[display_cols_unique_non].head(top_n_ipr_compare).to_markdown())
        except ImportError:
            print(df_ipr_non_esp_unique[display_cols_unique_non].head(top_n_ipr_compare))
            
    # Save unique lists
    unique_esp_filename = os.path.join(output_summary_dir_phase1, "unique_iprs_in_esp_hq_hits.csv")
    unique_non_esp_filename = os.path.join(output_summary_dir_phase1, "unique_iprs_in_non_esp_hq_hits.csv")
    try:
        df_ipr_esp_unique.reset_index().to_csv(unique_esp_filename, index=False)
        df_ipr_non_esp_unique.reset_index().to_csv(unique_non_esp_filename, index=False)
        print(f"\nSaved unique IPR lists to '{unique_esp_filename}' and '{unique_non_esp_filename}'")
    except Exception as e:
        print(f"Error saving unique IPR lists: {e}")


else:
    print("\nSkipping IPR overlap analysis: Could not calculate IPR counts for both ESP and Non-ESP groups.")


print("\n\n--- Cell 11 (IPR Overlap Analysis) Complete ---")
print("This cell analyzed the overlap and differences in IPR signatures between ESPs and Non-ESPs with high-quality eukaryotic hits.")
print("Examine the common and unique IPR lists for functional insights.")



In [ ]:
# Cell 12: Integrate Eukaryotic Hit Names with Asgard Domain/IPR Analysis

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time
from collections import Counter
import re 

# --- Plotly Imports ---
# Assuming plotly express and graph_objects were imported as px and go in Cell 1
# import plotly.express as px 
# import plotly.graph_objects as go 

print("\n\n--- Cell 12: Integrate Eukaryotic Hit Names with Asgard Domain/IPR Analysis ---")

# --- Assumptions ---
# This cell assumes Cell 1 and Cell 2 have been run successfully.
# It relies on the DataFrame 'df_full' being loaded from the LATEST database CSV 
# (v1.9 with Euk protein names) in Cell 1 (or re-loaded here).
# It also assumes 'df_high_quality_hits' was defined in Cell 2 based on quality/coverage filters.
# It relies on column names, helper functions (translate_architecture), ipr_lookup, 
# color maps, and output directories defined in Cell 1.

# --- Configuration ---
# Input DB path (ensure this is the LATEST version with Euk names)
db_path_v1_9 = 'proteome_database_combined_v2.0.csv' 

# Filtering criteria for high-quality hits (should match Cell 2 for consistency)
# Re-define here or ensure they are available from Cell 2's execution state
min_query_coverage_filter = 0.70 
min_subject_coverage_filter = 0.50
min_pident_filter = 30.0 

# Keywords to search for in Eukaryotic Hit Protein Names (Case-Insensitive)
# Customize this list based on functions of interest (ESP-like, etc.)
interesting_euk_keywords = [
    'kinase', 'phosphatase', 'gtpase', 'ras', 'rho', 'rab', 'ran', 'arl', # Signaling/GTPases
    'ubiquitin', 'nedd8', 'sumo', 'ubl', 'e1', 'e2', 'e3', 'ligase', 'dubb', # UBL system
    'actin', 'tubulin', 'cytoskeleton', 'kinesin', 'dynein', 'myosin', # Cytoskeleton/Motors
    'snare', 'escrt', 'vps', 'sec', 'adaptin', 'clathrin', 'copi', 'copii', # Trafficking
    'nuclear pore', 'importin', 'exportin', 'karyopherin', # Nuclear transport
    'wd40', 'tpr', 'ankyrin', 'armadillo', 'heat repeat', # Repeat domains common in Euks
    'cadherin', 'integrin', 'catenin', # Adhesion (less expected but possible)
    'signal recognition particle', 'translocon', # Secretion machinery
    'receptor', 'transporter', 'channel' # Membrane proteins
]
# Create a regex pattern for searching (case-insensitive OR match)
keyword_pattern = r'\b(?:' + '|'.join(re.escape(kw) for kw in interesting_euk_keywords) + r')\b'
print(f"Filtering Euk Hit Names using pattern: {keyword_pattern[:100]}...") # Print start of pattern

top_n_display = 20 # For tables/plots in this cell

# --- Check/Load Data and Re-filter for High Quality Hits ---
# It's safer to reload the v1.9 data here and re-apply filters
# to ensure we have the Euk_Hit_Protein_Name column.
print(f"\nLoading latest database: {db_path_v1_9}")
if 'df_full' not in locals() or euk_hit_protein_name_col not in df_full.columns:
    try:
        df_full = pd.read_csv(db_path_v1_9, low_memory=False)
        # Ensure ProteinID is string type right after loading
        if protein_id_col in df_full.columns:
             df_full[protein_id_col] = df_full[protein_id_col].astype(str)
        print(f"Successfully loaded database v1.9. Shape: {df_full.shape}")
        # Re-calculate coverage if necessary (if Cell 2 wasn't just run with v1.9)
        # Basic check - recalculate if coverage columns are missing or mostly NaN
        needs_coverage_recalc = (
            query_coverage_col not in df_full.columns or 
            subject_coverage_col not in df_full.columns or
            df_full[query_coverage_col].isna().mean() > 0.95 or # If >95% NaN, likely not calculated on this df
            df_full[subject_coverage_col].isna().mean() > 0.95
        )
        if needs_coverage_recalc:
            print("Coverage columns missing or incomplete. Re-running coverage calculation steps from Cell 2...")
            # --- Add code snippet from Cell 2's coverage calculation here if needed ---
            # This involves re-parsing DIAMOND, merging details, and calculating coverage.
            # For brevity, assuming coverage columns ARE present in the loaded v1.9 file for now.
            # If they are NOT, you MUST copy the relevant code block from Cell 2 here.
            print("WARNING: Coverage recalculation logic not included here for brevity. Assuming columns exist in CSV.")
            pass 
            
    except FileNotFoundError:
        print(f"ERROR: Database file '{db_path_v1_9}' not found.")
        sys.exit() # Stop if we can't load the required data
    except Exception as e:
        print(f"An error occurred while loading database '{db_path_v1_9}': {e}")
        sys.exit()

# Re-apply high-quality filter
required_filter_cols = [group_col, hit_flag_col, query_coverage_col, subject_coverage_col, euk_hit_pident_col]
if all(col in df_full.columns for col in required_filter_cols):
     df_high_quality_hits = df_full[
        (df_full[group_col] == 'Asgard') & 
        (df_full[hit_flag_col] == True) &
        (df_full[query_coverage_col] >= min_query_coverage_filter) &
        (df_full[subject_coverage_col] >= min_subject_coverage_filter) &
        (df_full[euk_hit_pident_col] >= min_pident_filter)
    ].copy()
     print(f"\nFiltered {len(df_high_quality_hits)} high-quality Asgard hits based on coverage/pident.")
else:
     print(f"ERROR: Cannot filter for high-quality hits, missing required columns: {[c for c in required_filter_cols if c not in df_full.columns]}")
     df_high_quality_hits = pd.DataFrame() # Ensure it's defined

# --- Check if necessary DataFrames/Columns exist ---
if df_high_quality_hits.empty:
    print("ERROR: DataFrame 'df_high_quality_hits' is empty after filtering. Cannot proceed.")
elif euk_hit_protein_name_col not in df_high_quality_hits.columns:
    print(f"ERROR: Column '{euk_hit_protein_name_col}' not found in df_high_quality_hits.")
elif esp_col not in df_high_quality_hits.columns:
     print(f"ERROR: Column '{esp_col}' not found in df_high_quality_hits.")
else:
    # --- Filter for Hits with Interesting Eukaryotic Protein Names ---
    print(f"\n--- Filtering for hits where Euk Protein Name contains keywords ---")
    # Ensure the column is string and handle NA
    df_high_quality_hits[euk_hit_protein_name_col] = df_high_quality_hits[euk_hit_protein_name_col].fillna('').astype(str)
    
    # Apply the regex filter (case-insensitive)
    keyword_hits_mask = df_high_quality_hits[euk_hit_protein_name_col].str.contains(keyword_pattern, case=False, regex=True, na=False)
    df_keyword_hits = df_high_quality_hits[keyword_hits_mask].copy()
    
    print(f"Found {len(df_keyword_hits)} high-quality hits where Euk Protein Name matches keywords.")

    if not df_keyword_hits.empty:
        # --- Analyze this interesting subset ---
        
        # Separate ESPs and Non-ESPs within this keyword-filtered subset
        df_keyword_esps = df_keyword_hits[df_keyword_hits[esp_col] == True].copy()
        df_keyword_non_esps = df_keyword_hits[df_keyword_hits[esp_col] == False].copy()
        print(f"  Keyword hits breakdown: {len(df_keyword_esps)} ESPs, {len(df_keyword_non_esps)} Non-ESPs.")

        # 1. Top Eukaryotic Protein Names in this subset (ESP vs Non-ESP)
        print("\n--- 1. Top Euk Protein Names in Keyword-Filtered Subset ---")
        if not df_keyword_esps.empty:
             df_keyword_esps['Cleaned_Euk_Prot_Name'] = df_keyword_esps[euk_hit_protein_name_col].apply(clean_protein_name)
             esp_keyword_name_counts = df_keyword_esps['Cleaned_Euk_Prot_Name'].value_counts()
             print(f"\nTop {top_n_display} Cleaned Euk Names (Keyword Hits - ESPs):")
             print(esp_keyword_name_counts.head(top_n_display).to_markdown())
        if not df_keyword_non_esps.empty:
             df_keyword_non_esps['Cleaned_Euk_Prot_Name'] = df_keyword_non_esps[euk_hit_protein_name_col].apply(clean_protein_name)
             non_esp_keyword_name_counts = df_keyword_non_esps['Cleaned_Euk_Prot_Name'].value_counts()
             print(f"\nTop {top_n_display} Cleaned Euk Names (Keyword Hits - Non-ESPs):")
             print(non_esp_keyword_name_counts.head(top_n_display).to_markdown())

        # 2. Domain Architectures of Non-ESPs hitting interesting Euk proteins
        print("\n--- 2. Domain Architectures of Non-ESPs hitting interesting Euk proteins ---")
        if not df_keyword_non_esps.empty and domain_arch_col in df_keyword_non_esps.columns:
            keyword_non_esp_arch_counts = df_keyword_non_esps[domain_arch_col].fillna('No_Domain').value_counts()
            df_keyword_non_esp_arch_plot = keyword_non_esp_arch_counts.head(top_n_display).reset_index()
            df_keyword_non_esp_arch_plot.columns = ['Architecture', 'Count']
            df_keyword_non_esp_arch_plot['Translated Architecture'] = df_keyword_non_esp_arch_plot['Architecture'].apply(translate_architecture, lookup_dict=ipr_lookup)
            
            print(f"\nTop {top_n_display} Architectures (Keyword Hits - Non-ESPs):")
            print(df_keyword_non_esp_arch_plot.to_markdown(index=False))

            # Plot
            fig_arch_kw_non_esp = px.bar(
                df_keyword_non_esp_arch_plot, x='Architecture', y='Count',
                title=f'Top {top_n_display} Architectures of Non-ESPs hitting Euk Proteins with Keywords',
                hover_data=['Translated Architecture'],
                labels={'Architecture': 'Domain Architecture (Raw)', 'Count': 'Number of Non-ESP Proteins'}
            )
            fig_arch_kw_non_esp.update_xaxes(tickangle=45, categoryorder='total descending',
                                          ticktext = [truncate_string(s, 50) for s in df_keyword_non_esp_arch_plot['Architecture']],
                                          tickvals = df_keyword_non_esp_arch_plot['Architecture'])
            fig_arch_kw_non_esp.show()
        else:
            print("No Non-ESPs found hitting Euk proteins with keywords, or domain column missing.")

        # 3. IPR Signatures of Non-ESPs hitting interesting Euk proteins
        print("\n--- 3. IPR Signatures of Non-ESPs hitting interesting Euk proteins ---")
        if not df_keyword_non_esps.empty and ipr_col in df_keyword_non_esps.columns and 'ipr_lookup' in locals():
            ipr_counts_kw_non_esp = get_ipr_counts(df_keyword_non_esps, ipr_col) # Use helper from Cell 11
            if ipr_counts_kw_non_esp:
                df_ipr_kw_non_esp = pd.DataFrame(ipr_counts_kw_non_esp.most_common(top_n_display), columns=['IPR_ID', 'Count'])
                df_ipr_kw_non_esp['Description'] = df_ipr_kw_non_esp['IPR_ID'].apply(lambda x: ipr_lookup.get(x, {}).get('Name', 'N/A'))
                df_ipr_kw_non_esp['Label'] = df_ipr_kw_non_esp['Description'].apply(lambda x: truncate_string(x, 60)) + " (" + df_ipr_kw_non_esp['IPR_ID'] + ")"

                print(f"\nTop {top_n_display} IPRs (Keyword Hits - Non-ESPs):")
                print(df_ipr_kw_non_esp[['IPR_ID', 'Count', 'Description']].to_markdown(index=False))

                # Plot
                fig_ipr_kw_non_esp = px.bar(
                    df_ipr_kw_non_esp, x='Label', y='Count',
                    title=f'Top {top_n_display} IPRs of Non-ESPs hitting Euk Proteins with Keywords',
                    hover_data=['IPR_ID', 'Description'],
                    labels={'Label': 'IPR Signature', 'Count': 'Number of Non-ESP Proteins'}
                )
                fig_ipr_kw_non_esp.update_xaxes(categoryorder='total descending')
                fig_ipr_kw_non_esp.show()
            else:
                print("No IPRs found for Non-ESPs hitting Euk proteins with keywords.")
        else:
            print(f"Skipping IPR analysis for keyword hits: Column '{ipr_col}' missing or ipr_lookup not defined.")

        # 4. Generate Summary Table of Potential "Missed" ESPs
        print("\n--- 4. Potential 'Missed' ESP Candidates (Non-ESPs hitting Euk proteins with keywords) ---")
        # Select relevant columns for the summary table
        summary_cols = [
            protein_id_col, orthogroup_col, broad_func_cat_col, domain_arch_col, ipr_col, 
            euk_hit_sseqid_col, euk_hit_organism_col, euk_hit_protein_name_col,
            euk_hit_pident_col, euk_hit_evalue_col, query_coverage_col, subject_coverage_col
        ]
        # Ensure columns exist before selecting
        summary_cols_present = [c for c in summary_cols if c in df_keyword_non_esps.columns]
        df_missed_esp_candidates = df_keyword_non_esps[summary_cols_present].copy()
        
        # Optionally add translated architecture
        if 'translate_architecture' in locals() and domain_arch_col in df_missed_esp_candidates.columns:
             df_missed_esp_candidates['Translated Architecture'] = df_missed_esp_candidates[domain_arch_col].apply(translate_architecture, lookup_dict=ipr_lookup)
             
        print(f"Found {len(df_missed_esp_candidates)} potential candidates.")
        print("Displaying first 15 candidates:")
        try:
            print(df_missed_esp_candidates.head(15).to_markdown(index=False))
        except ImportError:
            print(df_missed_esp_candidates.head(15))
            
        # Save this candidate list
        candidates_filename = os.path.join(output_summary_dir_phase1, "potential_missed_esp_candidates.csv")
        try:
             df_missed_esp_candidates.to_csv(candidates_filename, index=False)
             print(f"\nSaved potential 'missed' ESP candidate list to '{candidates_filename}'")
        except Exception as e:
             print(f"Error saving candidate list: {e}")

    else:
        print("No high-quality hits found where Euk Protein Name matches keywords.")

print("\n\n--- Cell 12 (Integrate Euk Names & Asgard Domains) Complete ---")
print("This cell filtered for Asgard proteins hitting Eukaryotic proteins with potentially")
print("interesting functions (based on keywords) and analyzed their domain content.")
print("The 'potential_missed_esp_candidates.csv' file lists Non-ESPs in this category.")
print("Focusing on these candidates for deeper dives (RBH, MSA, Phylogeny) is a good strategy for publication.")



In [ ]:
# Cell 13: Analysis of Eukaryotic Hit Protein Names & Focus on "No Domain" Hits

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time
from collections import Counter
import re 
import sys # Added for checking variable existence robustly

# --- Plotly Imports ---
# Assuming plotly express and graph_objects were imported as px and go in Cell 1
import plotly.express as px 
import plotly.graph_objects as go 

print("\n\n--- Cell 13: Euk Hit Protein Name Analysis & 'No Domain' Focus ---")

# --- Assumptions ---
# This cell assumes Cell 1 and the consolidated Cell 2 have been run successfully.
# It relies on the DataFrame 'df_high_quality_hits' being available and populated.
# It also relies on column names (euk_hit_protein_name_col, num_domains_col, domain_arch_col), 
# helper functions (clean_protein_name), and output directories defined in Cell 1.

# --- Configuration ---
top_n_names_display = 30 # How many top names to show in tables/plots

# --- Check if necessary DataFrames/Variables exist ---
required_dataframes = ['df_high_quality_hits']
required_variables = [
    'euk_hit_protein_name_col', 'num_domains_col', 'domain_arch_col', 
    'clean_protein_name', 'output_summary_dir_phase1' 
] # Removed function checks as they should be defined if Cell 1 ran

missing_data = False
for df_name in required_dataframes:
    if df_name not in locals() or locals()[df_name].empty:
        print(f"ERROR: Required DataFrame '{df_name}' is missing or empty. Please run previous cells (especially Cell 2).")
        missing_data = True
        # Define as empty to prevent downstream errors
        if df_name == 'df_high_quality_hits': df_high_quality_hits = pd.DataFrame() 
        
for var_name in required_variables:
     if var_name not in locals():
         print(f"ERROR: Required variable or function '{var_name}' is missing. Please run previous cells (especially Cell 1).")
         missing_data = True
         
if missing_data:
    print("Cannot proceed with Cell 13 due to missing data or variables.")
    # Optionally raise an error: raise NameError("Prerequisite data/variables missing.")
else:
    # --- Analyze Top Eukaryotic Hit Protein Names (Overall High-Quality Hits) ---
    print(f"\n--- Analyzing Eukaryotic Hit Protein Names for ALL {len(df_high_quality_hits)} High-Quality Hits ---")
    
    # Apply cleaning function (defined in Cell 1 or previous cells)
    if euk_hit_protein_name_col in df_high_quality_hits.columns:
        df_high_quality_hits['Cleaned_Euk_Prot_Name'] = df_high_quality_hits[euk_hit_protein_name_col].apply(clean_protein_name)
        overall_hit_name_counts = df_high_quality_hits['Cleaned_Euk_Prot_Name'].value_counts()
        
        print(f"\nTop {top_n_names_display} Cleaned Eukaryotic Protein Names (Overall High-Quality Hits):")
        # Filter out generic 'Uncharacterized' or 'Unknown' before displaying top N
        overall_hit_name_counts_filtered = overall_hit_name_counts[~overall_hit_name_counts.index.isin(['Uncharacterized', 'Unknown/Not Found'])]
        try:
            print(overall_hit_name_counts_filtered.head(top_n_names_display).to_markdown())
        except ImportError:
            print(overall_hit_name_counts_filtered.head(top_n_names_display))
            
        # Save summary (Full list)
        overall_names_filename = os.path.join(output_summary_dir_phase1, "overall_hq_hits_top_euk_prot_names.csv")
        try:
             overall_hit_name_counts.reset_index().rename(columns={'index':'Cleaned_Euk_Prot_Name', 'Cleaned_Euk_Prot_Name':'Count'}).to_csv(overall_names_filename, index=False)
             print(f"Saved overall hit protein name counts to '{overall_names_filename}'")
        except Exception as e:
             print(f"Error saving overall hit protein name counts: {e}")

        # Plot (Top N filtered names)
        df_overall_names_plot = overall_hit_name_counts_filtered.head(top_n_names_display).reset_index()
        if not df_overall_names_plot.empty: # Check if anything remains after filtering
            df_overall_names_plot.columns = ['Cleaned Name', 'Count']
            fig_names_overall = px.bar(
                df_overall_names_plot,
                x='Cleaned Name',
                y='Count',
                title=f'Top {top_n_names_display} Eukaryotic Hit Protein Names (Functionally Annotated) for Overall High-Quality Asgard Hits',
                labels={'Cleaned Name': 'Cleaned Eukaryotic Protein Name', 'Count': 'Number of Asgard Hits'}
            )
            fig_names_overall.update_xaxes(tickangle=45, categoryorder='total descending')
            fig_names_overall.show()
        else:
            print("No functionally annotated protein names found in the top overall high-quality hits after cleaning.")
    else:
         print(f"Cannot analyze Euk Hit Names: Column '{euk_hit_protein_name_col}' not found.")


    # --- Analyze Hits for "No Domain" Asgard Proteins ---
    print(f"\n\n--- Analyzing Eukaryotic Hits for 'No Domain' Asgard Proteins ---")
    
    # Identify "No Domain" proteins within the high-quality hits
    # Prefer checking Num_Domains is NaN, fallback to checking Domain_Architecture
    if num_domains_col in df_high_quality_hits.columns:
        no_domain_mask = df_high_quality_hits[num_domains_col].isna()
        print(f"Identifying 'No Domain' proteins based on '{num_domains_col}' being NaN.")
    elif domain_arch_col in df_high_quality_hits.columns:
         no_domain_mask = df_high_quality_hits[domain_arch_col].fillna('No_Domain') == 'No_Domain'
         print(f"Identifying 'No Domain' proteins based on '{domain_arch_col}' being 'No_Domain' or NaN.")
    else:
         print("Cannot identify 'No Domain' proteins, required columns missing.")
         no_domain_mask = pd.Series(False, index=df_high_quality_hits.index) # Create mask of False

    df_no_domain_hits = df_high_quality_hits[no_domain_mask].copy()
    
    if not df_no_domain_hits.empty:
        print(f"Found {len(df_no_domain_hits)} 'No Domain' Asgard proteins within the high-quality hits subset.")

        # Analyze the Euk Hit Protein Names for this subset
        # Use the already cleaned names from the parent df if available
        if 'Cleaned_Euk_Prot_Name' in df_no_domain_hits.columns:
             no_domain_hit_name_counts = df_no_domain_hits['Cleaned_Euk_Prot_Name'].value_counts()
        elif euk_hit_protein_name_col in df_no_domain_hits.columns: # Apply cleaning if needed
             df_no_domain_hits['Cleaned_Euk_Prot_Name'] = df_no_domain_hits[euk_hit_protein_name_col].apply(clean_protein_name)
             no_domain_hit_name_counts = df_no_domain_hits['Cleaned_Euk_Prot_Name'].value_counts()
        else:
             print("Cannot get Euk Hit names for No Domain subset.")
             no_domain_hit_name_counts = pd.Series(dtype='int64') # Empty series

        
        if not no_domain_hit_name_counts.empty:
            print(f"\nTop {top_n_names_display} Cleaned Eukaryotic Protein Names (Hits for 'No Domain' Asgard Proteins):")
            # Filter out generic 'Uncharacterized' or 'Unknown' 
            no_domain_hit_name_counts_filtered = no_domain_hit_name_counts[~no_domain_hit_name_counts.index.isin(['Uncharacterized', 'Unknown/Not Found'])]
            try:
                print(no_domain_hit_name_counts_filtered.head(top_n_names_display).to_markdown())
            except ImportError:
                print(no_domain_hit_name_counts_filtered.head(top_n_names_display))
                
            # Save summary (Full list)
            no_domain_names_filename = os.path.join(output_summary_dir_phase1, "no_domain_hq_hits_top_euk_prot_names.csv")
            try:
                 no_domain_hit_name_counts.reset_index().rename(columns={'index':'Cleaned_Euk_Prot_Name', 'Cleaned_Euk_Prot_Name':'Count'}).to_csv(no_domain_names_filename, index=False)
                 print(f"Saved 'No Domain' hit protein name counts to '{no_domain_names_filename}'")
            except Exception as e:
                 print(f"Error saving 'No Domain' hit protein name counts: {e}")

            # Plot (Top N filtered names)
            df_no_domain_names_plot = no_domain_hit_name_counts_filtered.head(top_n_names_display).reset_index()
            if not df_no_domain_names_plot.empty: # Check if anything remains after filtering
                df_no_domain_names_plot.columns = ['Cleaned Name', 'Count']
                fig_names_no_domain = px.bar(
                    df_no_domain_names_plot,
                    x='Cleaned Name',
                    y='Count',
                    title=f"Top {top_n_names_display} Eukaryotic Hit Protein Names (Functionally Annotated) for 'No Domain' Asgard Proteins",
                    labels={'Cleaned Name': 'Cleaned Eukaryotic Protein Name', 'Count': "Number of 'No Domain' Asgard Hits"}
                )
                fig_names_no_domain.update_xaxes(tickangle=45, categoryorder='total descending')
                fig_names_no_domain.show()
            else:
                print("No functionally annotated protein names found in the top hits for 'No Domain' Asgard proteins after cleaning.")
        else:
             print("Could not calculate Euk Hit name counts for 'No Domain' subset.")

    else:
        print("No 'No Domain' Asgard proteins found within the high-quality hits subset.")


print("\n\n--- Cell 13 (Euk Hit Name Analysis & 'No Domain' Focus) Complete ---")
print("This cell analyzed the Eukaryotic protein names associated with high-quality hits,")
print("both overall and specifically for Asgard proteins lacking annotated domains.")
print("Consider if the hits for 'No Domain' proteins provide functional clues.")



In [ ]:
# Cell 14: Prepare for Reciprocal Best Hit (RBH) Analysis - Select & Extract Euk Sequences (All HQ Hits)

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time
import re 
import sys 
import logging 
from pathlib import Path 
# Ensure Biopython is installed: pip install biopython
try:
    from Bio import SeqIO 
    from Bio.SeqRecord import SeqRecord
    from Bio.Seq import Seq
except ImportError:
    print("ERROR: Biopython is required for this script. Please install it: pip install biopython")
    sys.exit(1)

print("\n\n--- Cell 14: Prepare for Reciprocal Best Hit (RBH) Analysis (All High-Quality Hits) ---")

# --- Assumptions ---
# This cell assumes Cell 1 and Cell 2 have been run successfully.
# It relies on 'df_high_quality_hits' being available and populated (defined in Cell 2).
# It also relies on column names (euk_hit_sseqid_col, protein_id_col) and 
# file paths (euk_fasta_path, output_summary_dir_phase1) defined in Cell 1.

# --- Configuration ---
# Output: FASTA file containing sequences of ALL unique eukaryotic hits from the HQ set
output_euk_hits_fasta = Path(output_summary_dir_phase1) / "all_hq_euk_hits_for_rbh.fasta"
# Output: TSV file mapping Asgard ID to its HQ Euk Hit ID (for easy comparison later)
output_rbh_pairs_map = Path(output_summary_dir_phase1) / "asgard_euk_hq_rbh_pairs_to_test.tsv"

# --- Check if necessary DataFrames/Variables exist ---
required_dataframes = ['df_high_quality_hits']
required_variables = [
    'euk_fasta_path', 'euk_hit_sseqid_col', 'protein_id_col', 
    'output_summary_dir_phase1'
]

missing_data = False
for df_name in required_dataframes:
    if df_name not in locals() or locals()[df_name].empty:
        print(f"ERROR: Required DataFrame '{df_name}' is missing or empty. Please run previous cells (especially Cell 2).")
        missing_data = True
        if df_name == 'df_high_quality_hits': df_high_quality_hits = pd.DataFrame() 
        
for var_name in required_variables:
     if var_name not in locals():
         print(f"ERROR: Required variable or function '{var_name}' is missing. Please run previous cells (especially Cell 1).")
         missing_data = True
         
if missing_data:
    print("Cannot proceed with Cell 14 due to missing data or variables.")
    # Optionally raise an error: raise NameError("Prerequisite data/variables missing.")
else:
    # --- Step 1: Select ALL Unique Target Eukaryotic Hit SSEQIDs ---
    print("\n--- Selecting ALL Unique Eukaryotic Hit IDs from High-Quality Set ---")
    
    if euk_hit_sseqid_col not in df_high_quality_hits.columns:
         print(f"ERROR: Column '{euk_hit_sseqid_col}' not found. Cannot get Eukaryotic IDs.")
         euk_ids_to_extract = set()
         df_rbh_targets = pd.DataFrame() # Ensure df is defined
    else:
         # Get unique Euk IDs directly from the high-quality hits df
         euk_ids_to_extract = set(df_high_quality_hits[euk_hit_sseqid_col].dropna().astype(str))
         print(f"Found {len(euk_ids_to_extract)} unique Eukaryotic Hit SSEQIDs to extract from {len(df_high_quality_hits)} high-quality hits.")
         # Define the target pairs map (all high-quality hits)
         df_rbh_targets = df_high_quality_hits[[protein_id_col, euk_hit_sseqid_col]].copy()


    # --- Step 2: Extract Eukaryotic FASTA Sequences ---
    if not euk_ids_to_extract:
        print("\nNo Eukaryotic IDs selected to extract sequences.")
    elif not euk_fasta_path.is_file():
        print(f"\nERROR: Eukaryotic FASTA file '{euk_fasta_path}' not found. Cannot extract sequences.")
    else:
        print(f"\n--- Extracting {len(euk_ids_to_extract)} Eukaryotic FASTA sequences ---")
        print(f"Reading from: {euk_fasta_path}")
        print(f"Writing to: {output_euk_hits_fasta}")
        
        sequences_written = 0
        sequences_found = set() # Keep track of IDs we actually found and wrote
        
        try:
            with open(output_euk_hits_fasta, "w") as f_out:
                fasta_sequences = SeqIO.parse(euk_fasta_path, "fasta")
                for record in fasta_sequences:
                    # Extract ID from record (handle potential variations)
                    current_id = record.id 
                    # Also check the first part before pipe/space if needed
                    first_part_id = record.description.split('|')[0].split(' ')[0].lstrip('>')
                    
                    # Check if either the direct ID or the first part matches our target list
                    id_in_target = None
                    if current_id in euk_ids_to_extract:
                        id_in_target = current_id
                    elif first_part_id in euk_ids_to_extract:
                         id_in_target = first_part_id
                         # Optional: Log if we had to use the first part ID
                         # if current_id != first_part_id: 
                         #    logging.debug(f"Using parsed ID '{first_part_id}' instead of SeqIO ID '{current_id}' for matching.")

                    if id_in_target:
                        # Use the ID that was in the target list as the key/header
                        if id_in_target not in sequences_found: # Avoid writing duplicates
                             # Create a new SeqRecord with just the ID we need for the reverse search
                             output_record = SeqRecord(record.seq, id=id_in_target, description="") 
                             SeqIO.write(output_record, f_out, "fasta")
                             sequences_written += 1
                             sequences_found.add(id_in_target)
                             
                             if sequences_written % 10000 == 0: # Log less frequently for large numbers
                                  logging.info(f"  Written {sequences_written:,} sequences...")
                                 
            print(f"\nFinished extraction. Wrote {sequences_written} sequences to '{output_euk_hits_fasta}'.")
            
            # Check how many target IDs were not found
            ids_not_found_set = euk_ids_to_extract - sequences_found
            ids_not_found_count = len(ids_not_found_set)
            if ids_not_found_count > 0:
                 logging.warning(f"{ids_not_found_count} target Eukaryotic SSEQIDs were not found in the FASTA file.")
                 # Optionally save the list of missing IDs
                 missing_ids_file = Path(output_summary_dir_phase1) / "rbh_euk_ids_not_found_in_fasta.txt"
                 with open(missing_ids_file, 'w') as f_miss:
                     for missing_id in sorted(list(ids_not_found_set)):
                         f_miss.write(f"{missing_id}\n")
                 logging.warning(f"List of missing IDs saved to '{missing_ids_file}'")
                 
        except Exception as e:
            print(f"ERROR during FASTA extraction: {e}")

    # --- Step 3: Save the Asgard-Euk Pair Mapping ---
    if not df_rbh_targets.empty:
        print(f"\n--- Saving Asgard-Euk Pair Mapping for RBH ---")
        try:
            # Select only the Asgard ID and its corresponding Euk Hit ID from the HQ set
            df_pairs_to_save = df_rbh_targets[[protein_id_col, euk_hit_sseqid_col]].drop_duplicates().dropna()
            df_pairs_to_save.to_csv(output_rbh_pairs_map, sep='\t', index=False, header=True)
            print(f"Saved {len(df_pairs_to_save)} Asgard-Euk pairs to test to '{output_rbh_pairs_map}'")
        except Exception as e:
            print(f"ERROR saving pairs map: {e}")


print("\n\n--- Cell 14 (RBH Preparation - All HQ Hits) Complete ---")
print("Next Steps:")
print("1. Create a DIAMOND database from your Asgard proteome FASTA file (if not done already).")
print(f"   Example command: diamond makedb --in <path_to_asgard.fasta> -d asgard_proteome_db")
print(f"2. Run the reverse DIAMOND search using '{output_euk_hits_fasta}' as query against the Asgard DB.")
print(f"   Example command: diamond blastp -d asgard_proteome_db.dmnd -q '{output_euk_hits_fasta}' -o reverse_diamond_hits.tsv --outfmt 6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore --sensitive -k 10 --threads <N>")
print(f"3. Create a new cell (Cell 15) to parse 'reverse_diamond_hits.tsv' and '{output_rbh_pairs_map}' to identify reciprocal best hits.")



In [ ]:
# Cell 15: Identify Reciprocal Best Hits (RBH)

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time
import logging # Use logger defined in Cell 1
from pathlib import Path 
import sys # For exit checks

print("\n\n--- Cell 15: Identify Reciprocal Best Hits (RBH) ---")

# --- Assumptions ---
# This cell assumes Cell 1 and Cell 2 have been run successfully.
# It assumes the reverse DIAMOND search was run externally, producing 'reverse_diamond_hits.tsv'.
# It relies on 'df_full' (with coverage & annotations, saved as v2.1 in Cell 2) being available.
# It uses the mapping file 'asgard_euk_hq_rbh_pairs_to_test.tsv' generated in Cell 14.
# It relies on column name variables defined in Cell 1.

# --- Configuration ---
# Input: Path to the database file saved at the end of Cell 2 (with coverage)
db_with_coverage_path = output_db_with_coverage_path # Defined in Cell 1, points to v2.1

# Input: Mapping file of Asgard -> Euk pairs tested (from Cell 14)
forward_pairs_map_path = output_rbh_pairs_map # Defined in Cell 14

# Input: Results from the reverse DIAMOND search (Euk query -> Asgard DB)
reverse_diamond_results_path = Path(output_summary_dir_phase1) / "reverse_diamond_hits.tsv"

# Output: Database updated with an 'Is_RBH' column
output_db_with_rbh_path = Path(output_summary_dir_phase1) / 'proteome_database_combined_v2.2_rbh.csv'
# Output: List of confirmed RBH pairs
output_rbh_confirmed_pairs_path = Path(output_summary_dir_phase1) / "asgard_euk_rbh_confirmed_pairs.tsv"

# E-value threshold for filtering reverse hits (can be same or different from forward)
reverse_e_value_threshold = 1e-10 

# DIAMOND output columns for reverse search (should match the --outfmt 6 used)
reverse_diamond_col_names = [
    'qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen',
    'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore' 
    # Add qlen, slen if they were included in your reverse search output format
]
# Rename columns for clarity after parsing reverse hits
reverse_col_rename = {
    'qseqid': euk_hit_sseqid_col, # Query in reverse search is the Euk Hit SSEQID
    'sseqid': 'Reverse_Hit_Asgard_ID', # Subject in reverse search is the Asgard ProteinID
    'evalue': 'Reverse_Hit_Evalue',
    'bitscore': 'Reverse_Hit_Bitscore'
    # Add others if needed
}
rbh_flag_col = 'Is_RBH' # Name for the new boolean column

# --- Check if necessary Files/DataFrames exist ---
if 'df_full' not in locals() or df_full.empty:
     # Attempt to load from the file saved in Cell 2
     if output_db_with_coverage_path.is_file():
          print(f"Loading database with coverage from: {output_db_with_coverage_path}")
          try:
              df_full = pd.read_csv(output_db_with_coverage_path, low_memory=False)
              # Ensure ProteinID is string
              if protein_id_col in df_full.columns:
                   df_full[protein_id_col] = df_full[protein_id_col].astype(str)
              print(f"Loaded df_full. Shape: {df_full.shape}")
          except Exception as e:
              print(f"ERROR: Failed to load database from {output_db_with_coverage_path}. Error: {e}")
              df_full = pd.DataFrame() # Ensure it's defined as empty
     else:
          print(f"ERROR: DataFrame 'df_full' not found in memory and file '{output_db_with_coverage_path}' not found. Please run Cell 2.")
          df_full = pd.DataFrame()

if not forward_pairs_map_path.is_file():
    print(f"ERROR: Forward pairs map file not found: '{forward_pairs_map_path}'. Please run Cell 14.")
    # Cannot proceed without this map
    sys.exit(1) 

if not reverse_diamond_results_path.is_file():
    print(f"ERROR: Reverse DIAMOND results file not found: '{reverse_diamond_results_path}'. Please ensure the external DIAMOND search completed.")
    # Cannot proceed without reverse hits
    sys.exit(1) 

# --- Step 1: Load Forward Hit Pairs ---
print(f"\n--- Loading Forward Asgard -> Euk Hit Pairs ---")
try:
    df_forward_pairs = pd.read_csv(forward_pairs_map_path, sep='\t')
    # Ensure columns are string type for matching
    df_forward_pairs[protein_id_col] = df_forward_pairs[protein_id_col].astype(str)
    df_forward_pairs[euk_hit_sseqid_col] = df_forward_pairs[euk_hit_sseqid_col].astype(str)
    # Keep only the first mapping if an Asgard protein hit multiple Euks in the HQ set (unlikely with best hit logic)
    df_forward_pairs = df_forward_pairs.drop_duplicates(subset=[protein_id_col], keep='first')
    print(f"Loaded {len(df_forward_pairs)} unique Asgard -> Euk pairs to check for RBH.")
except Exception as e:
    print(f"ERROR loading forward pairs map: {e}")
    sys.exit(1)

# --- Step 2: Parse Reverse DIAMOND Hits ---
print(f"\n--- Parsing Reverse Euk -> Asgard DIAMOND Hits ---")
best_reverse_hits = {} # Euk_ID -> Best_Asgard_Hit_ID

try:
    # Read the raw reverse DIAMOND output
    df_reverse_raw = pd.read_csv(reverse_diamond_results_path, sep='\t', header=None, names=reverse_diamond_col_names)
    logging.info(f"  Read {len(df_reverse_raw)} total reverse DIAMOND alignments.")

    # Filter by e-value
    df_reverse_filtered = df_reverse_raw[df_reverse_raw['evalue'] <= reverse_e_value_threshold].copy()
    logging.info(f"  Found {len(df_reverse_filtered)} reverse alignments passing e-value threshold <= {reverse_e_value_threshold}.")

    if not df_reverse_filtered.empty:
        # Ensure IDs are strings
        df_reverse_filtered['qseqid'] = df_reverse_filtered['qseqid'].astype(str)
        df_reverse_filtered['sseqid'] = df_reverse_filtered['sseqid'].astype(str)
        
        # Sort by Euk query ID (qseqid), then by e-value/bitscore to get the best Asgard hit (sseqid)
        df_reverse_filtered.sort_values(by=['qseqid', 'evalue', 'bitscore'], ascending=[True, True, False], inplace=True)
        
        # Keep only the best Asgard hit per Eukaryotic query
        df_best_reverse_hits = df_reverse_filtered.drop_duplicates(subset=['qseqid'], keep='first')
        logging.info(f"  Identified {len(df_best_reverse_hits)} unique Eukaryotic queries with a best Asgard hit.")

        # Create the reverse mapping dictionary
        best_reverse_hits = pd.Series(
            df_best_reverse_hits['sseqid'].values, 
            index=df_best_reverse_hits['qseqid']
        ).to_dict()
        
        # Diagnostic: Print a few reverse mappings
        print("  Example Reverse Best Hits (Euk ID -> Best Asgard Hit ID):")
        count = 0
        for k, v in best_reverse_hits.items():
            print(f"    '{k}' -> '{v}'")
            count += 1
            if count >= 5: break
            
    else:
        print("  No reverse DIAMOND hits passed the e-value threshold.")

except FileNotFoundError:
    print(f"ERROR: Reverse DIAMOND results file not found at '{reverse_diamond_results_path}'.")
except Exception as e:
    print(f"An error occurred while parsing reverse DIAMOND results: {e}")
    # Continue without reverse hits if parsing fails, RBH will be False

# --- Step 3: Identify RBH Pairs ---
print("\n--- Identifying Reciprocal Best Hit Pairs ---")
rbh_pairs_list = []
rbh_count = 0

if not df_forward_pairs.empty and best_reverse_hits:
    for _, row in df_forward_pairs.iterrows():
        asgard_id = row[protein_id_col]
        euk_id = row[euk_hit_sseqid_col]
        
        # Check if the Euk ID has a best reverse hit recorded
        best_reverse_asgard_hit = best_reverse_hits.get(euk_id)
        
        # Check if the best reverse hit matches the original Asgard ID
        if best_reverse_asgard_hit == asgard_id:
            rbh_pairs_list.append({protein_id_col: asgard_id, euk_hit_sseqid_col: euk_id})
            rbh_count += 1
            
    print(f"Identified {rbh_count} Reciprocal Best Hit (RBH) pairs.")
    
    # Create DataFrame of confirmed RBHs
    if rbh_pairs_list:
        df_rbh_confirmed = pd.DataFrame(rbh_pairs_list)
        # Save the confirmed RBH pairs
        try:
            df_rbh_confirmed.to_csv(output_rbh_confirmed_pairs_path, sep='\t', index=False)
            print(f"Saved {len(df_rbh_confirmed)} confirmed RBH pairs to '{output_rbh_confirmed_pairs_path}'")
        except Exception as e:
            print(f"ERROR saving confirmed RBH pairs: {e}")
    else:
        df_rbh_confirmed = pd.DataFrame(columns=[protein_id_col, euk_hit_sseqid_col]) # Empty df
        
else:
    print("Skipping RBH identification: Missing forward pairs or reverse hits.")
    df_rbh_confirmed = pd.DataFrame(columns=[protein_id_col, euk_hit_sseqid_col]) # Empty df

# --- Step 4: Add RBH Flag to Main DataFrame ---
if not df_full.empty:
    print(f"\n--- Adding '{rbh_flag_col}' column to main DataFrame ---")
    
    # Create a set of Asgard IDs that are part of an RBH pair for quick lookup
    rbh_asgard_ids = set(df_rbh_confirmed[protein_id_col])
    
    # Add the boolean column
    if protein_id_col in df_full.columns:
        df_full[rbh_flag_col] = df_full[protein_id_col].isin(rbh_asgard_ids)
        print(f"Flagged {df_full[rbh_flag_col].sum()} proteins as RBH.")
    else:
        print(f"ERROR: Cannot add RBH flag, '{protein_id_col}' not in df_full.")
        # Add empty column if needed for consistency
        if rbh_flag_col not in df_full.columns: df_full[rbh_flag_col] = False

else:
    print("Skipping addition of RBH flag as df_full is not available.")


# --- Step 5: Save Final Updated Database ---
if not df_full.empty and rbh_flag_col in df_full.columns:
    # This df_full now contains original v2.0 info + coverage + RBH flag
    final_output_db_path = output_db_with_rbh_path # Use path defined earlier
    print(f"\n--- Saving Final Updated Database (with RBH flag) ---")
    print(f"Saving to: {final_output_db_path}")
    try:
        # Ensure output directory exists
        final_output_db_path.parent.mkdir(parents=True, exist_ok=True)
        df_full.to_csv(final_output_db_path, index=False)
        logging.info(f"Successfully saved final updated database to '{final_output_db_path}'.")
    except Exception as e:
        logging.error(f"Failed to save final updated database: {e}")
else:
     print("\nSkipping final save: df_full empty or RBH flag not added.")


print("\n\n--- Cell 15 (RBH Analysis) Complete ---")
print(f"Identified {rbh_count} RBH pairs.")
print(f"The main DataFrame 'df_full' now includes the '{rbh_flag_col}' column.")
print(f"The final database is saved as '{output_db_with_rbh_path}'.")
print("Next steps involve analyzing the RBH pairs (e.g., their functions, domains) and potentially performing MSA/Phylogeny on them.")



In [ ]:
# Cell 16: Prepare Input FASTA Files for Intra-Orthogroup MAFFT MSA (Size >= 5)

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time
import logging # Use logger defined in Cell 1
from pathlib import Path 
import sys 
import shutil # For copying files
# Ensure Biopython is installed: pip install biopython
try:
    from Bio import SeqIO 
    from Bio.SeqRecord import SeqRecord
    from Bio.Seq import Seq
except ImportError:
    print("ERROR: Biopython is required for this script. Please install it: pip install biopython")
    sys.exit(1)

print("\n\n--- Cell 16: Prepare Input FASTA Files for Intra-Orthogroup MAFFT MSA (Size >= 5) ---")

# --- Assumptions ---
# This cell assumes Cell 1 (Setup) has been run.
# It relies on the existence of directories containing per-OG FASTA files.
# It relies on output directories defined in Cell 1.

# --- Configuration ---
# *** UPDATED Input Directory based on user screenshot ***
og_fasta_input_dir = Path('diamond_run_all_ogs/all_ogs_standardized_fastas') 

# Output: Directory to store the selected multi-sequence FASTA files for MAFFT input
# Ensure output_summary_dir_phase1 is defined in Cell 1
if 'output_summary_dir_phase1' not in locals():
     print("ERROR: output_summary_dir_phase1 not defined. Please run Cell 1.")
     # Define a default or raise error
     output_summary_dir_phase1 = Path("./output_summary_data_hit_validation_phase1") # Default if not set
     
mafft_input_intra_og_dir = Path(output_summary_dir_phase1) / "mafft_input_intra_og"

# Minimum number of sequences required in an OG FASTA file to be included
min_og_size_for_msa = 5 

# --- Check if necessary Variables/Files exist ---
required_variables = ['output_summary_dir_phase1']
missing_data = False
for var_name in required_variables:
     if var_name not in locals():
         print(f"ERROR: Required variable '{var_name}' is missing. Please run Cell 1 first.")
         missing_data = True
if not og_fasta_input_dir.is_dir():
     print(f"ERROR: Input OG FASTA directory not found: '{og_fasta_input_dir}'")
     missing_data = True
     
if missing_data:
    print("Cannot proceed with Cell 16 due to missing directories or variables.")
    # sys.exit(1) # Or handle differently
else:
    # --- Step 1: Identify and Copy Target OG FASTA Files (Size >= 5) ---
    print(f"\n--- Identifying and Copying OG FASTA files (Size >= {min_og_size_for_msa}) ---")
    mafft_input_intra_og_dir.mkdir(parents=True, exist_ok=True) # Create output dir
    
    copied_count = 0
    skipped_count = 0
    error_count = 0
    
    # Process ALL FASTA files in the input directory
    print(f"\nProcessing FASTA files from: {og_fasta_input_dir}")
    # Use a pattern that matches both .ASG.fasta and .GV.fasta
    all_og_fasta_files = list(og_fasta_input_dir.glob('*.fasta')) 
    print(f"Found {len(all_og_fasta_files)} total OG FASTA files.")
    
    for fasta_file in all_og_fasta_files:
        try:
            # Count sequences in the file
            seq_count = 0
            with open(fasta_file, "r") as handle:
                for record in SeqIO.parse(handle, "fasta"):
                    seq_count += 1
            
            # Check size and copy if sufficient
            if seq_count >= min_og_size_for_msa:
                destination_file = mafft_input_intra_og_dir / fasta_file.name
                shutil.copy2(fasta_file, destination_file) # copy2 preserves metadata
                copied_count += 1
                if copied_count % 500 == 0:
                     logging.info(f"  Copied {copied_count} qualifying OG FASTA files...")
            else:
                skipped_count += 1
                logging.debug(f"Skipping {fasta_file.name}: Size ({seq_count}) < {min_og_size_for_msa}")
                
        except Exception as e:
            logging.error(f"Error processing file {fasta_file.name}: {e}")
            error_count += 1

    print(f"\nFinished selecting and copying OG FASTA files.")
    print(f"  Total FASTA files copied (Size >= {min_og_size_for_msa}): {copied_count}")
    print(f"  Total FASTA files skipped (Size < {min_og_size_for_msa}): {skipped_count}")
    print(f"  Errors during processing: {error_count}")
    print(f"  Input files for MAFFT are located in: '{mafft_input_intra_og_dir}'")

    # --- Step 2: Provide Instructions for MAFFT Script ---
    print("\n--- Instructions for Running MAFFT on Intra-OG files ---")
    print("You can now use your 'run_mafft_parallel.py' script on the generated OG files.")
    print("Example command (adjust paths and parameters as needed):")
    print("\npython run_mafft_parallel.py \\")
    print(f"    --input_dir {mafft_input_intra_og_dir} \\")
    print(f"    --output_dir {mafft_input_intra_og_dir}_aligned \\") 
    print(f"    --log_dir {mafft_input_intra_og_dir}_logs \\")      
    print(f"    --input_suffix .fasta \\") # Assuming input files end with .fasta              
    print(f"    --output_suffix .aln \\")                 
    print(f"    --mafft_exe mafft \\")                   
    print(f"    --mafft_args '--auto --thread 1' \\") # Use --auto or more specific MAFFT strategy
    print(f"    --num_cores 8") # Adjust core count                         
    print("\nEnsure the output and log directories are created before running.")
    print("After running MAFFT, proceed to the TrimAl step.")

print("\n\n--- Cell 16 (Prepare Input for Intra-OG MAFFT MSA) Complete ---")


In [ ]:
# Cell 17: Calculate Intra-Orthogroup Average Pairwise Sequence Identity (APSI)

# --- Standard Library Imports ---
import os
import glob
import time
import logging # Use logger defined in Cell 1
from pathlib import Path
import sys

# Ensure Biopython is installed and imported
try:
    from Bio import AlignIO
    from Bio.Align import MultipleSeqAlignment
except ImportError:
    logger.error("ERROR: Biopython is required for this script. Please install it: pip install biopython")
    sys.exit(1)

print("\n\n--- Cell 17: Calculate Intra-Orthogroup Average Pairwise Sequence Identity (APSI) ---")

# --- Assumptions ---
# This cell assumes Cell 1 (Setup) has been run successfully to define variables like output_summary_dir_phase1.
# It assumes the TrimAl step was run successfully, producing trimmed FASTA files.

# --- Configuration ---
# Input: Directory containing the trimmed alignment files (FASTA format)
# This should be the output directory from your TrimAl script
# Make sure this path matches where your trimmed files were saved!
# Example: If your TrimAl script output to 'trimal_output_intra_og_gappyout'
# trimmed_alignments_dir = Path('./trimal_output_intra_og_gappyout') # <-- Update this path
# Assuming the trimmed files end with '.trimmed.fasta' as per the refined script suggestion
trimmed_alignments_dir = Path('./intra_og_analysis/trimal_output_intra_og') # Assuming default output dir from refined script
trimmed_file_suffix = '*_trimmed.fasta' # Corrected: Removed selection tags. This pattern matches files ending with '_trimmed.fasta'

# Output: File to save the calculated APSI values
# Ensure output_summary_dir_phase1 is defined and is a Path object
if 'output_summary_dir_phase1' not in locals():
     logger.error("ERROR: output_summary_dir_phase1 not defined. Please run Cell 1.")
     # Define a default or exit
     output_summary_dir_phase1 = Path("./output_summary_data_hit_validation_phase1") # Default if not set
     # Ensure default directory exists
     output_summary_dir_phase1.mkdir(parents=True, exist_ok=True)
elif not isinstance(output_summary_dir_phase1, Path):
    # Defensive check: if the variable exists but isn't a Path, try to convert it
    logger.warning(f"output_summary_dir_phase1 is not a Path object (it's {type(output_summary_dir_phase1)}). Attempting to convert.")
    try:
        output_summary_dir_phase1 = Path(str(output_summary_dir_phase1))
        output_summary_dir_phase1.mkdir(parents=True, exist_ok=True) # Ensure directory exists after conversion
    except Exception as e:
        logger.error(f"Failed to convert output_summary_dir_phase1 to Path: {e}. Please ensure Cell 1 runs correctly.")
        sys.exit(1) # Exit if conversion fails

# Now use the Path object for joining
apsi_output_file = output_summary_dir_phase1 / "intra_og_apsi_values.csv"

# --- Helper Function to Calculate Pairwise Identity ---
def calculate_pairwise_identity(seq1, seq2):
    """ Calculates identity between two sequences, ignoring positions with gaps in either sequence. """
    if len(seq1) != len(seq2):
        logger.warning("Sequence lengths do not match for pairwise identity calculation.")
        return 0.0

    identical_residues = 0
    aligned_length = 0 # Count positions where neither sequence has a gap

    for i in range(len(seq1)):
        res1 = seq1[i]
        res2 = seq2[i]

        if res1 != '-' and res2 != '-':
            aligned_length += 1
            if res1 == res2:
                identical_residues += 1

    # Avoid division by zero if no aligned positions
    return (identical_residues / aligned_length) if aligned_length > 0 else 0.0

# --- Main APSI Calculation Logic ---
if not trimmed_alignments_dir.is_dir():
    logger.error(f"Input trimmed alignment directory not found: '{trimmed_alignments_dir}'")
    print("Cannot calculate APSI. Please check the 'trimmed_alignments_dir' path.")
else:
    print(f"Reading trimmed alignments from: {trimmed_alignments_dir}")
    print(f"Saving APSI results to: {apsi_output_file}")

    # Use glob with the corrected suffix pattern
    all_trimmed_files = list(trimmed_alignments_dir.glob(f"{trimmed_file_suffix}"))

    if not all_trimmed_files:
        logger.warning(f"No trimmed alignment files found matching pattern '{trimmed_file_suffix}' in '{trimmed_alignments_dir}'.")
        print("No alignments to calculate APSI for.")
    else:
        print(f"Found {len(all_trimmed_files)} trimmed alignment files.")
        apsi_results = []
        processed_count = 0
        start_time = time.time()

        for alignment_file in all_trimmed_files:
            og_id = alignment_file.stem # Get filename without suffix
            # Adjust og_id extraction based on actual filename format if needed
            # For example, if files are OG0000000_trimmed.fasta, stem is 'OG0000000_trimmed'
            # We want 'OG0000000'
            if og_id.endswith('_trimmed'):
                 og_id = og_id[:-len('_trimmed')]


            try:
                # Read the alignment
                alignment = AlignIO.read(alignment_file, "fasta")
                sequences = [str(record.seq) for record in alignment]
                num_sequences = len(sequences)

                if num_sequences < 2:
                    # Should not happen if input was filtered for size >= 5, but handle defensively
                    logger.warning(f"Alignment for {og_id} has less than 2 sequences ({num_sequences}). Cannot calculate APSI.")
                    apsi_results.append({'Orthogroup': og_id, 'APSI': None, 'Num_Sequences': num_sequences})
                    processed_count += 1
                    continue

                # Calculate pairwise identities for all pairs
                pairwise_identities = []
                for i in range(num_sequences):
                    for j in range(i + 1, num_sequences):
                        identity = calculate_pairwise_identity(sequences[i], sequences[j])
                        pairwise_identities.append(identity)

                # Calculate Average Pairwise Sequence Identity
                average_identity = sum(pairwise_identities) / len(pairwise_identities) if pairwise_identities else 0.0

                apsi_results.append({
                    'Orthogroup': og_id,
                    'APSI': average_identity,
                    'Num_Sequences': num_sequences
                })

                processed_count += 1
                if processed_count % 100 == 0:
                    logger.info(f"Processed {processed_count}/{len(all_trimmed_files)} alignments for APSI...")

            except Exception as e:
                logger.error(f"Error processing alignment file {alignment_file}: {e}")
                apsi_results.append({'Orthogroup': og_id, 'APSI': None, 'Num_Sequences': None, 'Error': str(e)})
                processed_count += 1


        # Save the results to a DataFrame and then to CSV
        if apsi_results:
            df_apsi = pd.DataFrame(apsi_results)
            try:
                df_apsi.to_csv(apsi_output_file, index=False)
                print(f"\nSuccessfully calculated and saved APSI for {len(df_apsi)} orthogroups to '{apsi_output_file}'.")
            except Exception as e:
                logger.error(f"Failed to save APSI results to '{apsi_output_file}': {e}")
        else:
            print("\nNo APSI results were generated.")

        end_time = time.time()
        print(f"APSI calculation completed in {end_time - start_time:.2f} seconds.")


print("\n\n--- Cell 17 (APSI Calculation) Complete ---")
print("The APSI values are saved and ready to be merged into your main database.")


In [ ]:
# Cell 18: Identify Conserved Motifs in Trimmed Alignments

# --- Standard Library Imports ---
import os
import glob
import time
import logging # Use logger defined in Cell 1
from pathlib import Path
import sys
from collections import Counter

# Ensure Biopython is installed and imported
try:
    from Bio import AlignIO
    from Bio.Align import MultipleSeqAlignment
except ImportError:
    logger.error("ERROR: Biopython is required for this script. Please install it: pip install biopython")
    sys.exit(1)

print("\n\n--- Cell 18: Identify Conserved Motifs in Trimmed Alignments ---")

# --- Assumptions ---
# This cell assumes Cell 1 (Setup) has been run.
# It assumes the TrimAl step was run successfully, producing trimmed FASTA files.
# It relies on the output directory variable (e.g., output_summary_dir_phase1) defined in Cell 1.

# --- Configuration ---
# Input: Directory containing the trimmed alignment files (FASTA format)
# This should be the output directory from your TrimAl script
# Make sure this path matches where your trimmed files were saved!
# Example: If your TrimAl script output to 'trimal_output_intra_og_gappyout'
# trimmed_alignments_dir = Path('./trimal_output_intra_og_gappyout') # <-- Update this path
# Assuming the trimmed files end with '.trimmed.fasta' as per the refined script suggestion
trimmed_alignments_dir = Path('intra_og_analysis/trimal_output_intra_og') # Assuming default output dir from refined script
trimmed_file_suffix = '_trimmed.fasta' # Assuming output suffix from refined script

# Output: File to save the identified conserved motifs
if 'output_summary_dir_phase1' not in locals():
     logger.error("ERROR: output_summary_dir_phase1 not defined. Please run Cell 1.")
     # Define a default or exit
     output_summary_dir_phase1 = Path("./output_summary_data_hit_validation_phase1") # Default if not set
     output_summary_dir_phase1.mkdir(parents=True, exist_ok=True) # Ensure it exists

motifs_output_file = output_summary_dir_phase1 / "intra_og_conserved_motifs.csv"

# Motif finding parameters
min_motif_length = 4 # Minimum length of a conserved motif
min_conservation_percentage = 90 # Minimum percentage of identical residues in a column for it to be considered conserved (e.g., 90 for 90%)

# --- Helper Function to Check Column Conservation ---
def is_column_conserved(column, min_conservation_percentage):
    """ Checks if a column in an alignment is conserved above a threshold, ignoring gaps. """
    # Remove gaps from the column
    nongap_residues = [res for res in column if res != '-']
    num_nongap = len(nongap_residues)

    if num_nongap == 0:
        return False, None # Cannot determine conservation if only gaps

    # Count frequency of each residue
    residue_counts = Counter(nongap_residues)
    most_common_residue, count = residue_counts.most_common(1)[0]

    # Calculate conservation percentage
    conservation = (count / num_nongap) * 100

    return conservation >= min_conservation_percentage, most_common_residue

# --- Main Motif Identification Logic ---
if not trimmed_alignments_dir.is_dir():
    logger.error(f"Input trimmed alignment directory not found: '{trimmed_alignments_dir}'")
    print("Cannot identify conserved motifs. Please check the 'trimmed_alignments_dir' path.")
else:
    print(f"Reading trimmed alignments from: {trimmed_alignments_dir}")
    print(f"Saving motif results to: {motifs_output_file}")
    print(f"Motif parameters: Min Length = {min_motif_length}, Min Conservation = {min_conservation_percentage}%")

    all_trimmed_files = list(trimmed_alignments_dir.glob(f"*{trimmed_file_suffix}"))

    if not all_trimmed_files:
        logger.warning(f"No trimmed alignment files found with suffix '{trimmed_file_suffix}' in '{trimmed_alignments_dir}'.")
        print("No alignments to identify motifs in.")
    else:
        print(f"Found {len(all_trimmed_files)} trimmed alignment files.")
        conserved_motifs_results = []
        processed_count = 0
        start_time = time.time()

        for alignment_file in all_trimmed_files:
            og_id = alignment_file.stem # Get filename without suffix
            if og_id.endswith('_trimmed'): # Remove the '_trimmed' part added by TrimAl script
                 og_id = og_id[:-len('_trimmed')]

            try:
                # Read the alignment
                alignment = AlignIO.read(alignment_file, "fasta")
                alignment_length = alignment.get_alignment_length()
                num_sequences = len(alignment)

                if num_sequences == 0 or alignment_length == 0:
                     logger.warning(f"Alignment for {og_id} is empty or has zero length. Skipping motif search.")
                     processed_count += 1
                     continue

                # Iterate through columns to find conserved regions
                current_motif = ""
                motif_start_col = -1

                for i in range(alignment_length):
                    column = alignment[:, i] # Get the i-th column
                    is_cons, conserved_residue = is_column_conserved(column, min_conservation_percentage)

                    if is_cons:
                        if motif_start_col == -1:
                            motif_start_col = i # Start of a potential motif
                        current_motif += conserved_residue # Append the conserved residue
                    else:
                        # End of a conserved stretch
                        if motif_start_col != -1:
                            # Check if the collected motif meets the minimum length
                            if len(current_motif) >= min_motif_length:
                                conserved_motifs_results.append({
                                    'Orthogroup': og_id,
                                    'Motif': current_motif,
                                    'Start_Column': motif_start_col,
                                    'End_Column': i - 1, # End column is the one *before* the break
                                    'Alignment_Length': alignment_length,
                                    'Num_Sequences': num_sequences
                                })
                            # Reset for the next potential motif
                            current_motif = ""
                            motif_start_col = -1

                # After the loop, check if the last stretch is a motif
                if motif_start_col != -1 and len(current_motif) >= min_motif_length:
                     conserved_motifs_results.append({
                        'Orthogroup': og_id,
                        'Motif': current_motif,
                        'Start_Column': motif_start_col,
                        'End_Column': alignment_length - 1, # Last column
                        'Alignment_Length': alignment_length,
                        'Num_Sequences': num_sequences
                    })


                processed_count += 1
                if processed_count % 100 == 0:
                    logger.info(f"Processed {processed_count}/{len(all_trimmed_files)} alignments for motifs...")

            except Exception as e:
                logger.error(f"Error processing alignment file {alignment_file} for motifs: {e}")
                # Optionally log the OG ID with the error
                processed_count += 1


        # Save the results to a DataFrame and then to CSV
        if conserved_motifs_results:
            df_motifs = pd.DataFrame(conserved_motifs_results)
            try:
                df_motifs.to_csv(motifs_output_file, index=False)
                print(f"\nSuccessfully identified and saved conserved motifs for {len(set(df_motifs['Orthogroup']))} orthogroups to '{motifs_output_file}'.")
                print(f"Total {len(df_motifs)} motifs identified across all orthogroups.")
            except Exception as e:
                logger.error(f"Failed to save motif results to '{motifs_output_file}': {e}")
        else:
            print("\nNo conserved motifs were identified based on the specified parameters.")

        end_time = time.time()
        print(f"Motif identification completed in {end_time - start_time:.2f} seconds.")


print("\n\n--- Cell 18 (Conserved Motif Identification) Complete ---")
print("The identified conserved motifs are saved and ready for analysis and integration.")



In [ ]:
# Cell Y: Check Output Directory Path and Existence

print("\n\n--- Cell Y: Checking Output Directory ---")

# This assumes 'output_summary_dir_phase1' is defined in a previous cell (e.g., Cell 1)
# and 'Path' is imported (also in Cell 1).

if 'output_summary_dir_phase1' in locals() and 'Path' in locals():
    output_dir_path = Path(output_summary_dir_phase1)

    print(f"Configured output directory path: {output_dir_path}")

    # Check if the directory exists
    if output_dir_path.is_dir():
        print("Directory exists.")
    else:
        print("Directory DOES NOT exist. Attempting to create it...")
        try:
            output_dir_path.mkdir(parents=True, exist_ok=True)
            print("Directory created successfully.")
        except Exception as e:
            print(f"ERROR: Failed to create directory. Please check permissions and the path: {e}")
            output_dir_path = None # Set to None to indicate failure

    # Check write permissions (basic check by trying to touch a dummy file)
    if output_dir_path and output_dir_path.is_dir():
        dummy_file = output_dir_path / ".test_write_permission"
        try:
            dummy_file.touch()
            dummy_file.unlink() # Clean up the dummy file
            print("Write permissions seem OK.")
        except Exception as e:
            print(f"WARNING: Could not write to directory. Saving might fail. Error: {e}")

else:
    print("ERROR: Required variables 'output_summary_dir_phase1' or 'Path' not found. Please run Cell 1.")

print("\n--- Cell Y (Check Output Directory) Complete ---\n")

In [ ]:
# Cell X: Apply Revised Broad Functional Categorization and Replace Original Column

print("\n\n--- Cell X: Applying Revised Broad Functional Categorization and Replacing Original Column ---")

# --- Assumptions ---
# This cell assumes 'df_full' DataFrame is loaded and available.
# It will replace the original 'Broad_Functional_Category' column with the revised one.
# It relies on the 'broad_func_cat_col' variable defined in Cell 1.

# --- Define the mapping from current to revised categories ---
category_mapping = {
    # Genome Information Processing
    "DNA Info Processing": "DNA Processing & Maintenance",
    "DNA Replication, Repair, Recombination": "DNA Processing & Maintenance",
    "RNA Info Processing": "RNA Processing & Transcription",
    "Transcription and RNA Processing": "RNA Processing & Transcription",

    # Protein Synthesis & Modification
    "Translation": "Translation & Protein Synthesis",
    "Translation and Protein Synthesis": "Translation & Protein Synthesis",
    # N-glycosylation is kept separate

    # Trafficking & ESCRT
    "Membrane Trafficking/Vesicles": "Membrane Trafficking, Vesicles & ESCRT",
    "ESCRT/Endosomal Sorting": "Membrane Trafficking, Vesicles & ESCRT",

    # General Features (Catch-all)
    "Homologous_superfamily": "General Protein Features",
    "Binding_site": "General Protein Features",
    "Active_site": "General Protein Features",
}

# --- Apply the mapping and replace the original column ---
print(f"\nApplying revised categorization and replacing the '{broad_func_cat_col}' column...")

if broad_func_cat_col in df_full.columns:
    # Create a temporary new column with the revised categories
    temp_revised_col_name = 'Temp_Revised_Category'
    df_full[temp_revised_col_name] = df_full[broad_func_cat_col].apply(
        lambda x: category_mapping.get(x, x) # Get the new category, or keep the old one if not in map
    )
    print(f"Created temporary column '{temp_revised_col_name}'.")

    # Drop the original column
    df_full = df_full.drop(columns=[broad_func_cat_col])
    print(f"Dropped original column '{broad_func_cat_col}'.")

    # Rename the temporary column to the original column name
    df_full = df_full.rename(columns={temp_revised_col_name: broad_func_cat_col})
    print(f"Renamed '{temp_revised_col_name}' to '{broad_func_cat_col}'.")

    # --- Verification ---
    print(f"\nVerification: Value counts for the updated '{broad_func_cat_col}' column:")
    # Ensure the column exists and display counts
    if broad_func_cat_col in df_full.columns:
        print(df_full[broad_func_cat_col].value_counts(dropna=False).to_markdown())
    else:
        print(f"Error: Column '{broad_func_cat_col}' not found after renaming.")


    # Optional: Display a few rows to show the change
    print("\nSample rows showing the updated functional category:")
    sample_cols = [broad_func_cat_col] # Only display the one column now
    if broad_func_cat_col in df_full.columns:
        try:
             print(df_full[sample_cols].head(10).to_markdown(index=False))
        except ImportError:
             print(df_full[sample_cols].head(10))
    else:
        print("Could not display sample rows: Required column not found.")


else:
    print(f"ERROR: Original broad functional category column '{broad_func_cat_col}' not found in df_full.")
    print("       Cannot apply the revised categorization.")


print(f"\n\n--- Cell X (Apply Revised Categorization and Replace) Complete ---")
print(f"The '{broad_func_cat_col}' column has been updated with the revised categories.")

# --- Optional: Save the updated DataFrame ---
# Decide if you want to overwrite the v2.1 file or save as v2.2
output_db_revised_cat_path = Path(output_summary_dir_phase1) / 'proteome_database_combined_v2.2_revised_cat.csv'

print(f"\nAttempting to save updated database with revised categories to: {output_db_revised_cat_path}") # <--- ADD THIS PRINT LINE

try:
    output_db_revised_cat_path.parent.mkdir(parents=True, exist_ok=True)
    df_full.to_csv(output_db_revised_cat_path, index=False)
    print(f"Successfully saved updated database to '{output_db_revised_cat_path}'.")
except Exception as e:
    print(f"Failed to save updated database: {e}")

In [ ]:
# Cell 20: Clean and Reorder DataFrame Columns

print("\n\n--- Cell 20: Cleaning and Reordering DataFrame Columns ---")

# --- Assumptions ---
# This cell assumes 'df_full' DataFrame is loaded and available
# with the revised Broad_Functional_Category already applied and named 'Broad_Functional_Category'.

# --- Define the desired order of columns ---
# This list includes the columns you want to keep, in your specified order.
# It excludes 'Specific_Functional_Category', 'Category_Trigger', and
# the duplicate 'Broad_Functional_Category' that was at the end.
desired_column_order = [
    'ProteinID',
    'Sequence',
    'Length',
    'Source_Dataset',
    'Source_Genome_Assembly_Accession',
    'Source_Protein_Annotation',
    'NCBI_TaxID',
    'Asgard_Phylum',
    'Virus_Family',
    'Virus_Name',
    'Orthogroup',
    'IPR_Signatures',
    'IPR_GO_Terms',
    'UniProtKB_AC',
    'Num_Domains',
    'Domain_Architecture',
    'Type',
    'Is_Hypothetical',
    'Has_Known_Structure',
    'Percent_Disorder',
    'Signal_Peptide_USPNet',
    'SP_Cleavage_Site_USPNet',
    'Original_Seq_Length',
    'Group',
    'SeqSearch_PDB_Hit',
    'SeqSearch_AFDB_Hit',
    'Has_Reference_Structure',
    'Predicted_Subcellular_Localization',
    'Mature_Protein_Sequence',
    'Mature_Seq_Length',
    'SeqSearch_MGnify_Hit',
    'SeqSearch_ESMA_Hit',
    'Is_Structurally_Dark',
    'Is_ESP',
    'Has_Euk_DIAMOND_Hit',
    'Euk_Hit_SSEQID',
    'Euk_Hit_Organism',
    'Euk_Hit_PIDENT',
    'Euk_Hit_EVALUE',
    'Euk_Hit_Protein_Name',
    'Euk_Hit_Qstart',
    'Euk_Hit_Qend',
    'Euk_Hit_Sstart',
    'Euk_Hit_Send',
    'Euk_Hit_Slen_Diamond',
    'Query_Coverage',
    'Subject_Coverage',
    'Is_RBH',
    'Broad_Functional_Category' # This column now contains the revised categories
    # Exclude the duplicate 'Broad_Functional_Category' here (if it existed after loading)
]

# --- Check for columns that are supposed to be in the desired list but are missing from df_full ---
# This helps identify potential issues before reindexing
current_cols = df_full.columns.tolist()
missing_desired_cols = [col for col in desired_column_order if col not in current_cols]

if missing_desired_cols:
    print(f"WARNING: The following columns are in the desired order list but not found in the DataFrame: {missing_desired_cols}")
    print("These columns will not be included in the cleaned DataFrame.")
    # Optionally, filter desired_column_order to only include present columns
    # desired_column_order = [col for col in desired_column_order if col in current_cols]


# --- Check for columns in df_full that are NOT in the desired list (these will be dropped) ---
cols_to_drop = [col for col in current_cols if col not in desired_column_order]
print(f"Columns that will be dropped: {cols_to_drop}")

# --- Reindex the DataFrame to select and reorder columns ---
try:
    df_full = df_full[desired_column_order]
    print(f"\nDataFrame cleaned and reordered successfully. New shape: {df_full.shape}")
    print("New columns order:")
    print(df_full.columns.tolist())

except KeyError as e:
    print(f"ERROR: Failed to reindex DataFrame. A specified column was not found after the previous steps: {e}")
    print("Please check the column names in the 'desired_column_order' list.")
except Exception as e:
    print(f"An unexpected error occurred during reindexing: {e}")


# --- Verification ---
print("\nVerification: First 5 rows of the cleaned DataFrame:")
try:
    print(df_full.head().to_markdown())
except ImportError:
    print(df_full.head())

print("\n\n--- Cell 20 (Clean and Reorder Columns) Complete ---")

# --- Optional: Save the cleaned DataFrame ---
# Choose a new filename to indicate this level of cleaning/revision
output_db_cleaned_path = Path(output_summary_dir_phase1) / 'proteome_database_combined_v2.3_cleaned.csv'
print(f"\nSaving cleaned database to: {output_db_cleaned_path}")
try:
     output_db_cleaned_path.parent.mkdir(parents=True, exist_ok=True)
     df_full.to_csv(output_db_cleaned_path, index=False)
     print(f"Successfully saved cleaned database to '{output_db_cleaned_path}'.")
except Exception as e:
     print(f"Failed to save cleaned database: {e}")

In [ ]:
# Cell 1: Setup, Data Loading for Deaminase Exploration

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time
import re
from collections import Counter
from pathlib import Path 

# --- Plotly Imports ---
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

# --- Arcadia Style Imports (Attempt) ---
try:
    import arcadia_pycolor as apc
    arcadia_primary_palette = apc.primary_standard_rgb_strings
    arcadia_secondary_palette = apc.secondary_standard_rgb_strings
    arcadia_neutrals_palette = apc.neutrals_standard_rgb_strings
    arcadia_paper_color = apc.paper_rgb_string
    arcadia_font_family = "Arial, sans-serif" 
    arcadia_font_color = apc.default_text_rgb_string
    arcadia_axis_color = apc.default_text_rgb_string 
    arcadia_grid_color = apc.gray_medium_rgb_string 
    ARCADIA_STYLE_AVAILABLE = True
    print("Successfully imported arcadia_pycolor. Arcadia style will be applied.")
except (ImportError, AttributeError) as e: 
    ARCADIA_STYLE_AVAILABLE = False
    print(f"Warning: arcadia_pycolor import or attribute access failed ({e}). Using manual Arcadia color palettes and Plotly defaults.")
    # Manual Fallback Arcadia Color Palettes
    arcadia_colors_manual = {
        "aegean": "#5088C5", "amber": "#F28360", "seaweed": "#3B9886", "canary": "#F7B846",
        "aster": "#7A77AB", "rose": "#F898AE", "vital": "#73B5E3", "tangerine": "#FFB984",
        "oat": "#F5E4BE", "wish": "#BABEE0", "lime": "#97CD78", "dragon": "#C85152",
        "sky": "#C6E7F4", "dress": "#F8C5C1", "taupe": "#DBD1C3", "denim": "#B6C8D4",
        "sage": "#B5BEA4", "mars": "#DA9085", "marine": "#8A99AD", "shell": "#EDE0D6",
        "white": "#FFFFFF", "gray": "#EBEDE8", "chateau": "#BAB0A8", "bark": "#8F8885",
        "slate": "#43413F", "charcoal": "#484B50", "crow": "#292928", "black": "#09090A",
        "forest": "#596F74", "parchment": "#FEF7F1", "zephyr": "#F4FBFF",
        "lichen": "#F7FBEF", "dawn": "#F8F4F1"
    }
    arcadia_primary_palette = [
        arcadia_colors_manual["aegean"], arcadia_colors_manual["amber"], arcadia_colors_manual["seaweed"],
        arcadia_colors_manual["canary"], arcadia_colors_manual["aster"], arcadia_colors_manual["rose"],
        arcadia_colors_manual["vital"], arcadia_colors_manual["tangerine"], arcadia_colors_manual["oat"],
        arcadia_colors_manual["wish"], arcadia_colors_manual["lime"], arcadia_colors_manual["dragon"]
    ]
    arcadia_secondary_palette = [
        arcadia_colors_manual["sky"], arcadia_colors_manual["dress"], arcadia_colors_manual["taupe"],
        arcadia_colors_manual["denim"], arcadia_colors_manual["sage"], arcadia_colors_manual["mars"],
        arcadia_colors_manual["marine"], arcadia_colors_manual["shell"]
    ]
    arcadia_neutrals_palette = [
        arcadia_colors_manual["gray"], arcadia_colors_manual["chateau"], arcadia_colors_manual["bark"],
        arcadia_colors_manual["slate"], arcadia_colors_manual["charcoal"], arcadia_colors_manual["forest"],
        arcadia_colors_manual["crow"]
    ]
    arcadia_paper_color = arcadia_colors_manual["white"]
    arcadia_font_family = "Arial, sans-serif" 
    arcadia_font_color = arcadia_colors_manual["slate"] 
    arcadia_axis_color = arcadia_colors_manual["slate"] 
    arcadia_grid_color = arcadia_colors_manual["gray"]   

print("\n--- Database Pub Analysis Notebook Setup ---")

# --- Configuration ---
# !!! USER: Ensure this path points to your latest, most complete database CSV !!!
DB_PATH = 'proteome_database_v2.5.csv' 

# Define output directories for this notebook's specific analyses
# These might differ from the GV_Analysis notebook
DEAMINASE_PLOT_OUTPUT_DIR = Path('output_plots_deaminase_analysis') 
DEAMINASE_SUMMARY_DATA_DIR = Path('output_summary_data_deaminase_analysis')

DEAMINASE_PLOT_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DEAMINASE_SUMMARY_DATA_DIR.mkdir(parents=True, exist_ok=True)
print(f"Plot output directory for deaminase analysis: {DEAMINASE_PLOT_OUTPUT_DIR.resolve()}")
print(f"Summary data directory for deaminase analysis: {DEAMINASE_SUMMARY_DATA_DIR.resolve()}")

INTERPRO_ENTRY_PATH = 'interpro_entry.txt' # Path to your InterPro names file

# --- Define Common Column Names (ensure these match your CSV) ---
protein_id_col = 'ProteinID'
og_col = 'Orthogroup'
group_col = 'Group' # 'Asgard' or 'GV'
source_dataset_col = 'Source_Dataset' # Asgard / GV
virus_family_col = 'Virus_Family' # For GVs
virus_name_col = 'Virus_Name' 
asgard_phylum_col = 'Asgard_Phylum' # For Asgard
broad_func_cat_col = 'Broad_Functional_Category'
specific_func_cat_col = 'Specific_Functional_Category' # If you have this
domain_arch_col = 'Domain_Architecture'
num_domains_col = 'Num_Domains'
ipr_col = 'IPR_Signatures'
source_annot_col = 'Source_Protein_Annotation' 
dark_col = 'Is_Structurally_Dark'
length_col = 'Original_Seq_Length' 
disorder_col = 'Percent_Disorder'
localization_col = 'Predicted_Subcellular_Localization'
sp_type_col = 'Signal_Peptide_USPNet'
sequence_col = 'Sequence'
is_esp_col = 'Is_ESP' # For Asgard proteins

# Eukaryotic Hit Columns
has_euk_hit_col = 'Has_Euk_DIAMOND_Hit'
euk_hit_sseqid_col = 'Euk_Hit_SSEQID'
euk_hit_organism_col = 'Euk_Hit_Organism' 
euk_hit_pident_col = 'Euk_Hit_PIDENT'
euk_hit_evalue_col = 'Euk_Hit_EVALUE'
euk_hit_protein_name_col = 'Euk_Hit_Protein_Name' 
euk_hit_qstart_col = 'Euk_Hit_Qstart'
euk_hit_qend_col = 'Euk_Hit_Qend'
euk_hit_sstart_col = 'Euk_Hit_Sstart'
euk_hit_send_col = 'Euk_Hit_Send'
euk_hit_slen_col = 'Euk_Hit_Slen_Diamond'
query_coverage_col = 'Query_Coverage'
subject_coverage_col = 'Subject_Coverage'

# Intra-OG Diversity Columns (if merged into main DB, otherwise load separately)
apsi_col = 'Intra_OG_APSI' # Example name, adjust if different
shannon_entropy_col = 'Shannon_Entropy' # Example name
observed_richness_col = 'Observed_Richness' # Example name

# List of all columns you anticipate needing for general analyses in this notebook
columns_to_load_main = list(set([ 
    protein_id_col, og_col, group_col, source_dataset_col, virus_family_col, virus_name_col, asgard_phylum_col,
    broad_func_cat_col, specific_func_cat_col, domain_arch_col, num_domains_col, ipr_col, source_annot_col,
    dark_col, length_col, disorder_col, localization_col, sp_type_col, sequence_col, is_esp_col,
    has_euk_hit_col, euk_hit_sseqid_col, euk_hit_organism_col, euk_hit_pident_col,
    euk_hit_evalue_col, euk_hit_protein_name_col, euk_hit_qstart_col, euk_hit_qend_col,
    euk_hit_sstart_col, euk_hit_send_col, euk_hit_slen_col,
    query_coverage_col, subject_coverage_col,
    apsi_col, shannon_entropy_col, observed_richness_col # Add diversity metrics here
]))
print(f"\nWill attempt to load these columns for general analysis: {columns_to_load_main}")

# --- Load Main DataFrame (df_full) ---
print(f"\n--- Loading Main Data from '{DB_PATH}' ---")
start_time = time.time()
df_full = pd.DataFrame() 

try:
    if not Path(DB_PATH).is_file():
        print(f"CRITICAL ERROR: Database file not found at '{DB_PATH}'. Cannot proceed.")
        raise FileNotFoundError(f"Database file not found: {DB_PATH}")

    header_df = pd.read_csv(DB_PATH, nrows=0)
    actual_cols_to_load = [col for col in columns_to_load_main if col in header_df.columns]
    missing_cols_in_file = [col for col in columns_to_load_main if col not in header_df.columns]
    if missing_cols_in_file:
        print(f"Warning: The following specified columns were NOT found in '{DB_PATH}': {missing_cols_in_file}")
    
    if not actual_cols_to_load:
        print(f"CRITICAL ERROR: No columns to load from '{DB_PATH}' based on 'columns_to_load_main'. Check column names.")
        raise ValueError("No columns to load.")

    df_full = pd.read_csv(DB_PATH, usecols=actual_cols_to_load, low_memory=False)
    print(f"Loaded main database. Shape: {df_full.shape}")

    print("\nPerforming initial data cleaning and type setting on loaded data...")
    # Ensure ProteinID is string
    if protein_id_col in df_full.columns:
        df_full[protein_id_col] = df_full[protein_id_col].astype(str)
    else:
        print(f"CRITICAL WARNING: '{protein_id_col}' not found in loaded df_full. This will cause issues.")

    # Set categorical columns
    cat_cols_main = [group_col, source_dataset_col, virus_family_col, asgard_phylum_col, 
                     broad_func_cat_col, specific_func_cat_col, localization_col, sp_type_col]
    cat_fill_values_main = {
        group_col: 'Unknown', source_dataset_col: 'Unknown', virus_family_col: 'Unknown', 
        asgard_phylum_col: 'Unknown', broad_func_cat_col: 'Unknown/Unclassified', 
        specific_func_cat_col: 'Unknown/Unclassified', localization_col: 'Unknown', sp_type_col: 'NO_SP'
    }
    for col in cat_cols_main:
        if col in df_full.columns:
            df_full[col] = df_full[col].fillna(cat_fill_values_main.get(col, 'Unknown')).astype('category')

    # Set string columns
    str_cols_main = [og_col, ipr_col, domain_arch_col, source_annot_col, virus_name_col, 
                     euk_hit_sseqid_col, euk_hit_organism_col, euk_hit_protein_name_col, sequence_col]
    for col in str_cols_main:
        if col in df_full.columns:
            df_full[col] = df_full[col].fillna('').astype(str)
    
    # Set boolean columns
    bool_cols_main = [dark_col, has_euk_hit_col, is_esp_col]
    for col in bool_cols_main:
        if col in df_full.columns:
            df_full[col] = df_full[col].fillna(False).astype(bool)
    
    # Set numeric columns
    num_cols_main = [
        num_domains_col, length_col, disorder_col, euk_hit_pident_col, euk_hit_evalue_col,
        euk_hit_qstart_col, euk_hit_qend_col, euk_hit_sstart_col, euk_hit_send_col,
        euk_hit_slen_col, query_coverage_col, subject_coverage_col,
        apsi_col, shannon_entropy_col, observed_richness_col # Diversity metrics
    ]
    for col in num_cols_main:
         if col in df_full.columns:
              df_full[col] = pd.to_numeric(df_full[col], errors='coerce')
    
    # Check if df_full is empty after loading/cleaning
    if df_full.empty:
        print("CRITICAL WARNING: df_full is empty after loading and initial processing. Check DB_PATH and column definitions.")

except FileNotFoundError:
    print(f"Error: Database file not found at '{DB_PATH}'. Please ensure the path is correct.")
    # df_full remains empty as initialized
except Exception as e:
    print(f"An error occurred while loading or cleaning the database CSV: {e}")
    # df_full remains empty
print(f"Database loading and initial processing finished in {time.time() - start_time:.2f} seconds.")
print(f"Shape of df_full: {df_full.shape}")


# --- Load InterPro Entry Data (for descriptions) ---
print(f"\n--- Loading InterPro Entry Data ---")
ipr_lookup = {}
start_time_ipr = time.time()
if Path(INTERPRO_ENTRY_PATH).is_file():
    try:
        ipr_info_df = pd.read_csv(
            INTERPRO_ENTRY_PATH, sep='\t', usecols=[0, 1, 2],
            names=['IPR_ID', 'Type', 'Name'], header=None, comment='#', on_bad_lines='warn'
        )
        ipr_info_df['IPR_ID'] = ipr_info_df['IPR_ID'].astype(str).str.strip()
        ipr_lookup = ipr_info_df.set_index('IPR_ID')[['Type', 'Name']].to_dict('index')
        print(f"Loaded InterPro entry data for {len(ipr_lookup)} entries in {time.time() - start_time_ipr:.2f} seconds.")
    except Exception as e:
        print(f"Warning: An error occurred loading or processing '{INTERPRO_ENTRY_PATH}': {e}")
else:
    print(f"Warning: InterPro entry file not found at '{INTERPRO_ENTRY_PATH}'. IPR name translations will be unavailable.")
if not ipr_lookup: print("Warning: ipr_lookup is empty. Domain name translations will not be available.")


# --- Configure Plotly Defaults (Arcadia Style) ---
print("\n--- Configuring Plotly Defaults (Arcadia Style) ---")
plotly_layout_defaults = go.Layout(
    font=dict(family=arcadia_font_family, size=12, color=arcadia_font_color),
    title_font=dict(family=arcadia_font_family, size=18, color=arcadia_font_color),
    paper_bgcolor=arcadia_paper_color,
    plot_bgcolor=arcadia_paper_color, 
    xaxis=dict(
        title_font=dict(size=15, family=arcadia_font_family, color=arcadia_axis_color), 
        tickfont=dict(size=12, family=arcadia_font_family, color=arcadia_axis_color),
        showgrid=False, zeroline=False,
        linecolor=arcadia_axis_color, linewidth=1.5, ticks='outside', ticklen=5, tickwidth=1.5, tickcolor=arcadia_axis_color
    ),
    yaxis=dict(
        title_font=dict(size=15, family=arcadia_font_family, color=arcadia_axis_color),
        tickfont=dict(size=12, family=arcadia_font_family, color=arcadia_axis_color),
        showgrid=False, zeroline=False,
        linecolor=arcadia_axis_color, linewidth=1.5, ticks='outside', ticklen=5, tickwidth=1.5, tickcolor=arcadia_axis_color
    ),
    legend=dict(
        font=dict(size=12, family=arcadia_font_family, color=arcadia_font_color),
        bgcolor=arcadia_paper_color,
        bordercolor=arcadia_grid_color, 
        borderwidth=1
    )
)
pio.templates["arcadia_theme_db_analysis"] = pio.templates["plotly_white"] 
pio.templates["arcadia_theme_db_analysis"].layout = plotly_layout_defaults
pio.templates.default = "arcadia_theme_db_analysis" 
print("Plotly default template set to 'arcadia_theme_db_analysis'.")

# --- Define Helper Functions ---
print("\n--- Defining Helper Functions ---")

def translate_architecture(arch_string, lookup_dict=ipr_lookup): 
    if not isinstance(arch_string, str) or not arch_string or not lookup_dict: return arch_string
    ipr_ids = arch_string.split(';')
    translated_parts = []
    for ipr_id in ipr_ids:
        ipr_id_clean = ipr_id.strip()
        if ipr_id_clean in lookup_dict:
            name = lookup_dict[ipr_id_clean].get('Name', ipr_id_clean)
            name = name[:40] + '...' if len(name) > 43 else name # Truncate long names
            translated_parts.append(f"{name} ({ipr_id_clean})")
        elif ipr_id_clean: # Handle cases where IPR ID might not be in lookup
            translated_parts.append(ipr_id_clean)
    full_translation = "; ".join(translated_parts)
    max_len = 150 
    if len(full_translation) > max_len:
         full_translation = full_translation[:max_len-3] + "..."
    return full_translation

def clean_protein_name(name_str):
    if not isinstance(name_str, str):
        return "Unknown" 
    name_str = re.sub(r'\[.*?\]', '', name_str) 
    name_str = re.sub(r'\s*\(Fragment\)$', '', name_str, flags=re.IGNORECASE)
    name_str = re.sub(r'^(PREDICTED|LOW QUALITY PROTEIN|UNCHARACTERIZED PROTEIN|HYPOTHETICAL PROTEIN):\s*', '', name_str, flags=re.IGNORECASE)
    name_str = re.sub(r',\s*partial$', '', name_str, flags=re.IGNORECASE)
    name_str = re.sub(r'\s*protein$', '', name_str, flags=re.IGNORECASE) 
    name_str = re.sub(r'\s*isoform X\d*$', '', name_str, flags=re.IGNORECASE) 
    name_str = re.sub(r'\s*isoform \w+$', '', name_str, flags=re.IGNORECASE) 
    name_str = re.sub(r'type\s+\w+', '', name_str, flags=re.IGNORECASE).strip() 
    name_str = name_str.strip() 
    if not name_str or name_str.lower() in ["hypothetical", "uncharacterized", "unknown", "predicted"]: # Added "predicted"
        return "Unknown"
    return name_str[0].upper() + name_str[1:] if len(name_str) > 0 else "Unknown"

# Color maps (can be expanded or moved to specific cells if only used there)
group_colors = {
    'Asgard': arcadia_primary_palette[0 % len(arcadia_primary_palette)],
    'GV': arcadia_primary_palette[1 % len(arcadia_primary_palette)],
    'Other': arcadia_neutrals_palette[0 % len(arcadia_neutrals_palette)]
}

broad_category_color_map = {}
if 'Broad_Functional_Category' in df_full.columns and not df_full.empty:
    all_broad_categories_cat = df_full['Broad_Functional_Category'].cat.categories.tolist()
    category_assignments = { 
        'Cytoskeleton': arcadia_primary_palette[0], 'Membrane Trafficking/Vesicles': arcadia_primary_palette[1],
        'ESCRT/Endosomal Sorting': arcadia_primary_palette[2], 'Ubiquitin System': arcadia_primary_palette[3],
        'N-glycosylation': arcadia_primary_palette[4], 'Nuclear Transport/Pore': arcadia_primary_palette[5],
        'DNA Info Processing': arcadia_primary_palette[6], 'DNA Processing & Maintenance': arcadia_primary_palette[6], 
        'RNA Info Processing': arcadia_primary_palette[7], 'RNA Processing & Transcription': arcadia_primary_palette[7], 
        'Translation': arcadia_primary_palette[8], 'Translation & Protein Synthesis': arcadia_primary_palette[8], 
        'Signal Transduction': arcadia_primary_palette[9], 'Metabolism': arcadia_primary_palette[10],
        'Other Specific Annotation': arcadia_secondary_palette[0 % len(arcadia_secondary_palette)],
        'Unknown/Unclassified': arcadia_neutrals_palette[0 % len(arcadia_neutrals_palette)],
        'General Protein Features': arcadia_neutrals_palette[1 % len(arcadia_neutrals_palette)],
        'Homologous_superfamily': arcadia_secondary_palette[1 % len(arcadia_secondary_palette)], 
        'Binding_site': arcadia_secondary_palette[2 % len(arcadia_secondary_palette)],
        'Active_site': arcadia_secondary_palette[3 % len(arcadia_secondary_palette)],
    }
    fallback_palette_cat = arcadia_primary_palette + arcadia_secondary_palette + arcadia_neutrals_palette
    fallback_idx_cat = 0
    for category in all_broad_categories_cat:
        if category in category_assignments: 
            broad_category_color_map[category] = category_assignments[category]
        else: 
            broad_category_color_map[category] = fallback_palette_cat[fallback_idx_cat % len(fallback_palette_cat)]
            fallback_idx_cat += 1
print(f"Broad functional category color map created for {len(broad_category_color_map)} categories (if df_full loaded).")


print("\n--- Database Pub Analysis Setup Complete ---")
# df_full is now loaded and cleaned.
# Key variables, helper functions, and color maps are defined.

# Add a final check to ensure df_full is not empty before subsequent cells try to use it.
if 'df_full' in locals() and df_full.empty:
    print("CRITICAL WARNING: df_full is defined but EMPTY after loading and processing. Please check your input CSV, the column definitions, and any filtering logic.")
elif 'df_full' not in locals():
    print("CRITICAL ERROR: df_full was NOT defined in Cell 1. Subsequent cells will fail.")
else:
    print(f"df_full is ready for analysis. Shape: {df_full.shape}")



In [ ]:
# Cell 22: Exploration of Putative Deaminases

# This cell assumes Cell 1 (Setup for Deaminase Exploration) has been run.
# Required variables from Cell 1:
# - df_full (DataFrame with all annotations)
# - DEAMINASE_PLOT_OUTPUT_DIR, DEAMINASE_SUMMARY_DATA_DIR (Path objects for output)
# - protein_id_col, group_col, broad_func_cat_col, specific_func_cat_col, 
#   source_annot_col, ipr_col, euk_hit_protein_name_col, has_euk_hit_col,
#   dark_col, is_esp_col
# - ipr_lookup (dictionary for IPR descriptions)
# - clean_protein_name (helper function)
# - group_colors, broad_category_color_map (color maps)
# - plotly_layout_defaults (implicitly used by pio.templates.default)

print("--- Cell 22: Deaminase Exploration ---")

if 'df_full' not in locals() or df_full.empty:
    raise NameError("DataFrame 'df_full' is not defined or empty. Please run Cell 1 (Setup) first.")

# --- Define Deaminase-Related Keywords and IPR IDs ---
# Keywords are case-insensitive for searching text fields
deaminase_keywords = [
    'deaminase', 'cytidine', 'adenosine', 'cda', 'ada', 'adar', 'apobec', 
    'aid', 'ampo', # APOBEC/AMP deaminase related
    'ctd', # cytidine triphosphate deaminase
    'dcd', # dCMP deaminase
    'monomethylarsonate reductase', # Some can have deaminase activity
    'nitrilase' # Broader family, some members are deaminases
]
# Specific IPR IDs known or suspected to be related to deaminases
# This list can be expanded significantly with more research
deaminase_ipr_ids = {
    'IPR002001', # Cytidine/deoxycytidylate deaminase zinc-binding region
    'IPR018218', # Cytidine deaminase-like
    'IPR000335', # Adenosine deaminase
    'IPR001365', # Adenosine deaminase domain
    'IPR002466', # Adenosine deaminase/editase
    'IPR013659', # Adenosine/AMP deaminase N-terminal
    'IPR006330', # Adenosine/adenine deaminase
    'IPR006329', # AMP deaminase
    'IPR006331', # Adenosine deaminase-related growth factor
    'IPR028893', # Adenisone deaminase
    'IPR042935', # tRNA-specific adenosine deaminase 1
    'IPR012839', # Adenosine deaminase, A.thaliana-like
    'IPR012340', # APOBEC, N-terminal
    'IPR003592', # APOBEC, C-terminal catalytic domain
    'IPR016292', # ADAR, dsRNA adenosine deaminase
    'IPR001907', # Nitrilase/cyanide hydratase and apolipoprotein N-acyltransferase
    'IPR036618', # Deaminase TadA-like domain
    'IPR006250', # dCMP deaminase, N-terminal
    'IPR006251', # dCMP deaminase, C-terminal
    'IPR004118', # Cytidine/deoxycytidylate deaminase family
    'IPR001570', # Phosphoribosyl-AMP cyclohydrolase/phosphoribosyl-ATP pyrophosphatase HisI
    # Add more relevant IPR IDs here
}
print(f"Searching for proteins related to {len(deaminase_keywords)} keywords and {len(deaminase_ipr_ids)} IPR IDs.")

# --- Search for Putative Deaminases ---
# Create boolean masks for each search criterion

# 1. Search in Broad Functional Category
mask_broad_cat = pd.Series(False, index=df_full.index)
if broad_func_cat_col in df_full.columns:
    mask_broad_cat = df_full[broad_func_cat_col].astype(str).str.contains('|'.join(deaminase_keywords), case=False, na=False)
    print(f"Found {mask_broad_cat.sum()} proteins by Broad Functional Category keywords.")

# 2. Search in Specific Functional Category (if it exists and is used differently)
mask_specific_cat = pd.Series(False, index=df_full.index)
if specific_func_cat_col in df_full.columns and specific_func_cat_col != broad_func_cat_col:
    mask_specific_cat = df_full[specific_func_cat_col].astype(str).str.contains('|'.join(deaminase_keywords), case=False, na=False)
    print(f"Found {mask_specific_cat.sum()} proteins by Specific Functional Category keywords.")

# 3. Search in Source Protein Annotation
mask_source_annot = pd.Series(False, index=df_full.index)
if source_annot_col in df_full.columns:
    mask_source_annot = df_full[source_annot_col].astype(str).str.contains('|'.join(deaminase_keywords), case=False, na=False)
    print(f"Found {mask_source_annot.sum()} proteins by Source Annotation keywords.")

# 4. Search in Eukaryotic Hit Protein Name
mask_euk_hit_name = pd.Series(False, index=df_full.index)
if euk_hit_protein_name_col in df_full.columns and has_euk_hit_col in df_full.columns:
    # Apply clean_protein_name before searching
    cleaned_euk_names = df_full[df_full[has_euk_hit_col]][euk_hit_protein_name_col].astype(str).apply(clean_protein_name)
    mask_euk_hit_name.loc[cleaned_euk_names.index] = cleaned_euk_names.str.contains('|'.join(deaminase_keywords), case=False, na=False)
    print(f"Found {mask_euk_hit_name.sum()} proteins by Eukaryotic Hit Protein Name keywords (after cleaning).")

# 5. Search in IPR Signatures (IDs and Descriptions)
mask_ipr = pd.Series(False, index=df_full.index)
if ipr_col in df_full.columns and ipr_lookup:
    ipr_hits_indices = []
    for index, row in df_full.iterrows():
        if pd.notna(row[ipr_col]) and row[ipr_col].strip():
            protein_ipr_ids = set(pid.strip() for pid in row[ipr_col].split(';') if pid.strip())
            # Check direct IPR ID match
            if not protein_ipr_ids.isdisjoint(deaminase_ipr_ids):
                ipr_hits_indices.append(index)
                continue 
            # Check IPR descriptions for keywords
            for ipr_id_in_protein in protein_ipr_ids:
                ipr_data = ipr_lookup.get(ipr_id_in_protein)
                if ipr_data and 'Name' in ipr_data:
                    if any(keyword in ipr_data['Name'].lower() for keyword in deaminase_keywords):
                        ipr_hits_indices.append(index)
                        break # Found a keyword match for this protein, move to next protein
    if ipr_hits_indices:
        mask_ipr.loc[list(set(ipr_hits_indices))] = True # Use set to handle duplicates if a protein hits multiple ways
    print(f"Found {mask_ipr.sum()} proteins by IPR ID match or IPR description keywords.")

# Combine masks: a protein is a putative deaminase if it matches any criterion
combined_mask = mask_broad_cat | mask_specific_cat | mask_source_annot | mask_euk_hit_name | mask_ipr
df_deaminases = df_full[combined_mask].copy()
# Add a column indicating how it was identified (for review)
df_deaminases['Deaminase_Evidence'] = ''
if mask_broad_cat.any(): df_deaminases.loc[mask_broad_cat[combined_mask], 'Deaminase_Evidence'] += 'BroadFuncCat;'
if mask_specific_cat.any(): df_deaminases.loc[mask_specific_cat[combined_mask], 'Deaminase_Evidence'] += 'SpecificFuncCat;'
if mask_source_annot.any(): df_deaminases.loc[mask_source_annot[combined_mask], 'Deaminase_Evidence'] += 'SourceAnnot;'
if mask_euk_hit_name.any(): df_deaminases.loc[mask_euk_hit_name[combined_mask], 'Deaminase_Evidence'] += 'EukHitName;'
if mask_ipr.any(): df_deaminases.loc[mask_ipr[combined_mask], 'Deaminase_Evidence'] += 'IPR;'
df_deaminases['Deaminase_Evidence'] = df_deaminases['Deaminase_Evidence'].str.rstrip(';')


n_putative_deaminases = len(df_deaminases)
print(f"\n--- Found {n_putative_deaminases} Putative Deaminase Proteins ---")

if n_putative_deaminases > 0:
    # --- Characterize Putative Deaminases ---
    print("\n--- Characterization of Putative Deaminases ---")

    # 1. Distribution by Group (Asgard vs GV)
    if group_col in df_deaminases.columns:
        group_counts = df_deaminases[group_col].value_counts().reset_index()
        group_counts.columns = ['Group', 'Count']
        print("\nDistribution by Group (Asgard/GV):")
        try: print(group_counts.to_markdown(index=False))
        except ImportError: print(group_counts)
        fig_group = px.bar(group_counts, x='Group', y='Count', color='Group', color_discrete_map=group_colors, title="Putative Deaminases by Group")
        fig_group.show()
        fig_group.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "putative_deaminases_by_group.html")

    # 2. Broad Functional Category of these proteins
    if broad_func_cat_col in df_deaminases.columns:
        func_cat_counts = df_deaminases[broad_func_cat_col].value_counts().nlargest(15).reset_index() # Top 15
        func_cat_counts.columns = [broad_func_cat_col, 'Count']
        print(f"\nTop Broad Functional Categories of Putative Deaminases:")
        try: print(func_cat_counts.to_markdown(index=False))
        except ImportError: print(func_cat_counts)
        fig_func = px.bar(func_cat_counts, x='Count', y=broad_func_cat_col, orientation='h', color=broad_func_cat_col, color_discrete_map=broad_category_color_map,
                          title="Functional Categories of Putative Deaminases")
        fig_func.update_layout(showlegend=False, yaxis_categoryorder='total ascending', height=max(400, len(func_cat_counts)*25))
        fig_func.show()
        fig_func.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "putative_deaminases_functional_categories.html")

    # 3. Eukaryotic Hits for these deaminases
    if has_euk_hit_col in df_deaminases.columns:
        df_deaminases_with_euk_hits = df_deaminases[df_deaminases[has_euk_hit_col] == True].copy()
        print(f"\n{len(df_deaminases_with_euk_hits)} of these putative deaminases have a eukaryotic DIAMOND hit.")
        if not df_deaminases_with_euk_hits.empty:
            # Top Euk Organisms hit by these deaminases
            euk_org_hit_counts = df_deaminases_with_euk_hits[euk_hit_organism_col].value_counts().nlargest(15).reset_index()
            euk_org_hit_counts.columns = [euk_hit_organism_col, 'Count']
            print(f"\nTop Eukaryotic Organisms Hit by Putative Deaminases:")
            try: print(euk_org_hit_counts.to_markdown(index=False))
            except ImportError: print(euk_org_hit_counts)
            fig_euk_org = px.bar(euk_org_hit_counts, x='Count', y=euk_hit_organism_col, orientation='h', title="Top Euk. Organisms Hit by Putative Deaminases")
            fig_euk_org.update_layout(yaxis_categoryorder='total ascending', height=max(400, len(euk_org_hit_counts)*20))
            fig_euk_org.show()
            fig_euk_org.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "putative_deaminases_euk_org_hits.html")

            # Top Euk Protein Names hit by these deaminases
            if euk_hit_protein_name_col in df_deaminases_with_euk_hits.columns:
                df_deaminases_with_euk_hits['Cleaned_Euk_Hit_Name'] = df_deaminases_with_euk_hits[euk_hit_protein_name_col].apply(clean_protein_name)
                euk_prot_name_counts = df_deaminases_with_euk_hits['Cleaned_Euk_Hit_Name'].value_counts().nlargest(20).reset_index()
                euk_prot_name_counts.columns = ['Euk_Protein_Function', 'Count']
                print(f"\nTop Eukaryotic Protein Functions Hit by Putative Deaminases:")
                try: print(euk_prot_name_counts.to_markdown(index=False))
                except ImportError: print(euk_prot_name_counts)
                fig_euk_func = px.bar(euk_prot_name_counts, x='Count', y='Euk_Protein_Function', orientation='h', title="Top Euk. Protein Functions Hit by Putative Deaminases")
                fig_euk_func.update_layout(yaxis_categoryorder='total ascending', height=max(400, len(euk_prot_name_counts)*20))
                fig_euk_func.show()
                fig_euk_func.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "putative_deaminases_euk_func_hits.html")

    # 4. Structural Darkness
    if dark_col in df_deaminases.columns:
        dark_counts = df_deaminases[dark_col].value_counts(normalize=True).mul(100).reset_index()
        dark_counts.columns = ['Is_Structurally_Dark', 'Percentage']
        print(f"\nStructural Darkness of Putative Deaminases:")
        try: print(dark_counts.to_markdown(index=False, floatfmt=".1f"))
        except ImportError: print(dark_counts)
        fig_dark = px.pie(dark_counts, names='Is_Structurally_Dark', values='Percentage', title="Structural Darkness of Putative Deaminases")
        fig_dark.show()
        fig_dark.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "putative_deaminases_structural_darkness.html")

    # --- Save the identified deaminases to a CSV ---
    output_csv_path = DEAMINASE_SUMMARY_DATA_DIR / "putative_deaminase_proteins.csv"
    try:
        # Select a subset of relevant columns for the output
        cols_to_save_deaminase = [
            protein_id_col, group_col, og_col, virus_name_col, asgard_phylum_col,
            broad_func_cat_col, specific_func_cat_col, source_annot_col, ipr_col,
            'Deaminase_Evidence', dark_col, is_esp_col,
            has_euk_hit_col, euk_hit_sseqid_col, euk_hit_organism_col, euk_hit_protein_name_col,
            euk_hit_pident_col, euk_hit_evalue_col, query_coverage_col, subject_coverage_col,
            length_col, disorder_col, localization_col, sp_type_col, sequence_col
        ]
        # Filter for columns that actually exist in df_deaminases
        existing_cols_to_save = [col for col in cols_to_save_deaminase if col in df_deaminases.columns]
        df_deaminases[existing_cols_to_save].to_csv(output_csv_path, index=False, na_rep='NA')
        print(f"\nSaved details of {n_putative_deaminases} putative deaminases to: {output_csv_path}")
    except Exception as e:
        print(f"ERROR: Could not save the putative deaminases CSV. Error: {e}")
else:
    print("No putative deaminases found based on the defined criteria.")

print("\n\n--- Cell 22: Deaminase Exploration Complete ---")



In [ ]:
# Cell 23: Create Adenosine Deaminase Sub-database

# This cell assumes Cell 1 (Setup for Deaminase Exploration) has been run for helper functions and ipr_lookup.
# It also assumes Cell 22 has been run to generate "putative_deaminase_proteins.csv".
# Required variables from Cell 1:
# - DEAMINASE_SUMMARY_DATA_DIR (Path object for input/output)
# - protein_id_col, group_col, broad_func_cat_col, specific_func_cat_col, 
#   source_annot_col, ipr_col, euk_hit_protein_name_col, has_euk_hit_col,
#   virus_name_col, asgard_phylum_col, dark_col, is_esp_col,
#   euk_hit_sseqid_col, euk_hit_organism_col, euk_hit_pident_col, euk_hit_evalue_col,
#   query_coverage_col, subject_coverage_col, length_col, disorder_col, localization_col,
#   sp_type_col, sequence_col
# - ipr_lookup (dictionary for IPR descriptions)
# - clean_protein_name (helper function)

print("--- Cell 23: Creating Adenosine Deaminase Sub-database ---")

if 'DEAMINASE_SUMMARY_DATA_DIR' not in locals():
    raise NameError("DEAMINASE_SUMMARY_DATA_DIR is not defined. Please run Cell 1 and Cell 22 setup first.")
if 'ipr_lookup' not in locals():
    print("WARNING: ipr_lookup not found. IPR descriptions might be limited.")
    ipr_lookup = {} # Define as empty to prevent errors, though descriptions will be missing
if 'clean_protein_name' not in locals():
    raise NameError("Helper function 'clean_protein_name' is not defined. Please run Cell 1 setup.")


# --- Configuration ---
PUTATIVE_DEAMINASES_CSV_PATH = DEAMINASE_SUMMARY_DATA_DIR / "putative_deaminase_proteins.csv"
OUTPUT_ADENOSINE_DEAMINASE_CSV_PATH = DEAMINASE_SUMMARY_DATA_DIR / "adenosine_deaminase_sub_database.csv"

# --- Define Adenosine Deaminase-Specific Keywords and IPR IDs ---
adenosine_deaminase_keywords = [
    'adenosine deaminase', 'adar', 'tad', 'adenosine aminohydrolase',
    'adenine deaminase', 'amp deaminase' # AMP deaminase is related
]
# More specific IPRs for Adenosine Deaminases
adenosine_deaminase_ipr_ids = {
    'IPR000335', # Adenosine deaminase
    'IPR013659', # Adenosine/AMP deaminase N-terminal domain
    'IPR006330', # Adenosine/adenine deaminase catalytic domain
    'IPR006329', # AMP deaminase
    'IPR001365', # Adenosine deaminase domain
    'IPR002466', # Adenosine deaminase/editase
    'IPR013659', # Adenosine/AMP deaminase N-terminal
    'IPR006330', # Adenosine/adenine deaminase
    'IPR006329', # AMP deaminase
    'IPR006331', # Adenosine deaminase-related growth factor
    'IPR028893', # Adenisone deaminase
    'IPR042935', # tRNA-specific adenosine deaminase 1
    'IPR012839', # Adenosine deaminase, A.thaliana-like
    'IPR016292', # ADAR, dsRNA adenosine deaminase
    'IPR042935', # tRNA-specific adenosine deaminase 1
    'IPR012839', # Adenosine deaminase, A.thaliana-like
    'IPR016292', # ADAR, dsRNA adenosine deaminase catalytic domain
    'IPR036618', # Deaminase TadA-like domain (TadA is tRNA adenosine deaminase)
    'IPR002466', # Adenosine deaminase/editase domain
    'IPR001365', # Adenosine deaminase domain
    # Consider adding IPRs for ADATs (tRNA-specific adenosine deaminases) if relevant
}
print(f"Searching for adenosine deaminases using {len(adenosine_deaminase_keywords)} keywords and {len(adenosine_deaminase_ipr_ids)} IPR IDs.")

# --- Load the Putative Deaminase Database ---
print(f"\n--- Loading Putative Deaminase Data from: {PUTATIVE_DEAMINASES_CSV_PATH} ---")
if not PUTATIVE_DEAMINASES_CSV_PATH.is_file():
    raise FileNotFoundError(f"ERROR: Putative deaminases CSV file not found at '{PUTATIVE_DEAMINASES_CSV_PATH}'. Please run Cell 22 first.")

try:
    df_putative_deaminases = pd.read_csv(PUTATIVE_DEAMINASES_CSV_PATH, low_memory=False)
    print(f"Successfully loaded {len(df_putative_deaminases)} putative deaminases.")
    # Ensure key columns are present and fill NaNs for string columns used in search
    for col in [broad_func_cat_col, specific_func_cat_col, source_annot_col, ipr_col, euk_hit_protein_name_col, 'Deaminase_Evidence']:
        if col in df_putative_deaminases.columns:
            df_putative_deaminases[col] = df_putative_deaminases[col].fillna('').astype(str)
        else:
            print(f"Warning: Column '{col}' not found in loaded putative deaminases. Search in this field will be skipped.")
            df_putative_deaminases[col] = '' # Add as empty string to prevent errors
    if has_euk_hit_col in df_putative_deaminases.columns:
         df_putative_deaminases[has_euk_hit_col] = df_putative_deaminases[has_euk_hit_col].fillna(False).astype(bool)
    else:
        print(f"Warning: Column '{has_euk_hit_col}' not found. Assuming no euk hits for filtering.")
        df_putative_deaminases[has_euk_hit_col] = False


except Exception as e:
    print(f"Error loading '{PUTATIVE_DEAMINASES_CSV_PATH}': {e}")
    raise

# --- Filter for Adenosine Deaminases ---
print("\n--- Filtering for Adenosine Deaminases ---")

# 1. Search in Broad Functional Category
mask_ad_broad_cat = df_putative_deaminases[broad_func_cat_col].str.contains('|'.join(adenosine_deaminase_keywords), case=False, na=False)

# 2. Search in Specific Functional Category
mask_ad_specific_cat = pd.Series(False, index=df_putative_deaminases.index)
if specific_func_cat_col in df_putative_deaminases.columns:
    mask_ad_specific_cat = df_putative_deaminases[specific_func_cat_col].str.contains('|'.join(adenosine_deaminase_keywords), case=False, na=False)

# 3. Search in Source Protein Annotation
mask_ad_source_annot = df_putative_deaminases[source_annot_col].str.contains('|'.join(adenosine_deaminase_keywords), case=False, na=False)

# 4. Search in Eukaryotic Hit Protein Name (if hit exists)
mask_ad_euk_hit_name = pd.Series(False, index=df_putative_deaminases.index)
if euk_hit_protein_name_col in df_putative_deaminases.columns and has_euk_hit_col in df_putative_deaminases.columns:
    # Apply clean_protein_name before searching
    idx_with_euk_hits = df_putative_deaminases[df_putative_deaminases[has_euk_hit_col]].index
    if not idx_with_euk_hits.empty:
        cleaned_euk_names_ad = df_putative_deaminases.loc[idx_with_euk_hits, euk_hit_protein_name_col].astype(str).apply(clean_protein_name)
        mask_ad_euk_hit_name.loc[idx_with_euk_hits] = cleaned_euk_names_ad.str.contains('|'.join(adenosine_deaminase_keywords), case=False, na=False)

# 5. Search in IPR Signatures (IDs and Descriptions)
mask_ad_ipr = pd.Series(False, index=df_putative_deaminases.index)
if ipr_col in df_putative_deaminases.columns:
    ipr_hits_indices_ad = []
    for index, row in df_putative_deaminases.iterrows():
        if row[ipr_col].strip(): # Check if not empty
            protein_ipr_ids = set(pid.strip() for pid in row[ipr_col].split(';') if pid.strip())
            if not protein_ipr_ids.isdisjoint(adenosine_deaminase_ipr_ids):
                ipr_hits_indices_ad.append(index)
                continue
            for ipr_id_in_protein in protein_ipr_ids:
                ipr_data = ipr_lookup.get(ipr_id_in_protein)
                if ipr_data and 'Name' in ipr_data:
                    if any(keyword in ipr_data['Name'].lower() for keyword in adenosine_deaminase_keywords):
                        ipr_hits_indices_ad.append(index)
                        break 
    if ipr_hits_indices_ad:
        mask_ad_ipr.loc[list(set(ipr_hits_indices_ad))] = True

# Combine masks
combined_ad_mask = mask_ad_broad_cat | mask_ad_specific_cat | mask_ad_source_annot | mask_ad_euk_hit_name | mask_ad_ipr
df_adenosine_deaminases = df_putative_deaminases[combined_ad_mask].copy()

# Add a column for adenosine deaminase specific evidence
df_adenosine_deaminases['Adenosine_Deaminase_Evidence'] = ''
if mask_ad_broad_cat.any(): df_adenosine_deaminases.loc[mask_ad_broad_cat[combined_ad_mask], 'Adenosine_Deaminase_Evidence'] += 'AD_BroadFuncCat;'
if mask_ad_specific_cat.any(): df_adenosine_deaminases.loc[mask_ad_specific_cat[combined_ad_mask], 'Adenosine_Deaminase_Evidence'] += 'AD_SpecificFuncCat;'
if mask_ad_source_annot.any(): df_adenosine_deaminases.loc[mask_ad_source_annot[combined_ad_mask], 'Adenosine_Deaminase_Evidence'] += 'AD_SourceAnnot;'
if mask_ad_euk_hit_name.any(): df_adenosine_deaminases.loc[mask_ad_euk_hit_name[combined_ad_mask], 'Adenosine_Deaminase_Evidence'] += 'AD_EukHitName;'
if mask_ad_ipr.any(): df_adenosine_deaminases.loc[mask_ad_ipr[combined_ad_mask], 'Adenosine_Deaminase_Evidence'] += 'AD_IPR;'
df_adenosine_deaminases['Adenosine_Deaminase_Evidence'] = df_adenosine_deaminases['Adenosine_Deaminase_Evidence'].str.rstrip(';')


n_adenosine_deaminases = len(df_adenosine_deaminases)
print(f"\n--- Found {n_adenosine_deaminases} Putative Adenosine Deaminase Proteins ---")

if n_adenosine_deaminases > 0:
    # --- Select Columns for the Sub-database ---
    # These columns should mostly exist in putative_deaminase_proteins.csv
    # The 'Deaminase_Evidence' column is from the broader search, 'Adenosine_Deaminase_Evidence' is specific.
    sub_db_cols = [
        protein_id_col, group_col, og_col, virus_name_col, asgard_phylum_col,
        source_annot_col, broad_func_cat_col, specific_func_cat_col, 
        ipr_col, 'Deaminase_Evidence', 'Adenosine_Deaminase_Evidence', 
        dark_col, is_esp_col,
        has_euk_hit_col, euk_hit_sseqid_col, euk_hit_organism_col, euk_hit_protein_name_col,
        euk_hit_pident_col, euk_hit_evalue_col, query_coverage_col, subject_coverage_col,
        length_col, disorder_col, localization_col, sp_type_col,
        sequence_col # Include sequence if present and desired
    ]
    # Filter for columns that actually exist in the df_adenosine_deaminases DataFrame
    existing_sub_db_cols = [col for col in sub_db_cols if col in df_adenosine_deaminases.columns]
    df_adenosine_deaminase_sub_db = df_adenosine_deaminases[existing_sub_db_cols]

    # --- Save the Sub-database ---
    try:
        df_adenosine_deaminase_sub_db.to_csv(OUTPUT_ADENOSINE_DEAMINASE_CSV_PATH, index=False, na_rep='NA')
        print(f"\nSuccessfully saved adenosine deaminase sub-database ({len(df_adenosine_deaminase_sub_db)} entries) to: {OUTPUT_ADENOSINE_DEAMINASE_CSV_PATH}")
        
        print("\nFirst 5 rows of the Adenosine Deaminase Sub-database:")
        try:
            print(df_adenosine_deaminase_sub_db.head().to_markdown(index=False, floatfmt=".2f"))
        except ImportError:
            print(df_adenosine_deaminase_sub_db.head())
            
    except Exception as e:
        print(f"ERROR: Could not save the adenosine deaminase sub-database. Error: {e}")
else:
    print("No putative adenosine deaminases found based on the specific criteria.")

print("\n\n--- Cell 23: Adenosine Deaminase Sub-database Creation Complete ---")



In [ ]:
# Cell 24: Comprehensive Visualization of the Adenosine Deaminase Sub-database (with SVG Export)

# This cell assumes Cell 1 (Setup for Deaminase Exploration) has been run.
# Required variables from Cell 1:
# - DEAMINASE_PLOT_OUTPUT_DIR, DEAMINASE_SUMMARY_DATA_DIR
# - protein_id_col, group_col, broad_func_cat_col, specific_func_cat_col, 
#   source_annot_col, ipr_col, euk_hit_protein_name_col, has_euk_hit_col,
#   dark_col, is_esp_col, length_col, disorder_col, localization_col,
#   euk_hit_organism_col, euk_hit_pident_col, query_coverage_col, subject_coverage_col
# - ipr_lookup, clean_protein_name, group_colors, broad_category_color_map, 
#   arcadia_primary_palette, arcadia_secondary_palette, arcadia_neutrals_palette
# - plotly_layout_defaults (implicitly used by pio.templates.default)

print("--- Cell 24: Visualizing the Adenosine Deaminase Sub-database (with SVG Export) ---")

if 'DEAMINASE_SUMMARY_DATA_DIR' not in locals():
    raise NameError("DEAMINASE_SUMMARY_DATA_DIR is not defined. Please run Cell 1 setup first.")
if 'ipr_lookup' not in locals():
    print("WARNING: ipr_lookup not found. IPR descriptions might be limited.")
    ipr_lookup = {} 
if 'clean_protein_name' not in locals():
    raise NameError("Helper function 'clean_protein_name' is not defined. Please run Cell 1 setup.")
if 'DEAMINASE_PLOT_OUTPUT_DIR' not in locals(): # Ensure plot output dir is defined
    raise NameError("DEAMINASE_PLOT_OUTPUT_DIR is not defined. Please run Cell 1 setup first.")


# --- Configuration ---
ADENOSINE_DEAMINASE_SUB_DB_PATH = DEAMINASE_SUMMARY_DATA_DIR / "adenosine_deaminase_sub_database.csv"
TOP_N_DISPLAY = 15 # For bar charts

# --- Load the Adenosine Deaminase Sub-database ---
print(f"\n--- Loading Adenosine Deaminase Sub-database from: {ADENOSINE_DEAMINASE_SUB_DB_PATH} ---")
if not ADENOSINE_DEAMINASE_SUB_DB_PATH.is_file():
    raise FileNotFoundError(f"ERROR: Adenosine Deaminase sub-database file not found at '{ADENOSINE_DEAMINASE_SUB_DB_PATH}'. Please run Cell 23 first.")

try:
    df_ad = pd.read_csv(ADENOSINE_DEAMINASE_SUB_DB_PATH, low_memory=False)
    print(f"Successfully loaded {len(df_ad)} putative adenosine deaminases.")
    # Basic type conversions for safety
    for col in [group_col, broad_func_cat_col, specific_func_cat_col, 'Adenosine_Deaminase_Evidence', localization_col, euk_hit_organism_col]:
        if col in df_ad.columns:
            df_ad[col] = df_ad[col].fillna('Unknown').astype('category')
    for col in [ipr_col, source_annot_col, euk_hit_protein_name_col]:
        if col in df_ad.columns:
            df_ad[col] = df_ad[col].fillna('').astype(str)
    for col in [has_euk_hit_col, dark_col, is_esp_col]:
        if col in df_ad.columns:
            df_ad[col] = df_ad[col].fillna(False).astype(bool)
    for col in [length_col, disorder_col, euk_hit_pident_col, query_coverage_col, subject_coverage_col]:
        if col in df_ad.columns:
            df_ad[col] = pd.to_numeric(df_ad[col], errors='coerce')

except Exception as e:
    print(f"Error loading '{ADENOSINE_DEAMINASE_SUB_DB_PATH}': {e}")
    raise

if df_ad.empty:
    print("Adenosine deaminase sub-database is empty. No visualizations will be generated.")
else:
    # --- 1. Overall Summary ---
    print(f"\n--- 1. Overall Summary of {len(df_ad)} Putative Adenosine Deaminases ---")

    # A. Distribution by Group (Asgard vs GV)
    if group_col in df_ad.columns:
        ad_group_counts = df_ad[group_col].value_counts().reset_index()
        ad_group_counts.columns = ['Group', 'Count']
        print("\nDistribution by Group:")
        try: print(ad_group_counts.to_markdown(index=False))
        except ImportError: print(ad_group_counts)
        fig_ad_group = px.bar(ad_group_counts, x='Group', y='Count', color='Group', 
                              color_discrete_map=group_colors, title="Adenosine Deaminases by Group")
        fig_ad_group.show()
        fig_ad_group.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_by_group.html")
        try:
            fig_ad_group.write_image(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_by_group.svg")
            print(f"SVG saved: ad_deaminase_by_group.svg")
        except Exception as e:
            print(f"Warning: Could not export ad_deaminase_by_group to SVG. Ensure 'kaleido' is installed. Error: {e}")


    # B. Distribution by Adenosine_Deaminase_Evidence
    if 'Adenosine_Deaminase_Evidence' in df_ad.columns:
        s = df_ad['Adenosine_Deaminase_Evidence'].str.split(';').explode()
        evidence_counts = s.value_counts().nlargest(TOP_N_DISPLAY).reset_index()
        evidence_counts.columns = ['Evidence_Type', 'Count']
        print("\nDistribution by Identification Evidence:")
        try: print(evidence_counts.to_markdown(index=False))
        except ImportError: print(evidence_counts)
        fig_ad_evidence = px.bar(evidence_counts, x='Count', y='Evidence_Type', orientation='h',
                                 title="Adenosine Deaminase Identification Evidence")
        fig_ad_evidence.update_layout(yaxis_categoryorder='total ascending', height=max(300, len(evidence_counts)*25))
        fig_ad_evidence.show()
        fig_ad_evidence.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_by_evidence.html")
        try:
            fig_ad_evidence.write_image(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_by_evidence.svg")
            print(f"SVG saved: ad_deaminase_by_evidence.svg")
        except Exception as e:
            print(f"Warning: Could not export ad_deaminase_by_evidence to SVG. Error: {e}")

    # --- 2. Functional Characterization ---
    print(f"\n--- 2. Functional Characterization ---")
    # A. Broad Functional Categories
    if broad_func_cat_col in df_ad.columns:
        ad_func_cat_counts = df_ad[broad_func_cat_col].value_counts().nlargest(TOP_N_DISPLAY).reset_index()
        ad_func_cat_counts.columns = [broad_func_cat_col, 'Count']
        print(f"\nTop Broad Functional Categories:")
        try: print(ad_func_cat_counts.to_markdown(index=False))
        except ImportError: print(ad_func_cat_counts)
        fig_ad_func = px.bar(ad_func_cat_counts, x='Count', y=broad_func_cat_col, orientation='h',
                             color=broad_func_cat_col, color_discrete_map=broad_category_color_map,
                             title="Broad Functional Categories of Adenosine Deaminases")
        fig_ad_func.update_layout(showlegend=False, yaxis_categoryorder='total ascending', height=max(400, len(ad_func_cat_counts)*25))
        fig_ad_func.show()
        fig_ad_func.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_broad_func_cat.html")
        try:
            fig_ad_func.write_image(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_broad_func_cat.svg")
            print(f"SVG saved: ad_deaminase_broad_func_cat.svg")
        except Exception as e:
            print(f"Warning: Could not export ad_deaminase_broad_func_cat to SVG. Error: {e}")

    # B. Top IPR Signatures
    if ipr_col in df_ad.columns and ipr_lookup:
        all_iprs_ad = []
        for ipr_string in df_ad[ipr_col].dropna():
            all_iprs_ad.extend([ipr.strip() for ipr in ipr_string.split(';') if ipr.strip()])
        ipr_counts_ad = Counter(all_iprs_ad)
        top_iprs_data = []
        for ipr_id, count in ipr_counts_ad.most_common(TOP_N_DISPLAY):
            desc = ipr_lookup.get(ipr_id, {}).get('Name', 'Unknown IPR')
            desc = desc[:60] + '...' if len(desc) > 63 else desc
            top_iprs_data.append({'IPR_Domain': f"{desc} ({ipr_id})", 'Count': count, 'IPR_ID': ipr_id})
        if top_iprs_data:
            df_top_iprs_ad = pd.DataFrame(top_iprs_data)
            print(f"\nTop IPR Domains/Families in Adenosine Deaminases:")
            try: print(df_top_iprs_ad[['IPR_Domain', 'Count']].to_markdown(index=False))
            except ImportError: print(df_top_iprs_ad[['IPR_Domain', 'Count']])
            fig_ad_ipr = px.bar(df_top_iprs_ad, x='Count', y='IPR_Domain', orientation='h',
                                title="Top IPR Domains in Putative Adenosine Deaminases")
            fig_ad_ipr.update_layout(yaxis_categoryorder='total ascending', height=max(400, len(df_top_iprs_ad)*25))
            fig_ad_ipr.show()
            fig_ad_ipr.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_top_ipr.html")
            try:
                fig_ad_ipr.write_image(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_top_ipr.svg")
                print(f"SVG saved: ad_deaminase_top_ipr.svg")
            except Exception as e:
                print(f"Warning: Could not export ad_deaminase_top_ipr to SVG. Error: {e}")

    # --- 3. Eukaryotic Homology ---
    print(f"\n--- 3. Eukaryotic Homology of Adenosine Deaminases ---")
    if has_euk_hit_col in df_ad.columns:
        df_ad_with_euk_hits = df_ad[df_ad[has_euk_hit_col] == True].copy()
        n_ad_with_euk_hits = len(df_ad_with_euk_hits)
        print(f"{n_ad_with_euk_hits} of {len(df_ad)} putative adenosine deaminases have a eukaryotic DIAMOND hit.")
        if n_ad_with_euk_hits > 0:
            ad_euk_org_counts = df_ad_with_euk_hits[euk_hit_organism_col].value_counts().nlargest(TOP_N_DISPLAY).reset_index()
            ad_euk_org_counts.columns = [euk_hit_organism_col, 'Count']
            fig_ad_euk_org = px.bar(ad_euk_org_counts, x='Count', y=euk_hit_organism_col, orientation='h', 
                                    title="Top Euk. Organisms Hit by Adenosine Deaminases")
            fig_ad_euk_org.update_layout(yaxis_categoryorder='total ascending', height=max(400, len(ad_euk_org_counts)*20))
            fig_ad_euk_org.show()
            fig_ad_euk_org.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_euk_org_hits.html")
            try:
                fig_ad_euk_org.write_image(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_euk_org_hits.svg")
                print(f"SVG saved: ad_deaminase_euk_org_hits.svg")
            except Exception as e:
                print(f"Warning: Could not export ad_deaminase_euk_org_hits to SVG. Error: {e}")

            if euk_hit_protein_name_col in df_ad_with_euk_hits.columns:
                df_ad_with_euk_hits['Cleaned_Euk_Hit_Name_AD'] = df_ad_with_euk_hits[euk_hit_protein_name_col].apply(clean_protein_name)
                ad_euk_prot_counts = df_ad_with_euk_hits['Cleaned_Euk_Hit_Name_AD'].value_counts().nlargest(TOP_N_DISPLAY).reset_index()
                ad_euk_prot_counts.columns = ['Euk_Protein_Function', 'Count']
                fig_ad_euk_func = px.bar(ad_euk_prot_counts, x='Count', y='Euk_Protein_Function', orientation='h',
                                         title="Top Euk. Protein Functions Hit by Adenosine Deaminases")
                fig_ad_euk_func.update_layout(yaxis_categoryorder='total ascending', height=max(400, len(ad_euk_prot_counts)*20))
                fig_ad_euk_func.show()
                fig_ad_euk_func.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_euk_func_hits.html")
                try:
                    fig_ad_euk_func.write_image(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_euk_func_hits.svg")
                    print(f"SVG saved: ad_deaminase_euk_func_hits.svg")
                except Exception as e:
                    print(f"Warning: Could not export ad_deaminase_euk_func_hits to SVG. Error: {e}")

            hit_quality_cols = {euk_hit_pident_col: 'Euk Hit PIDENT (%)', query_coverage_col: 'Query Coverage', subject_coverage_col: 'Subject Coverage'}
            for col, label in hit_quality_cols.items():
                if col in df_ad_with_euk_hits.columns:
                    fig_qual_filename_base = f"ad_deaminase_euk_hit_{col.replace('(', '').replace(')', '').replace('%', 'pct').replace(' ', '_').lower()}_dist"
                    fig_qual = px.histogram(df_ad_with_euk_hits.dropna(subset=[col]), x=col, color=group_col,
                                            marginal="box", barmode="overlay", title=f"Distribution of {label} for Adenosine Deaminase Euk. Hits")
                    fig_qual.update_traces(opacity=0.75)
                    fig_qual.show()
                    fig_qual.write_html(DEAMINASE_PLOT_OUTPUT_DIR / f"{fig_qual_filename_base}.html")
                    try:
                        fig_qual.write_image(DEAMINASE_PLOT_OUTPUT_DIR / f"{fig_qual_filename_base}.svg")
                        print(f"SVG saved: {fig_qual_filename_base}.svg")
                    except Exception as e:
                        print(f"Warning: Could not export {fig_qual_filename_base} to SVG. Error: {e}")
    else:
        print("No adenosine deaminases found with eukaryotic hits.")

    # --- 4. Other Protein Features ---
    print(f"\n--- 4. Other Features of Putative Adenosine Deaminases ---")
    if dark_col in df_ad.columns:
        ad_dark_counts = df_ad[dark_col].value_counts(normalize=True).mul(100).reset_index()
        ad_dark_counts.columns = ['Is_Structurally_Dark', 'Percentage']
        fig_ad_dark = px.pie(ad_dark_counts, names='Is_Structurally_Dark', values='Percentage', title="Structural Darkness of Adenosine Deaminases")
        fig_ad_dark.show()
        fig_ad_dark.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_structural_darkness.html")
        try:
            fig_ad_dark.write_image(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_structural_darkness.svg")
            print(f"SVG saved: ad_deaminase_structural_darkness.svg")
        except Exception as e:
            print(f"Warning: Could not export ad_deaminase_structural_darkness to SVG. Error: {e}")

    if is_esp_col in df_ad.columns and group_col in df_ad.columns:
        df_ad_asgard = df_ad[df_ad[group_col] == 'Asgard']
        if not df_ad_asgard.empty:
            ad_esp_counts = df_ad_asgard[is_esp_col].value_counts().reset_index()
            ad_esp_counts.columns = ['Is_ESP', 'Count']
            fig_ad_esp = px.bar(ad_esp_counts, x='Is_ESP', y='Count', color='Is_ESP', title="ESP Status of Asgard Adenosine Deaminases")
            fig_ad_esp.show()
            fig_ad_esp.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_asgard_esp_status.html")
            try:
                fig_ad_esp.write_image(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_asgard_esp_status.svg")
                print(f"SVG saved: ad_deaminase_asgard_esp_status.svg")
            except Exception as e:
                print(f"Warning: Could not export ad_deaminase_asgard_esp_status to SVG. Error: {e}")

    if length_col in df_ad.columns:
        fig_ad_len = px.histogram(df_ad.dropna(subset=[length_col]), x=length_col, color=group_col, marginal="box",
                                  title="Length Distribution of Adenosine Deaminases")
        fig_ad_len.show()
        fig_ad_len.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_length_distribution.html")
        try:
            fig_ad_len.write_image(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_length_distribution.svg")
            print(f"SVG saved: ad_deaminase_length_distribution.svg")
        except Exception as e:
            print(f"Warning: Could not export ad_deaminase_length_distribution to SVG. Error: {e}")

    if disorder_col in df_ad.columns:
        fig_ad_dis = px.histogram(df_ad.dropna(subset=[disorder_col]), x=disorder_col, color=group_col, marginal="box",
                                   title="Disorder (%) Distribution of Adenosine Deaminases")
        fig_ad_dis.show()
        fig_ad_dis.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_disorder_distribution.html")
        try:
            fig_ad_dis.write_image(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_disorder_distribution.svg")
            print(f"SVG saved: ad_deaminase_disorder_distribution.svg")
        except Exception as e:
            print(f"Warning: Could not export ad_deaminase_disorder_distribution to SVG. Error: {e}")

    if localization_col in df_ad.columns:
        ad_loc_counts = df_ad.groupby(group_col)[localization_col].value_counts(normalize=True).mul(100).rename('Percentage').reset_index()
        fig_ad_loc = px.bar(ad_loc_counts, x=localization_col, y='Percentage', color=group_col, barmode='group',
                            title="Predicted Subcellular Localization of Adenosine Deaminases",
                            category_orders={localization_col: sorted(df_ad[localization_col].cat.categories.tolist())}) 
        fig_ad_loc.update_xaxes(tickangle=30)
        fig_ad_loc.show()
        fig_ad_loc.write_html(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_localization.html")
        try:
            fig_ad_loc.write_image(DEAMINASE_PLOT_OUTPUT_DIR / "ad_deaminase_localization.svg")
            print(f"SVG saved: ad_deaminase_localization.svg")
        except Exception as e:
            print(f"Warning: Could not export ad_deaminase_localization to SVG. Error: {e}")

print("\n\n--- Cell 24: Adenosine Deaminase Sub-database Visualization (with SVG Export) Complete ---")

